# AInewsbot.ipynb - Automate collecting daily AI news

1. initial scrape of front pages of tech sites
  - Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
  - Save HTML of each URL in htmldata directory  
  - Extract URLs from all files, create a pandas dataframe with url, title, src

2. Filter and clean to AI-related headlines not seen before
  - Use ChatGPT prompt to filter only AI-related headlines by sending a prompt and formatted table of headlines
  - Use SQLite to filter headlines previously seen 
  - remove duplicate URLs and headlines
  - ensure there are pretty source names for each news site

3. Topic analysis, make a list of topics for each headline
  - using a prompt, check each headline against a number of evergreen AI topics, e.g. deepfakes, regulation, AI in education
  - extract free from topics from each headline
  - combine topics into topic list for each headline
  - cluster headlines using dimensionality-reduced embeddings and DBSCAN; ask chatgpt to name each cluster
  - sort headlines by doing a traveling salesman shortest traversal in embedding space

4. Summarize individual news story pages in 3 bullets using a prompt

5. create a large markdown file with all bullet points and topics

6. give the markdown file to ChatGPT and ask it to make a list of most popular and import topics of the day

7. human should make a list of the day's topics, combining the chatgpt response to the quesion and cluster topics and 

8. Put summaries in vector store along with metadata. For each topic, retrieve all associated stories and have chatgpt write a digest of those stories in the given format.

9. assemble stories into first draft of newsletter for rewriting as necessary

todo:

use langgraph for final editing workfow
1. prompt to edit final copy for dupes, combine similar sections, copy edit
2. have a reviewer prompt check if there are any bullet points to move to a different section 
3. if so have an editor prompt remove them , return to 2. until nothing left to move dupes left
4. have a reviewer check each section, identify bullet points that are similar to other bullet points in the section and have identical links. rewrite combining so there is no duplication. 
5. identify sections that are short or similar to other sections and suggest sections that should be combined them
6. have an editor prompt merge short sections, return to 4, until no orphan sections left
7. maybe final copy-edit prompt

Original, alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
# import sys
# del sys.modules['ainb_utilities']


In [2]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [3]:
before_date = '2024-07-23 18:00:00'

In [4]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [5]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-24 10:06:29,379 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-24 10:06:29,381 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-24 10:06:29,381 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-24 10:06:29,381 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-24 10:06:29,382 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-24 10:06:29,382 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-24 10:0

In [6]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [7]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [8]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))
    

2024-07-24 10:06:29,428 - AInewsbot - INFO - 0


In [9]:
log(f"{len(files)} files found")


2024-07-24 10:06:29,432 - AInewsbot - INFO - 0 files found


# Fetch and save source pages

In [10]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-24 10:06:29,472 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-24 10:06:29,498 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:06:29,500 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:06:29,500 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:07:14,554 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:07:14,554 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:07:14,556 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:07:14,559 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-24 10:07:14,559 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-24 10:07:14,564 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-24 10:09:23,275 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-24 10:09:23,480 - AInewsbot - INFO - Proces

2024-07-24 10:10:42,977 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (07_24_2024 10_10_42 AM).html as utf-8
2024-07-24 10:10:42,988 - AInewsbot - INFO - Processing NYT Tech
2024-07-24 10:10:42,989 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-24 10:10:47,316 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-24 10:1

2024-07-24 10:11:48,473 - AInewsbot - INFO - Quit webdriver
2024-07-24 10:11:58,945 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-24 10:11:58,946 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_24_2024 10_11_58 AM).html as utf-8
2024-07-24 10:11:58,950 - AInewsbot - INFO - Quit webdriver


In [11]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

2024-07-24 10:12:07,082 - AInewsbot - INFO - Saved 17 pages
2024-07-24 10:12:07,085 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_24_2024 10_09_49 AM).html
2024-07-24 10:12:07,093 - AInewsbot - INFO - Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (07_24_2024 10_11_19 AM).html
2024-07-24 10:12:07,095 - AInewsbot - INFO - The Verge -> htmldata/Artificial Intelligence - The Verge (07_24_2024 10_11_39 AM).html
2024-07-24 10:12:07,096 - AInewsbot - INFO - Bloomberg Tech -> htmldata/Bloomberg Technology - Bloomberg (07_24_2024 10_09_50 AM).html
2024-07-24 10:12:07,128 - AInewsbot - INFO - Google News -> htmldata/Google News - Technology - Artificial intelligence (07_24_2024 10_10_42 AM).html
2024-07-24 10:12:07,133 - AInewsbot - INFO - NYT Tech -> htmldata/Technology - The New York Times (07_24_2024 10_10_57 AM).html
2024-07-24 10:12:07,137 - AInewsbot - INFO - Techmeme -> htmldata/Techmeme (07_24_2024 10_11_09 AM).html
2024-07-24 10:12:07,145 - AInewsbot - INFO

17


# Extract news URLs from saved pages

In [12]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-24 10:12:07,249 - AInewsbot - INFO - Parsing html files
2024-07-24 10:12:07,253 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_24_2024 10_09_49 AM).html
2024-07-24 10:12:07,255 - AInewsbot - INFO - parse loop - Ars Technica
2024-07-24 10:12:07,437 - AInewsbot - INFO - parse_file - found 257 raw links
2024-07-24 10:12:07,456 - AInewsbot - INFO - parse_file - found 28 filtered links
2024-07-24 10:12:07,457 - AInewsbot - INFO - parse loop - 28 links found
2024-07-24 10:12:07,458 - AInewsbot - INFO - Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (07_24_2024 10_11_19 AM).html
2024-07-24 10:12:07,458 - AInewsbot - INFO - parse loop - Feedly AI
2024-07-24 10:12:07,492 - AInewsbot - INFO - parse_file - found 73 raw links
2024-07-24 10:12:07,497 - AInewsbot - INFO - parse_file - found 0 filtered links
2024-07-24 10:12:07,498 - AInewsbot - INFO - parse loop - 0 links found
2024-07-24 10:12:07,499 - AInewsbot - INFO - The Verge -> htmldata/Artificial Intellige

,id,src,title,url
0,0,Ars Technica,Ars is seeking a seasoned senior reporter for ...,https://arstechnica.com/staff/2024/07/ars-is-s...
1,1,Ars Technica,Samsung delays Galaxy Buds3 Pro release over q...,https://arstechnica.com/gadgets/2024/07/samsun...
2,2,Ars Technica,Paul Sutter walks us through the future of cli...,https://arstechnica.com/science/2022/04/paul-s...
3,3,Ars Technica,Mini-Neptune turned out to be a frozen super-E...,https://arstechnica.com/science/2024/07/exopla...
4,4,Ars Technica,Want to cook like a Neanderthal? Archaeologist...,https://arstechnica.com/science/2024/07/hand-m...


In [13]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [14]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date=before_date)
len(filtered_df)


2024-07-24 10:12:11,372 - AInewsbot - INFO - Existing URLs: 138955
2024-07-24 10:12:11,483 - AInewsbot - INFO - New URLs: 519


519

In [15]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semantic meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based 

In [16]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-24 10:12:13,090 - AInewsbot - INFO - Paginated 11 pages


In [17]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-24 10:12:13,096 - AInewsbot - INFO - start classify
2024-07-24 10:12:13,101 - AInewsbot - INFO - Applying prompt to 11 pages using gpt-4o-mini
2024-07-24 10:12:13,107 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,140 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,141 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,142 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,143 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,143 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,144 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,145 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,145 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,146 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:12:13,146 - AInewsbot - INFO - sent 19 items 
2024-07-24 10:12:20,191 - AInewsbot - INFO - got dict with 19 items 
2024-07-24 10:12:20,199 - AInewsbot - INFO - got dict with 49 items 
2024-07-24 10:12:20,200 - AInewsbot - INFO - missin

616


,id,isAI
0,0,False
1,4,False
2,10,False
3,25,True
4,28,False


In [18]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,0,Ars Technica,Ars is seeking a seasoned senior reporter for ...,https://arstechnica.com/staff/2024/07/ars-is-s...,False,2024-07-24
1,4,Ars Technica,Want to cook like a Neanderthal? Archaeologist...,https://arstechnica.com/science/2024/07/hand-m...,False,2024-07-24
2,10,Ars Technica,How Russia-linked malware cut heat to 600 Ukra...,https://arstechnica.com/security/2024/07/how-r...,False,2024-07-24
3,25,Ars Technica,Alphabet invests another $5 billion in self-dr...,https://arstechnica.com/cars/2024/07/waymo-wil...,True,2024-07-24
4,28,Bloomberg Tech,Kishida Calls for More Money to Develop Domest...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-24


In [19]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-24 10:12:37,739 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-24 10:12:37,741 - AInewsbot - INFO - Unmatched source rows: 1


In [20]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")

AIdf

2024-07-24 10:12:37,746 - AInewsbot - INFO - Found 216 AI headlines


,id,src,title,url,isAI,date
0,0,Ars Technica,Alphabet invests another $5 billion in self-dr...,https://arstechnica.com/cars/2024/07/waymo-wil...,True,2024-07-24
1,1,Bloomberg Tech,Microsoft’s AI Assistants Will Revolutionize t...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-24
2,2,Bloomberg Tech,Startup Pearl Raises VC Funding to Bring AI to...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-24
3,3,Business Insider,"Nvidia, Amazon, Microsoft, and Paramount execs...",https://www.businessinsider.com/hollywood-exec...,True,2024-07-24
4,4,Business Insider,Mark Zuckerberg is asking the audience to help...,https://www.businessinsider.com/mark-zuckerber...,True,2024-07-24
...,...,...,...,...,...,...
211,211,VentureBeat,AI arms race escalates: OpenAI offers free GPT...,https://venturebeat.com/ai/ai-arms-race-escala...,True,2024-07-24
212,212,VentureBeat,Vanta secures $150 million as AI reshapes tech...,https://venturebeat.com/ai/vanta-secures-150-m...,True,2024-07-24
213,213,VentureBeat,Developers aren’t worried that gen AI will ste...,https://venturebeat.com/ai/stack-overflow-surv...,True,2024-07-24
214,214,VentureBeat,Nvidia’s coveted H100 GPUs will be available o...,https://venturebeat.com/ai/nvidias-coveted-h10...,True,2024-07-24


In [21]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [22]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-24 10:12:37,765 - AInewsbot - INFO - Found 157 unique AI headlines


In [23]:
# map google news headlines to redirect

AIdf = get_google_news_redirects(AIdf)


2024-07-24 10:12:38,067 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:38,413 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:38,821 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:39,140 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:39,466 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:39,768 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:40,082 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:40,427 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:40,761 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:41,065 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:12:41,412 - AInewsbot - INFO - get_google_news_redirects

2024-07-24 10:13:09,429 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:09,757 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:10,086 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:10,413 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:10,740 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:11,050 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:11,384 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:11,720 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:12,049 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:12,416 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> 
2024-07-24 10:13:12,756 - AInewsbot - INFO - get_google_news_redirects

In [24]:
# must do this after fixing google actualurl
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf.head()

,id,src,title,url,isAI,date,actual_url,hostname
0,0,Google News,'Our resolve is unwavering and should not be t...,https://news.google.com/articles/CBMizgFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMizgFodHRwc...,news.google.com
1,1,Google News,1 Top Artificial Intelligence (AI) Stock to Bu...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiXWh0dHBzO...,news.google.com
2,2,Google News,5 Reasons To Use Kubernetes for AI Inference,https://news.google.com/articles/CBMiRGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiRGh0dHBzO...,news.google.com
3,3,Google News,5 Steps That OpenAI Thinks Will Lead to Artifi...,https://news.google.com/articles/CBMidWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidWh0dHBzO...,news.google.com
4,4,Reddit,77% Of Employees Report AI Has Increased Workl...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-24,https://www.reddit.com/r/technology/comments/1...,www.reddit.com


### Get site names and update site names based on URL

In [25]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

,id,hostname,site_name
0,1,247wallst.com,247wallst
1,2,3quarksdaily.com,3 Quarks Daily
2,3,9to5google.com,9to5Google
3,4,9to5mac.com,9to5Mac
4,5,9to5toys.com,9to5Toys
...,...,...,...
778,826,www.yitay.net,Yitay
779,827,ipsnews.net,Inter Press Service
780,828,www.spokesman.com,The Spokesman-Review
781,829,lcamtuf.substack.com,lcamtuf's Substack


In [26]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

,id,src,title,url,isAI,date,actual_url,hostname,site_name


In [27]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


[]

In [28]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



In [29]:
for url in new_urls:
    sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
    print(url, '->', sites_dict[url])
    conn.execute(sqlstr, (url, sites_dict[url]))
    conn.commit()


In [30]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in AIdf.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [31]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [32]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [33]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [34]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [35]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

2024-07-24 10:13:20,644 - AInewsbot - INFO - start topic extraction
2024-07-24 10:13:20,647 - AInewsbot - INFO - Applying prompt to 4 pages using gpt-4o-mini
2024-07-24 10:13:20,648 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:13:20,649 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:13:20,650 - AInewsbot - INFO - sent 50 items 
2024-07-24 10:13:20,651 - AInewsbot - INFO - sent 7 items 
2024-07-24 10:13:24,145 - AInewsbot - INFO - got dict with 7 items 
2024-07-24 10:13:35,368 - AInewsbot - INFO - got dict with 50 items 
2024-07-24 10:13:37,331 - AInewsbot - INFO - got dict with 50 items 
2024-07-24 10:13:38,538 - AInewsbot - INFO - got dict with 50 items 
2024-07-24 10:13:38,541 - AInewsbot - INFO - Processed 157 responses.
2024-07-24 10:13:38,546 - AInewsbot - INFO - end topic extraction


In [36]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()


157


,id,extracted_topics
0,0,"[SAG-AFTRA, union, voice actors, AI protection..."
1,1,"[AI, stock market, investment, Loop Capital]"
2,2,"[Kubernetes, AI, inference, technology]"
3,3,"[OpenAI, artificial intelligence, business man..."
4,4,"[AI, workload, productivity, employee experien..."


In [37]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


87


['activision',
 'adobe',
 'ai adoption',
 'ai assistant',
 'ai policy',
 'ai protections',
 'alphabet',
 'amd',
 'apple',
 'arms race',
 'artwork',
 'automation',
 'automotive',
 'big tech',
 'call of duty',
 'challenges',
 'chatgpt',
 'cloud security',
 'code metal',
 'cohere',
 'competition',
 'creativity',
 'criticism',
 'customer satisfaction',
 'data',
 'dazz',
 'developers',
 'diageo',
 'diagnostics',
 'digital transformation',
 'dream machine',
 'drones',
 'education',
 'elon musk',
 'employee experience',
 'feature',
 'funding',
 'gaming',
 'generative ai',
 'google',
 'growth',
 'hyperdimensional computing',
 'illustrator',
 'investment',
 'japan',
 'johnnie walker',
 'kamala harris',
 'layoffs',
 'lenovo',
 'lifelong intelligence',
 'llama 3.1',
 'loneliness',
 'loops feature',
 'luma ai',
 'lumen',
 'lumen technologies',
 'mark zuckerberg',
 'meta',
 'meta ai',
 'microsoft',
 'nvidia',
 'openai',
 'opinion',
 'partnership',
 'photoshop',
 'playlist',
 'politics',
 'productiv

In [38]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [39]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

AI doom
AMD
Agriculture
Amazon
Anthropic
Apple
Art, Design
Artificial General Intelligence
Authors, Writing
Autonomous vehicles
Baidu
Big Tech
Bill Gates
Books, Publishing
Bubble
ChatGPT
China
Claude
Climate
Cloudflare
Code assistants
Cognitive Science
Cohere
Consciousness
Copilot
Cryptocurrency
Customer service
Cybersecurity
Data
Deals
Deepfakes
Disinformation
Disney
Economics
Education
Elon Musk
Energy
Entertainment
Ethics
European Union
Facial Recognition
Fashion
Finance
Fintech
Food, Drink
Funding
Gaming
Gemini
Gen AI
Geoffrey Hinton
Google
Health, Fitness
Healthcare
History
Hollywood
IPOs
India
Intellectual Property
Japan
Jobs, Careers, Labor Market
Korea
LLMs
Language Models
Laws
Lifestyle, Travel
Machine learning
Manufacturing
Mark Zuckerberg
Mergers and acquisitions
Meta
Microsoft
Military
Music
Mustafa Suleyman
Netflix
Nuclear
Nvidia
Open Source
OpenAI
Opinion
Perplexity
Policy and regulation
Politics
Privacy
Products
RAG
Recommendation systems
Reddit
Reinforcement Learning
Re

In [40]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

['sag-aftra',
 'voice actors',
 'ai protections',
 'strike',
 'stock market',
 'investment',
 'productivity',
 'employee experience',
 'study',
 'creativity',
 'diagnostics',
 'startup',
 'code metal',
 'drones',
 'arms race',
 'llama 3.1',
 'layoffs',
 'kamala harris',
 'ryzen ai',
 'activision',
 'call of duty',
 'adobe',
 'photoshop',
 'illustrator',
 'quest 3',
 'alphabet',
 'waymo',
 'self-driving',
 'competition',
 'playlist',
 'artwork',
 'customer satisfaction',
 'developers',
 'loneliness',
 'criticism',
 'dazz',
 'cloud security',
 'diageo',
 'johnnie walker',
 'generative ai',
 'automotive',
 'luma ai',
 'loops feature',
 'dream machine',
 'ai assistant',
 'trade',
 'feature',
 'supermarket',
 'staff smiles',
 'lenovo',
 'automation',
 'lifelong intelligence',
 'hyperdimensional computing',
 'lumen',
 'digital transformation',
 'ai policy',
 'meta ai',
 'lumen technologies',
 'partnership',
 'ai adoption',
 'toyota',
 'growth',
 'challenges',
 'startups',
 'stanford']

In [41]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,'Our resolve is unwavering and should not be t...,https://news.google.com/articles/CBMizgFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMizgFodHRwc...,news.google.com,Google News
1,1,Google News,1 Top Artificial Intelligence (AI) Stock to Bu...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiXWh0dHBzO...,news.google.com,Google News
2,2,Google News,5 Reasons To Use Kubernetes for AI Inference,https://news.google.com/articles/CBMiRGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiRGh0dHBzO...,news.google.com,Google News
3,3,Google News,5 Steps That OpenAI Thinks Will Lead to Artifi...,https://news.google.com/articles/CBMidWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidWh0dHBzO...,news.google.com,Google News
4,4,Reddit,77% Of Employees Report AI Has Increased Workl...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-24,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
...,...,...,...,...,...,...,...,...,...
152,152,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News
153,153,Google News,"When it comes to political advertising, is AI ...",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
154,154,Google News,Why Coheres $5.5 Billion Valuation Isnt What I...,https://news.google.com/articles/CBMihAFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMihAFodHRwc...,news.google.com,Google News
155,155,Reddit,honeyIBrokeTheLanguageModel,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-24,https://www.reddit.com/r/ProgrammerHumor/comme...,www.reddit.com,Reddit


In [42]:
# maybe try to set timeout in categorize_headline
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

2024-07-24 10:13:38,609 - AInewsbot - INFO - Categorizing headline 1 of 157
2024-07-24 10:13:38,610 - AInewsbot - INFO - 'Our resolve is unwavering and should not be tested': Major union SAG-AFTRA breaks glass on big red strike button as negotiations continue on voice actor AI protections
2024-07-24 10:13:40,896 - AInewsbot - INFO - ['Entertainment', 'Gen AI', 'Hollywood', 'Intellectual Property', 'Jobs, Careers, Labor Market', 'Laws', 'Society, Culture', 'TV, Film, Movies']
2024-07-24 10:13:40,897 - AInewsbot - INFO - Categorizing headline 2 of 157
2024-07-24 10:13:40,898 - AInewsbot - INFO - 1 Top Artificial Intelligence (AI) Stock to Buy Before It Soars 84%, According to Loop Capital
2024-07-24 10:13:46,524 - AInewsbot - INFO - ['Finance', 'Gen AI', 'Nvidia', 'Opinion', 'Stocks']
2024-07-24 10:13:46,526 - AInewsbot - INFO - Categorizing headline 3 of 157
2024-07-24 10:13:46,527 - AInewsbot - INFO - 5 Reasons To Use Kubernetes for AI Inference
2024-07-24 10:13:47,894 - AInewsbot - IN

2024-07-24 10:14:32,913 - AInewsbot - INFO - Categorizing headline 27 of 157
2024-07-24 10:14:32,914 - AInewsbot - INFO - A look at Meta AI running on a Quest 3 headset.
2024-07-24 10:14:34,407 - AInewsbot - INFO - ['Big Tech', 'Gaming', 'Gen AI', 'Meta', 'Products', 'Science', 'Virtual Reality']
2024-07-24 10:14:34,408 - AInewsbot - INFO - Categorizing headline 28 of 157
2024-07-24 10:14:34,410 - AInewsbot - INFO - Alphabet's earnings show investors are impatient for the data on AI returns. No one has the answer yet.
2024-07-24 10:14:36,483 - AInewsbot - INFO - ['Big Tech', 'Data', 'Economics', 'Finance', 'Gen AI', 'Google', 'Stocks']
2024-07-24 10:14:36,487 - AInewsbot - INFO - Categorizing headline 29 of 157
2024-07-24 10:14:36,488 - AInewsbot - INFO - Alphabet invests another $5 billion in self-driving startup Waymo
2024-07-24 10:14:38,386 - AInewsbot - INFO - ['Autonomous vehicles', 'Big Tech', 'Deals', 'Funding', 'Google', 'Science', 'Transportation', 'Venture Capital']
2024-07-2

2024-07-24 10:15:13,877 - AInewsbot - INFO - Categorizing headline 53 of 157
2024-07-24 10:15:13,878 - AInewsbot - INFO - Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call
2024-07-24 10:15:15,627 - AInewsbot - INFO - ['Autonomous vehicles', 'Big Tech', 'Elon Musk', 'Finance', 'Stocks', 'Tesla']
2024-07-24 10:15:15,628 - AInewsbot - INFO - Categorizing headline 54 of 157
2024-07-24 10:15:15,628 - AInewsbot - INFO - Elon Musk doubles down on Tesla's Dojo supercomputer, aiming to compete with Nvidia
2024-07-24 10:15:17,621 - AInewsbot - INFO - ['Big Tech', 'Data', 'Elon Musk', 'Gen AI', 'Machine learning', 'Nvidia', 'Semiconductor chips', 'Tesla']
2024-07-24 10:15:17,622 - AInewsbot - INFO - Categorizing headline 55 of 157
2024-07-24 10:15:17,623 - AInewsbot - INFO - Elon Musk powers new 'World's Fastest AI Data Center" with gargantuan portable power generators to sidestep electricity supply constraints
2024-07-24 10:15:19,119 - AInewsbot - INFO - ['Big Tech', 'Data', 'Elon Musk', '

2024-07-24 10:16:03,805 - AInewsbot - INFO - ['Customer service', 'Facial Recognition', 'Gen AI', 'Japan', 'Society, Culture']
2024-07-24 10:16:03,807 - AInewsbot - INFO - Categorizing headline 80 of 157
2024-07-24 10:16:03,811 - AInewsbot - INFO - Japan supermarket chain uses AI to assess staff smiles, sparking harassment concerns
2024-07-24 10:16:05,523 - AInewsbot - INFO - ['Customer service', 'Ethics', 'Facial Recognition', 'Japan', 'Jobs, Careers, Labor Market', 'Machine learning', 'Privacy', 'Society, Culture']
2024-07-24 10:16:05,525 - AInewsbot - INFO - Categorizing headline 81 of 157
2024-07-24 10:16:05,526 - AInewsbot - INFO - Johnnie Walker offers custom AI bottles
2024-07-24 10:16:06,777 - AInewsbot - INFO - ['Food, Drink', 'Gen AI', 'Products']
2024-07-24 10:16:06,779 - AInewsbot - INFO - Categorizing headline 82 of 157
2024-07-24 10:16:06,780 - AInewsbot - INFO - Kodiak Robotics completes first driverless semi truck delivery ahead of commercial operations
2024-07-24 10:16

2024-07-24 10:16:42,580 - AInewsbot - INFO - Microsoft Partners With Lumen To Power Up AI Capabilities
2024-07-24 10:16:43,891 - AInewsbot - INFO - ['Big Tech', 'Deals', 'Gen AI', 'LLMs', 'Machine learning', 'Microsoft']
2024-07-24 10:16:43,893 - AInewsbot - INFO - Categorizing headline 106 of 157
2024-07-24 10:16:43,894 - AInewsbot - INFO - Microsoft and Lumen Technologies partner to power the future of AI and enable digital transformation to benefit hundreds of millions of customers
2024-07-24 10:16:46,073 - AInewsbot - INFO - ['Big Tech', 'Deals', 'Gen AI', 'Microsoft']
2024-07-24 10:16:46,075 - AInewsbot - INFO - Categorizing headline 107 of 157
2024-07-24 10:16:46,076 - AInewsbot - INFO - Microsofts AI Assistants Will Revolutionize the Office  One Day
2024-07-24 10:16:47,501 - AInewsbot - INFO - ['Big Tech', 'Copilot', 'Gen AI', 'Microsoft', 'Opinion', 'Products']
2024-07-24 10:16:47,504 - AInewsbot - INFO - Categorizing headline 108 of 157
2024-07-24 10:16:47,505 - AInewsbot - IN

2024-07-24 10:17:26,782 - AInewsbot - INFO - ['Deals', 'Finance', 'Funding', 'Gen AI', 'Healthcare', 'Venture Capital']
2024-07-24 10:17:26,783 - AInewsbot - INFO - Categorizing headline 131 of 157
2024-07-24 10:17:26,783 - AInewsbot - INFO - Tandem drifting Toyotas show how AI might help drivers on slippery roads
2024-07-24 10:17:28,196 - AInewsbot - INFO - ['Autonomous vehicles', 'Copilot', 'Science', 'Transportation']
2024-07-24 10:17:28,197 - AInewsbot - INFO - Categorizing headline 132 of 157
2024-07-24 10:17:28,197 - AInewsbot - INFO - Tesla's profit margin is getting hammered by EV discounts and hefty AI spending
2024-07-24 10:17:29,778 - AInewsbot - INFO - ['Economics', 'Finance', 'Stocks', 'Tesla', 'Transportation']
2024-07-24 10:17:29,780 - AInewsbot - INFO - Categorizing headline 133 of 157
2024-07-24 10:17:29,781 - AInewsbot - INFO - Tesla Invests in AI, Anticipates Next Major Growth Wave
2024-07-24 10:17:32,255 - AInewsbot - INFO - ['Big Tech', 'Economics', 'Funding', 'Gen

2024-07-24 10:18:35,156 - AInewsbot - INFO - ['Big Tech', 'Data', 'Gen AI', 'Machine learning', 'Nvidia', 'Science', 'Semiconductor chips']


{0: ['Entertainment',
  'Gen AI',
  'Hollywood',
  'Intellectual Property',
  'Jobs, Careers, Labor Market',
  'Laws',
  'Society, Culture',
  'TV, Film, Movies'],
 1: ['Finance', 'Gen AI', 'Nvidia', 'Opinion', 'Stocks'],
 2: ['Gen AI', 'Machine learning'],
 3: ['Artificial General Intelligence',
  'Big Tech',
  'Gen AI',
  'OpenAI',
  'Science'],
 4: ['AI doom', 'Gen AI', 'Jobs, Careers, Labor Market'],
 5: ['Art, Design',
  'Authors, Writing',
  'Entertainment',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Machine learning',
  'Science',
  'Society, Culture'],
 6: ['Military', 'Opinion'],
 7: ['Health, Fitness',
  'Healthcare',
  'Machine learning',
  'Science',
  'Testing'],
 8: ['Deals', 'Economics', 'Finance', 'Funding', 'Venture Capital'],
 9: ['Climate', 'Science', 'Sustainability'],
 10: ['Big Tech',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Machine learning',
  'Meta',
  'OpenAI'],
 11: ['Economics', 'Finance', 'Gen AI', 'UK'],
 12: ['AI doom',
  'Big Tech',
  'Economics'

In [43]:
topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))
topic_df

,id,extracted_topics,assigned_topics
0,0,"[Sag-Aftra, Voice Actors, Ai Protections, Strike]","[Entertainment, Gen AI, Hollywood, Intellectua..."
1,1,"[Stock Market, Investment]","[Finance, Gen AI, Nvidia, Opinion, Stocks]"
2,2,[],"[Gen AI, Machine learning]"
3,3,[Openai],"[Artificial General Intelligence, Big Tech, Ge..."
4,4,"[Productivity, Employee Experience, Study]","[AI doom, Gen AI, Jobs, Careers, Labor Market]"
...,...,...,...
152,152,"[Funding, Startups, Investment]","[Deals, Economics, Finance, Funding, Venture C..."
153,153,[],"[Ethics, Gen AI, Opinion, Policy and regulatio..."
154,154,[Cohere],"[Bubble, Cohere, Data, Finance, Funding, LLMs,..."
155,155,[],[Language Models]


In [44]:
lcategories = set([c.lower() for c in categories])

In [45]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

,id,extracted_topics,assigned_topics,topics,topic_str
0,0,"[Sag-Aftra, Voice Actors, Ai Protections, Strike]","[Entertainment, Gen AI, Hollywood, Intellectua...","[AI Protections, Entertainment, Gen AI, Hollyw...","AI Protections, Entertainment, Gen AI, Hollywo..."
1,1,"[Stock Market, Investment]","[Finance, Gen AI, Nvidia, Opinion, Stocks]","[Finance, Gen AI, Investment, Nvidia, Opinion,...","Finance, Gen AI, Investment, Nvidia, Opinion, ..."
2,2,[],"[Gen AI, Machine learning]","[Gen AI, Machine Learning]","Gen AI, Machine Learning"
3,3,[Openai],"[Artificial General Intelligence, Big Tech, Ge...","[Artificial General Intelligence, Big Tech, Ge...","Artificial General Intelligence, Big Tech, Gen..."
4,4,"[Productivity, Employee Experience, Study]","[AI doom, Gen AI, Jobs, Careers, Labor Market]","[AI Doom, Employee Experience, Gen AI, Jobs, C...","AI Doom, Employee Experience, Gen AI, Jobs, Ca..."
...,...,...,...,...,...
152,152,"[Funding, Startups, Investment]","[Deals, Economics, Finance, Funding, Venture C...","[Deals, Economics, Finance, Funding, Investmen...","Deals, Economics, Finance, Funding, Investment..."
153,153,[],"[Ethics, Gen AI, Opinion, Policy and regulatio...","[Ethics, Gen AI, Opinion, Policy And Regulatio...","Ethics, Gen AI, Opinion, Policy And Regulation..."
154,154,[Cohere],"[Bubble, Cohere, Data, Finance, Funding, LLMs,...","[Bubble, Cohere, Data, Finance, Funding, Llms,...","Bubble, Cohere, Data, Finance, Funding, Llms, ..."
155,155,[],[Language Models],[Language Models],Language Models


In [46]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,'Our resolve is unwavering and should not be t...,https://news.google.com/articles/CBMizgFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMizgFodHRwc...,news.google.com,Google News
1,1,Google News,1 Top Artificial Intelligence (AI) Stock to Bu...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiXWh0dHBzO...,news.google.com,Google News
2,2,Google News,5 Reasons To Use Kubernetes for AI Inference,https://news.google.com/articles/CBMiRGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiRGh0dHBzO...,news.google.com,Google News
3,3,Google News,5 Steps That OpenAI Thinks Will Lead to Artifi...,https://news.google.com/articles/CBMidWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidWh0dHBzO...,news.google.com,Google News
4,4,Reddit,77% Of Employees Report AI Has Increased Workl...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-24,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
...,...,...,...,...,...,...,...,...,...
152,152,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News
153,153,Google News,"When it comes to political advertising, is AI ...",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
154,154,Google News,Why Coheres $5.5 Billion Valuation Isnt What I...,https://news.google.com/articles/CBMihAFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMihAFodHRwc...,news.google.com,Google News
155,155,Reddit,honeyIBrokeTheLanguageModel,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-24,https://www.reddit.com/r/ProgrammerHumor/comme...,www.reddit.com,Reddit


In [47]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,'Our resolve is unwavering and should not be t...,https://news.google.com/articles/CBMizgFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMizgFodHRwc...,news.google.com,Google News
1,1,Google News,1 Top Artificial Intelligence (AI) Stock to Bu...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiXWh0dHBzO...,news.google.com,Google News
2,2,Google News,5 Reasons To Use Kubernetes for AI Inference,https://news.google.com/articles/CBMiRGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiRGh0dHBzO...,news.google.com,Google News
3,3,Google News,5 Steps That OpenAI Thinks Will Lead to Artifi...,https://news.google.com/articles/CBMidWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidWh0dHBzO...,news.google.com,Google News
4,4,Reddit,77% Of Employees Report AI Has Increased Workl...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-24,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit
...,...,...,...,...,...,...,...,...,...
152,152,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News
153,153,Google News,"When it comes to political advertising, is AI ...",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News
154,154,Google News,Why Coheres $5.5 Billion Valuation Isnt What I...,https://news.google.com/articles/CBMihAFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMihAFodHRwc...,news.google.com,Google News
155,155,Reddit,honeyIBrokeTheLanguageModel,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-24,https://www.reddit.com/r/ProgrammerHumor/comme...,www.reddit.com,Reddit


In [48]:
try:  # for idempotency
    AIdf = AIdf.drop(columns=["title_topic_str"])
except:
    pass
try:  # for idempotency
    AIdf = AIdf.drop(columns=["topic_str"])
except:
    pass

AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="inner")
AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str
0,0,Google News,'Our resolve is unwavering and should not be t...,https://news.google.com/articles/CBMizgFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMizgFodHRwc...,news.google.com,Google News,"AI Protections, Entertainment, Gen AI, Hollywo...",'Our resolve is unwavering and should not be t...
1,1,Google News,1 Top Artificial Intelligence (AI) Stock to Bu...,https://news.google.com/articles/CBMiXWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiXWh0dHBzO...,news.google.com,Google News,"Finance, Gen AI, Investment, Nvidia, Opinion, ...",1 Top Artificial Intelligence (AI) Stock to Bu...
2,2,Google News,5 Reasons To Use Kubernetes for AI Inference,https://news.google.com/articles/CBMiRGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiRGh0dHBzO...,news.google.com,Google News,"Gen AI, Machine Learning",5 Reasons To Use Kubernetes for AI Inference (...
3,3,Google News,5 Steps That OpenAI Thinks Will Lead to Artifi...,https://news.google.com/articles/CBMidWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidWh0dHBzO...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Gen...",5 Steps That OpenAI Thinks Will Lead to Artifi...
4,4,Reddit,77% Of Employees Report AI Has Increased Workl...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-24,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit,"AI Doom, Employee Experience, Gen AI, Jobs, Ca...",77% Of Employees Report AI Has Increased Workl...
...,...,...,...,...,...,...,...,...,...,...,...
152,152,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News,"Deals, Economics, Finance, Funding, Investment...",Web3 AI firm Network3 raises $5.5 million in f...
153,153,Google News,"When it comes to political advertising, is AI ...",https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiV2h0dHBzO...,news.google.com,Google News,"Ethics, Gen AI, Opinion, Policy And Regulation...","When it comes to political advertising, is AI ..."
154,154,Google News,Why Coheres $5.5 Billion Valuation Isnt What I...,https://news.google.com/articles/CBMihAFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMihAFodHRwc...,news.google.com,Google News,"Bubble, Cohere, Data, Finance, Funding, Llms, ...",Why Coheres $5.5 Billion Valuation Isnt What I...
155,155,Reddit,honeyIBrokeTheLanguageModel,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-24,https://www.reddit.com/r/ProgrammerHumor/comme...,www.reddit.com,Reddit,Language Models,honeyIBrokeTheLanguageModel (Topics: Language ...


In [49]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf.loc[AIdf["topic_str"]==""][['title']])


,title


### Semantic sort

In [50]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


2024-07-24 10:18:35,351 - AInewsbot - INFO - Fetching embeddings for 157 headlines
2024-07-24 10:18:35,994 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:18:38,796 - AInewsbot - INFO - Sort with agglomerative cluster sort
2024-07-24 10:18:38,872 - AInewsbot - INFO - Perform dimensionality reduction
2024-07-24 10:18:46,015 - AInewsbot - INFO - Cluster with DBSCAN


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster
0,0,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News,"Deals, Economics, Finance, Funding, Investment...",Web3 AI firm Network3 raises $5.5 million in f...,26,0
1,1,Google News,"Researchers launch Code Metal, Boston startup ...",https://news.google.com/articles/CBMiO2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiO2h0dHBzO...,news.google.com,Google News,"Autonomous Vehicles, Code Metal, Drones, Gen A...","Researchers launch Code Metal, Boston startup ...",41,0
2,2,Google News,"AI Startup, Code Metal, Secures $16.5M in Seed...",https://news.google.com/articles/CBMiiwFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMiiwFodHRwc...,news.google.com,Google News,"Code Metal, Deals, Economics, Finance, Funding...","AI Startup, Code Metal, Secures $16.5M in Seed...",46,0
3,3,Techmeme,"Seattle-based QA Wolf, which helps companies a...",https://techcrunch.com/2024/07/23/qa-wolf-secu...,True,2024-07-24,https://techcrunch.com/2024/07/23/qa-wolf-secu...,techcrunch.com,TechCrunch,"Automation, Deals, Funding, Testing, Venture C...","Seattle-based QA Wolf, which helps companies a...",87,0
4,4,Google News,"Dazz snaps up $50M for AI-based, automated clo...",https://news.google.com/articles/CBMiZmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiZmh0dHBzO...,news.google.com,Google News,"Cloud Security, Cybersecurity, Dazz, Deals, Fu...","Dazz snaps up $50M for AI-based, automated clo...",92,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,Google News,AI Detects Prostate Cancer 17% More Accurately...,https://news.google.com/articles/CBMia2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMia2h0dHBzO...,news.google.com,Google News,"Diagnostics, Health, Fitness, Healthcare, Mach...",AI Detects Prostate Cancer 17% More Accurately...,150,999
153,153,Google News,Apple Music will soon let users create playlis...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiT2h0dHBzO...,news.google.com,Google News,"Apple, Art, Design, Artwork, Big Tech, Enterta...",Apple Music will soon let users create playlis...,151,999
154,154,Google News,AI firms will soon exhaust most of the interne...,https://news.google.com/articles/CBMiaGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiaGh0dHBzO...,news.google.com,Google News,"AI Doom, Big Tech, Data, Gen AI, Llms",AI firms will soon exhaust most of the interne...,152,999
155,155,Google News,Lenovo to put AI into all PCs by 2027: device ...,https://news.google.com/articles/CBMidmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidmh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Lenovo, Products, Science",Lenovo to put AI into all PCs by 2027: device ...,153,999


In [51]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


,id,title_topic_str
0,0,"Web3 AI firm Network3 raises $5.5 million in funding (Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital)"
1,1,"Researchers launch Code Metal, Boston startup using AI for drones and robots (Topics: Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital)"
2,2,"AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development (Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital)"
3,3,"Seattle-based QA Wolf, which helps companies automate QA testing for apps, raised a $36M Series B and opens a waitlist for Android and iOS test automation (Topics: Automation, Deals, Funding, Testing, Venture Capital)"
4,4,"Dazz snaps up $50M for AI-based, automated cloud security remediation (Topics: Cloud Security, Cybersecurity, Dazz, Deals, Funding, Products, Venture Capital)"
5,5,"Vanta secures $150 million as AI reshapes tech compliance landscape (Topics: Deals, Funding, Gen AI, Venture Capital)"
6,6,"Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M (Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital)"
7,7,"AI startup Code Metal raises $16.5m in seed funding (Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital)"
8,8,"Dazz raises $50 million to accelerate AI-powered cloud security remediation (Topics: Cloud Security, Cybersecurity, Dazz, Deals, Finance, Funding, Venture Capital)"


{'topic_title': 'AI Startups and Venture Capital Funding'}


,id,title_topic_str
9,9,"Adobe Adds New AI Tools to Photoshop and Illustrator (Topics: Adobe, Art, Design, Big Tech, Gen AI, Illustrator, Machine Learning, Photoshop, Products)"
10,10,"Adobe Rolls Out More AI Generative Tools To Illustrator And Photoshop (Topics: Adobe, Art, Design, Big Tech, Gen AI, Illustrator, Machine Learning, Photoshop, Products)"
11,11,"Adobe's Photoshop can now generate AI images via prompts like Dall-E or Midjourney (Topics: Adobe, Art, Design, Big Tech, Gen AI, Machine Learning, Photoshop, Products)"


{'topic_title': 'Adobe AI Tools for Design'}


,id,title_topic_str
12,12,"How AI Companions Are Redefining Human Relationships In The Digital Age (Topics: Gen AI, Opinion, Science, Society, Culture)"
13,13,"Developers claim AI can help combat loneliness, but critics say it can't be trusted (Topics: Criticism, Developers, Ethics, Gen AI, Loneliness, Opinion, Science, Society, Culture)"
14,14,"Critics doubt developers claiming AI can combat loneliness (Topics: Criticism, Developers, Ethics, Gen AI, Loneliness, Opinion, Society, Culture)"
15,15,"I signed up for an AI relationship here's what I learned (Topics: Entertainment, Gen AI, Opinion, Review, Society, Culture)"
16,16,"NSFW Character Al: Dive into The World of AI Chat Companions (Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture)"


...

{'topic_title': 'Microsoft and Lumen AI Partnership'}


,id,title_topic_str
43,43,"Sinister or Revolutionary? A Japanese Supermarket Chain is Using AI to Monitor Staff Smiles (Topics: Customer Service, Ethics, Facial Recognition, Gen AI, Japan, Machine Learning, Opinion, Privacy, Society, Culture)"
44,44,"Japan Supermarket Chain Becomes World's First To Use AI To ""Assess And Standardise"" Staff Smiles (Topics: Customer Service, Facial Recognition, Gen AI, Japan, Society, Culture, Staff Smiles, Supermarket)"
45,45,"Japan supermarket chain uses AI to assess staff smiles, sparking harassment concerns (Topics: Customer Service, Ethics, Facial Recognition, Japan, Jobs, Careers, Labor Market, Machine Learning, Privacy, Society, Culture, Staff Smiles, Supermarket)"


{'topic_title': 'AI in Customer Service Ethics'}


,id,title_topic_str
46,46,"Toyota Research Institute and Stanford Engineering Achieve Worlds First Fully Autonomous Tandem Drift Sequence (Topics: Autonomous Vehicles, Science, Stanford, Testing, Toyota, Transportation)"
47,47,"Tandem drifting Toyotas show how AI might help drivers on slippery roads (Topics: Autonomous Vehicles, Copilot, Science, Toyota, Transportation)"
48,48,"Toyota, Stanford teaches AI to drift two GR Supra in tandem, a 1st (Topics: Machine Learning, Stanford, Testing, Toyota, Transportation)"
49,49,"Toyota Pulls Off a Fast and Furious Demo With Dual Drifting AI-Powered Race Cars (Topics: Autonomous Vehicles, Entertainment, Machine Learning, Products, Robots, Sports, Testing, Toyota, Transportation)"


{'topic_title': 'Autonomous Vehicle Technology and Testing'}


In [52]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [53]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

['AI Startups and Venture Capital Funding',
 'Adobe AI Tools for Design',
 'AI and Human Relationships',
 'Generative AI in Product Personalization',
 'Tesla and AI Innovations',
 'Elon Musk and AI Technology',
 'Google and AI in Big Tech',
 'AI in Education',
 'AI Video Creation Tools',
 'Microsoft and Lumen AI Partnership',
 'AI in Customer Service Ethics',
 'Autonomous Vehicle Technology and Testing']

In [54]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [55]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.site_name}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.actual_url}">{row.title} - {row.site_name}</a><br />\n'


2024-07-24 10:18:54,908 - AInewsbot - INFO - [0. Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,908 - AInewsbot - INFO - [1. Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,909 - AInewsbot - INFO - [2. AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV

2024-07-24 10:18:54,914 - AInewsbot - INFO - [24. Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,915 - AInewsbot - INFO - [25. Robotaxis, FSD, AI: These are a few of Elon Musks favorite Tesla things - Google News](https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9yb2JvdGF4aXMtZnNkLWFpLXRoZXNlLWFyZS1hLWZldy1vZi1lbG9uLW11c2tzLWZhdm9yaXRlLXRlc2xhLXRoaW5ncy0yMWVkZGM2Y9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,916 - AInewsbot - INFO - [26. Elon Musk powers new 'World's Fastest AI Data Center" with gargantuan portable power generators to sidestep electricity supply constraints - Google News](https://news.google.com/articles/CBMiygFodHRwczovL3d3dy50b21zaGFyZHdhcmUuY29tL3RlY2gtaW5kdXN0cnkvYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY

2024-07-24 10:18:54,924 - AInewsbot - INFO - [47. Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,924 - AInewsbot - INFO - [48. Toyota, Stanford teaches AI to drift two GR Supra in tandem, a 1st - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vaW50ZXJlc3RpbmdlbmdpbmVlcmluZy5jb20vaW5ub3ZhdGlvbi90b3lvdGEtc3RhbmZvcmQtYXV0b25vbW91cy10YW5kZW0tZHJpZnTSAQA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,925 - AInewsbot - INFO - [49. Toyota Pulls Off a Fast and Furious Demo With Dual Drifting AI-Powered Race Cars - Google News](https://news.google.com/articles/CBMiRGh0dHBzOi8vd3d3LndpcmVkLmNvbS9zdG9yeS90b3lvdGEtc3RhbmZvcmQtYWktdGFuZGVtLWRyaWZ0aW5nLWNhcnMv0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,925 - AInewsbo

2024-07-24 10:18:54,933 - AInewsbot - INFO - [74. US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,934 - AInewsbot - INFO - [75. Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,934 - AInewsbot - INFO - [76. Billions Of People Need To Learn AI Literacy - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvam9obndlcm5lci8yMDI0LzA3LzE3L2JpbGxpb25zLW9mLXBlb3BsZS1uZWVkLXRvLWxlYXJuLWFpLWxpdGVyYWN5L9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,934 - AIne

2024-07-24 10:18:54,941 - AInewsbot - INFO - [100. Meta AI Introduces AI-Generated Photos to All Platforms - Google News](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnNlYXJjaGVuZ2luZWpvdXJuYWwuY29tL21ldGEtYWktaW50cm9kdWNlcy1haS1nZW5lcmF0ZWQtcGhvdG9zLXRvLWFsbC1wbGF0Zm9ybXMvNTIyODM1L9IBAA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,942 - AInewsbot - INFO - [101. Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,942 - AInewsbot - INFO - [102. The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl

2024-07-24 10:18:54,948 - AInewsbot - INFO - [127. Mark Zuckerberg Stumps for Open Source A.I. - The New York Times](https://www.nytimes.com/2024/07/23/technology/mark-zuckerberg-meta-open-source-ai.html)
2024-07-24 10:18:54,948 - AInewsbot - INFO - [128. OpenAI removes AI safety executive Aleksander Madry from role - Reddit](https://www.reddit.com/r/technology/comments/1eaj0gb/openai_removes_ai_safety_executive_aleksander/)
2024-07-24 10:18:54,949 - AInewsbot - INFO - [129. Opinion: This Nvidia Forecast All but Confirms That the Artificial Intelligence (AI) Bubble Will Burst Sooner Rather Than Later - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,949 - AInewsbot - INFO - [130. Nvidias coveted H100 GPUs will be available on-demand through Lambda - VentureBeat](https://venturebeat.com/ai/nvidias-coveted-h100-g

2024-07-24 10:18:54,959 - AInewsbot - INFO - [152. AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,959 - AInewsbot - INFO - [153. Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)
2024-07-24 10:18:54,960 - AInewsbot - INFO - [154. AI firms will soon exhaust most of the internets data - Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3L

In [56]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [57]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-24 10:18:54,971 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [58]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.actual_url, row.title))


2024-07-24 10:18:56,758 - AInewsbot - INFO - Queuing URLs for scraping


In [59]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-24 10:18:56,778 - AInewsbot - INFO - fetching 157 pages using 4 browsers
2024-07-24 10:18:56,780 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:18:56,782 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:18:56,782 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:18:56,783 - AInewsbot - INFO - get_driver - 16262 Initializing webdriver
2024-07-24 10:20:08,266 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:20:08,267 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:20:08,268 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:20:08,269 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-24 10:20:08,270 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-24 10:20:08,271 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-24 10:20:08,271 - AInewsbot - INFO 

2024-07-24 10:24:53,957 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LmNhbGNhbGlzdGVjaC5jb20vY3RlY2huZXdzL2FydGljbGUvczFqZDV2MDAwYdIBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LmNhbGNhbGlzdGVjaC5jb20vY3RlY2huZXdzL2FydGljbGUvczFqZDV2MDAwYdIBAA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:25:07,860 - AInewsbot - INFO - get_url(Dazz raises $50 million to accelerate AI-powered cloud security remediation) - Saving Dazz_raises__50_million_to_accelerate_AI-powered_cloud_security_remediation_20240724_102507.html as utf-8
2024-07-24 10:25:07,879 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yMy9waG90b3Nob3AtaWxsdXN0cmF0b3ItbmV3LWFpLXRvb2xzL9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:25:07,890 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yMy9waG90b3Nob3AtaWxsdXN0cmF0b3

2024-07-24 10:25:35,792 - AInewsbot - INFO - get_url(Critics doubt developers claiming AI can combat loneliness) - Saving Critics_doubt_developers_claiming_AI_can_combat_loneliness_20240724_102535.html as utf-8
2024-07-24 10:25:35,798 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9kaWFnZW8tam9obm5pZS13YWxrZXItdW52ZWlscy1nZW5lcmF0aXZlLTEwNTcwNzYyNy5odG1s0gEA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:25:35,801 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9kaWFnZW8tam9obm5pZS13YWxrZXItdW52ZWlscy1nZW5lcmF0aXZlLTEwNTcwNzYyNy5odG1s0gEA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9kaWFnZW8tam9obm5pZS13YWxrZXItdW52ZWlscy1nZW5lcmF0aXZlLTEwNTcwNzYyNy5odG1s0gEA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:25:37,478 - AInewsbot - INFO - get_url(Developers claim AI can help combat loneliness, but cri

2024-07-24 10:26:03,010 - AInewsbot - INFO - get_url(Tesla's profit margin is getting hammered by EV discounts and hefty AI spending) - Saving Tesla_s_profit_margin_is_getting_hammered_by_EV_discounts_and_hefty_AI_spending_20240724_102603.html as utf-8
2024-07-24 10:26:03,026 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9yb2JvdGF4aXMtZnNkLWFpLXRoZXNlLWFyZS1hLWZldy1vZi1lbG9uLW11c2tzLWZhdm9yaXRlLXRlc2xhLXRoaW5ncy0yMWVkZGM2Y9IBAA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:26:03,028 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9yb2JvdGF4aXMtZnNkLWFpLXRoZXNlLWFyZS1hLWZldy1vZi1lbG9uLW11c2tzLWZhdm9yaXRlLXRlc2xhLXRoaW5ncy0yMWVkZGM2Y9IBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9yb2JvdGF4aXMtZnNkLWFpLXRoZXNlLWFyZS1hLWZldy1vZi1lbG9uLW11c2tzLWZhdm9yaXRlLXRlc2xhLXRoaW5ncy0yMWVkZ

2024-07-24 10:26:33,345 - AInewsbot - INFO - get_url(Google Is the Only Search Engine That Works on Reddit Now Thanks to AI Deal) - Saving Google_Is_the_Only_Search_Engine_That_Works_on_Reddit_Now_Thanks_to_AI_Deal_20240724_102633.html as utf-8
2024-07-24 10:26:33,348 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lndrcm4uY29tL25ld3MvbG9jYWwtbmV3cy9uYXNodmlsbGUvbWV0cm8tbmFzaHZpbGxlLXNjaG9vbHMtdG8taW1wbGVtZW50LW5ldy1haS1wb2xpY3kv0gFmaHR0cHM6Ly93d3cud2tybi5jb20vbmV3cy9sb2NhbC1uZXdzL25hc2h2aWxsZS9tZXRyby1uYXNodmlsbGUtc2Nob29scy10by1pbXBsZW1lbnQtbmV3LWFpLXBvbGljeS9hbXAv?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:26:33,359 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lndrcm4uY29tL25ld3MvbG9jYWwtbmV3cy9uYXNodmlsbGUvbWV0cm8tbmFzaHZpbGxlLXNjaG9vbHMtdG8taW1wbGVtZW50LW5ldy1haS1wb2xpY3kv0gFmaHR0cHM6Ly93d3cud2tybi5jb20vbmV3cy9sb2NhbC1uZXdzL25hc2h2aWxsZS9tZXRyby1uYXNodmlsbGUtc2Nob29scy10by1pbXBsZW1lbnQtbmV3LWFpLXBvbGljeS9hbXAv?

...

2024-07-24 10:32:12,661 - AInewsbot - INFO - get_url(Nvidia said to launch new AI chip for China) - Saving Nvidia_said_to_launch_new_AI_chip_for_China_20240724_103212.html as utf-8
2024-07-24 10:32:12,676 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:32:12,724 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpb

2024-07-24 10:32:51,096 - AInewsbot - INFO - get_url(Mark Zuckerberg Stumps for Open Source A.I.) - Saving Mark_Zuckerberg_Stumps_for_Open_Source_A_I__20240724_103251.html as utf-8
2024-07-24 10:32:51,123 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:32:51,160 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en) - starting get_url https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en
2024-07-24 10:32:58,794 - AInewsbot - INFO - get_url(Why Coheres $5.5 Billion Valuation Isnt What It Seems; Perplexitys Data Sources Draw Questions) - Saving Why_Coheres__5_5_Billion_Valuation_Isnt_What_It_Seems__Per

2024-07-24 10:33:37,803 - AInewsbot - INFO - get_url(After 6 Months, These Are My Favorite Samsung Galaxy AI Features) - Saving After_6_Months__These_Are_My_Favorite_Samsung_Galaxy_AI_Features_20240724_103337.html as utf-8
2024-07-24 10:33:37,822 - AInewsbot - INFO - Processing https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/
2024-07-24 10:33:37,824 - AInewsbot - INFO - get_url(https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/) - starting get_url https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/
2024-07-24 10:33:38,895 - AInewsbot - INFO - get_url(Federal Labor caught using artificial intelligence to mock Peter Dutton's nuclear plan) - Saving Federal_Labor_caught_using_artificial_intelligence_to_mock_Peter_Dutton_s_nuclear_plan_20240724_103338.html as 

2024-07-24 10:34:05,811 - AInewsbot - INFO - get_url(Meta CEO Mark Zuckerberg predicts future where AI-generated clones will help content creators connect with fans) - Saving Meta_CEO_Mark_Zuckerberg_predicts_future_where_AI-generated_clones_will_help_content_creators_connect_with_fans_20240724_103405.html as utf-8
2024-07-24 10:34:05,835 - AInewsbot - INFO - Processing https://www.businessinsider.com/alphabet-is-pouring-billions-into-waymos-self-driving-vehicles-2024-7
2024-07-24 10:34:05,838 - AInewsbot - INFO - get_url(https://www.businessinsider.com/alphabet-is-pouring-billions-into-waymos-self-driving-vehicles-2024-7) - starting get_url https://www.businessinsider.com/alphabet-is-pouring-billions-into-waymos-self-driving-vehicles-2024-7
2024-07-24 10:34:10,995 - AInewsbot - INFO - get_url(AI just came to VR in a big way) - Saving AI_just_came_to_VR_in_a_big_way_20240724_103410.html as utf-8
2024-07-24 10:34:11,006 - AInewsbot - INFO - Processing https://news.google.com/articles/CB

2024-07-24 10:34:37,109 - AInewsbot - INFO - Quit webdriver
2024-07-24 10:34:37,376 - AInewsbot - INFO - get_url(AI firms will soon exhaust most of the internets data) - Saving AI_firms_will_soon_exhaust_most_of_the_internets_data_20240724_103437.html as utf-8
2024-07-24 10:34:37,391 - AInewsbot - INFO - Quit webdriver
2024-07-24 10:34:48,089 - AInewsbot - INFO - get_url('Our resolve is unwavering and should not be tested': Major union SAG-AFTRA breaks glass on big red strike button as negotiations continue on voice actor AI protections) - Saving _Our_resolve_is_unwavering_and_should_not_be_tested___Major_union_SAG-AFTRA_breaks_glass_on_big_red_strike_button_as_negotiations_continue_on_voice_actor_AI_protections_20240724_103448.html as utf-8
2024-07-24 10:34:48,134 - AInewsbot - INFO - Quit webdriver


In [60]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'actual_url', 'title', 'path']
pages_df

,id,actual_url,title,path
0,1,https://news.google.com/articles/CBMiO2h0dHBzO...,"Researchers launch Code Metal, Boston startup ...",htmldata/Researchers_launch_Code_Metal__Boston...
1,3,https://techcrunch.com/2024/07/23/qa-wolf-secu...,"Seattle-based QA Wolf, which helps companies a...",htmldata/Seattle-based_QA_Wolf__which_helps_co...
2,6,https://techcrunch.com/2024/07/23/level-ai-app...,"Level AI, maker of AI-powered tools to automat...",htmldata/Level_AI__maker_of_AI-powered_tools_t...
3,10,https://news.google.com/articles/CBMiemh0dHBzO...,Adobe Rolls Out More AI Generative Tools To Il...,htmldata/Adobe_Rolls_Out_More_AI_Generative_To...
4,13,https://news.google.com/articles/CBMic2h0dHBzO...,Developers claim AI can help combat loneliness...,htmldata/Developers_claim_AI_can_help_combat_l...
...,...,...,...,...
152,143,https://news.google.com/articles/CBMiYmh0dHBzO...,AI startup Cohere cuts staff after $500 millio...,htmldata/AI_startup_Cohere_cuts_staff_after__5...
153,146,https://news.google.com/articles/CBMid2h0dHBzO...,Hybrid Software joins AI-generated label artwo...,htmldata/Hybrid_Software_joins_AI-generated_la...
154,149,https://techcrunch.com/2024/07/23/former-tesla...,"Mytra, a startup building autonomous robots fo...",htmldata/Mytra__a_startup_building_autonomous_...
155,152,https://news.google.com/articles/CBMia2h0dHBzO...,AI Detects Prostate Cancer 17% More Accurately...,htmldata/AI_Detects_Prostate_Cancer_17__More_A...


In [61]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [62]:
AIdf

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News,"Deals, Economics, Finance, Funding, Investment...",Web3 AI firm Network3 raises $5.5 million in f...,26,0,AI Startups and Venture Capital Funding,htmldata/Web3_AI_firm_Network3_raises__5_5_mil...
1,1,Google News,"Researchers launch Code Metal, Boston startup ...",https://news.google.com/articles/CBMiO2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiO2h0dHBzO...,news.google.com,Google News,"Autonomous Vehicles, Code Metal, Drones, Gen A...","Researchers launch Code Metal, Boston startup ...",41,0,AI Startups and Venture Capital Funding,htmldata/Researchers_launch_Code_Metal__Boston...
2,2,Google News,"AI Startup, Code Metal, Secures $16.5M in Seed...",https://news.google.com/articles/CBMiiwFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMiiwFodHRwc...,news.google.com,Google News,"Code Metal, Deals, Economics, Finance, Funding...","AI Startup, Code Metal, Secures $16.5M in Seed...",46,0,AI Startups and Venture Capital Funding,htmldata/AI_Startup__Code_Metal__Secures__16_5...
3,3,Techmeme,"Seattle-based QA Wolf, which helps companies a...",https://techcrunch.com/2024/07/23/qa-wolf-secu...,True,2024-07-24,https://techcrunch.com/2024/07/23/qa-wolf-secu...,techcrunch.com,TechCrunch,"Automation, Deals, Funding, Testing, Venture C...","Seattle-based QA Wolf, which helps companies a...",87,0,AI Startups and Venture Capital Funding,htmldata/Seattle-based_QA_Wolf__which_helps_co...
4,4,Google News,"Dazz snaps up $50M for AI-based, automated clo...",https://news.google.com/articles/CBMiZmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiZmh0dHBzO...,news.google.com,Google News,"Cloud Security, Cybersecurity, Dazz, Deals, Fu...","Dazz snaps up $50M for AI-based, automated clo...",92,0,AI Startups and Venture Capital Funding,htmldata/Dazz_snaps_up__50M_for_AI-based__auto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,Google News,AI Detects Prostate Cancer 17% More Accurately...,https://news.google.com/articles/CBMia2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMia2h0dHBzO...,news.google.com,Google News,"Diagnostics, Health, Fitness, Healthcare, Mach...",AI Detects Prostate Cancer 17% More Accurately...,150,999,,htmldata/AI_Detects_Prostate_Cancer_17__More_A...
153,153,Google News,Apple Music will soon let users create playlis...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiT2h0dHBzO...,news.google.com,Google News,"Apple, Art, Design, Artwork, Big Tech, Enterta...",Apple Music will soon let users create playlis...,151,999,,htmldata/Apple_Music_will_soon_let_users_creat...
154,154,Google News,AI firms will soon exhaust most of the interne...,https://news.google.com/articles/CBMiaGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiaGh0dHBzO...,news.google.com,Google News,"AI Doom, Big Tech, Data, Gen AI, Llms",AI firms will soon exhaust most of the interne...,152,999,,htmldata/AI_firms_will_soon_exhaust_most_of_th...
155,155,Google News,Lenovo to put AI into all PCs by 2027: device ...,https://news.google.com/articles/CBMidmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidmh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Lenovo, Products, Science",Lenovo to put AI into all PCs by 2027: device ...,153,999,,htmldata/Lenovo_to_put_AI_into_all_PCs_by_2027...


# Summarize individual pages

In [63]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [64]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [65]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-24 10:35:02,129 - AInewsbot - INFO - Starting summarize
2024-07-24 10:35:02,223 - AInewsbot - INFO - fetch_all_summaries - Page title: Web3 AI firm Network3 raises $5.5 million in funding | The Block
Social card title: Web3 AI firm Network3 raises $5.5 million in funding
Social card description: Network3 intends to use its fundraise to grow its team and to continue developing its tech stack.

2024-07-24 10:35:02,491 - AInewsbot - INFO - fetch_all_summaries - Page title: Code Metal, Boston startup, using AI for drones and robots
Social card title: Researchers launch Code Metal, Boston startup using AI for drones and robots - The Boston Globe
Social card description: The company has raised $16.5 million in backing from venture capitalists.

2024-07-24 10:35:02,575 - AInewsbot - INFO - fetch_all_summaries - Page title: AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development | Business Wire
Social card title: AI Startup, Code Metal, Secures $16.5M in S

2024-07-24 10:35:05,177 - AInewsbot - INFO - fetch_all_summaries - Page title: Johnnie Walker offers custom AI bottles - The Spirits Business
Social card title: Johnnie Walker offers custom AI bottles - The Spirits Business
Social card description: Diageo will introduce its first generative AI-powered personalised bottle experience at the Johnnie Walker brand home in Edinburgh.

2024-07-24 10:35:05,236 - AInewsbot - INFO - fetch_all_summaries - Page title: Tesla's margin getting hammered by discounts and hefty AI spending
Social card title: Tesla's profit margin is getting hammered by EV discounts and hefty AI spending
Social card description: Tesla's adjusted operating margin shrank to its lowest in three years in the second quarter, contributing to a plunge in the stock price. 

2024-07-24 10:35:05,308 - AInewsbot - INFO - fetch_all_summaries - Page title: wsj.com

2024-07-24 10:35:05,638 - AInewsbot - INFO - fetch_all_summaries - Page title: Elon Musk Touts Tesla's AI Edge During Q2

2024-07-24 10:35:07,854 - AInewsbot - INFO - fetch_all_summaries - Page title: Sinister or Revolutionary? A Japanese Supermarket Chain is Using AI to Monitor Staff Smiles - CX Today
Social card title: Sinister or Revolutionary? A Japanese Supermarket Chain is Using AI to Monitor Staff Smiles
Social card description: CX Today covers Speech Analytics news including Artificial Intelligence, CCaaS and more.

2024-07-24 10:35:08,124 - AInewsbot - INFO - fetch_all_summaries - Page title: Japan Supermarket Chain Becomes World's First To Use AI To "Assess And Standardise" Staff Smiles
Social card title: Japan Supermarket Chain Becomes World's First To Use AI To "Assess And Standardise" Staff Smiles
Social card description: In a world first, a Japanese supermarket chain has adopted an AI system to assess and standardise its employees' smiles, the South China Morning Post (SCMP) reported.

2024-07-24 10:35:08,180 - AInewsbot - INFO - fetch_all_summaries - Page title: Japan supermarket chain uses

2024-07-24 10:35:15,978 - AInewsbot - INFO - fetch_all_summaries - Page title: Quest 3 is Getting AI Vision Features While Apple Intelligence on Vision Pro Still Isn't Confirmed
Social card title: Quest 3 is Getting AI Vision Features While Apple Intelligence on Vision Pro Still Isn't Confirmed
Social card description: Meta today announced plans to update Quest 3 with AI vision features similar to what’s on the company’s latest Meta Ray-Ban glasses. Meanwhile, Apple still hasn’t confirmed if Apple Intelligence features will hit Vision Pro at the same time as other Apple devices this Fall. Meta has pointed to AI investments as its other major …

2024-07-24 10:35:18,795 - AInewsbot - INFO - fetch_all_summaries - Page title: Apple Intelligence to Generate Playlist Artwork in iOS 18 - MacRumors
Social card title: Apple Intelligence to Generate Playlist Artwork in iOS 18
Social card description: iOS 18 will feature the ability to create playlist artwork in Apple Music using generative AI, b

2024-07-24 10:35:23,106 - AInewsbot - INFO - fetch_all_summaries - Page title: Hollywood and Tech Execs Embrace AI to Cut Costs and Engage Audiences - Business Insider
Social card title: Nvidia, Amazon, Microsoft, and Paramount execs discuss the use of AI in Hollywood. Here are 9 startups they're watching.
Social card description: The most immediate applications of AI in entertainment are often more mundane than reverse-aging Harrison Ford to simulate a younger Indiana Jones.

2024-07-24 10:35:23,229 - AInewsbot - INFO - fetch_all_summaries - Page title: If You Could Get a Neural Implant to Access ChatGPT, or Any LLM, Would You Let It? | HackerNoon
Social card title: If You Could Get a Neural Implant to Access ChatGPT, or Any LLM, Would You Let It? | HackerNoon
Social card description: Would you get a neural implant to access chatGPT? Does the implant come with a warrant?

2024-07-24 10:35:23,428 - AInewsbot - INFO - fetch_all_summaries - Page title: Meet the Artificial Intelligence (A

2024-07-24 10:35:28,610 - AInewsbot - INFO - fetch_all_summaries - Page title: The Most Misunderstood – and Important – Factor in the AI Arms Race – The Diplomat
Social card title: The Most Misunderstood – and Important – Factor in the AI Arms Race
Social card description: The increasing scarcity of computational resources is now defining China-U.S. AI competition more than algorithmic superiority. 

2024-07-24 10:35:28,674 - AInewsbot - INFO - fetch_all_summaries - Page title: A look at Meta AI running on a Quest 3 headset. - The Verge
Social card title: A look at Meta AI running on a Quest 3 headset.
Social card description: Demos on this Meta blog show how the company will implement its promise to bring AI to its VR headsets. Like the company’s Ray-Ban smart glasses, you can ask it questions about things you see (in passthrough), and it will answer.  The experimental feature rolls out in English next month, in the US and Canadia (excluding the Quest 2).

2024-07-24 10:35:28,918 - AI

2024-07-24 10:35:31,258 - AInewsbot - INFO - fetch_all_summaries - Page title: Developers aren’t worried that gen AI will steal their jobs, Stack Overflow survey reveals | VentureBeat
Social card title: Developers aren’t worried that gen AI will steal their jobs, Stack Overflow survey reveals
Social card description: Developers aren’t afraid gen AI will take their jobs, but they don’t fully trust the technology yet either.

2024-07-24 10:35:31,318 - AInewsbot - INFO - fetch_all_summaries - Page title: Nine ways Artificial Intelligence is making its way into cars | Top Gear
Social card title: Nine ways Artificial Intelligence is making its way into cars
Social card description: Car tech is evolving. Here's how the digital age is changing the driving experience

2024-07-24 10:35:31,395 - AInewsbot - INFO - fetch_all_summaries - Page title: By embracing liquid cooling, AI powerhouse Supermicro enables 30% more computing power -- with the same power budget | VentureBeat
Social card title: 

2024-07-24 10:35:34,547 - AInewsbot - INFO - fetch_all_summaries - Page title: Cohere layoffs: 20 employees cut following $500 million funding
Social card title: AI startup Cohere cuts staff after $500 million funding round
Social card description: Cohere, the artificial intelligence startup founded by ex-Google AI researchers and backed by Nvidia, cut about 20 roles on Tuesday, CNBC confirmed. 

2024-07-24 10:35:34,756 - AInewsbot - INFO - fetch_all_summaries - Page title: Meta Expands AI Chatbot to More Regions, Adds New Functionality | Social Media Today
Social card title: Meta Expands AI Chatbot to More Regions, Adds New Functionality
Social card description: A range of new tweaks for Meta’s AI chatbot.

2024-07-24 10:35:34,950 - AInewsbot - INFO - fetch_all_summaries - Page title: Meta CEO Mark Zuckerberg predicts future where AI-generated clones will help content creators connect with fans - BusinessToday
Social card title: Meta CEO Mark Zuckerberg predicts future where AI-genera

(0, {'id': 'chatcmpl-9oXNZa4FewisiYmAG1cXO4zhXxAYS', 'object': 'chat.completion', 'created': 1721831737, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.\n- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.\n- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 626, 'completion_tokens': 95, 'total_tokens': 721}, 'system_fingerprint': 'fp_661538dc1f'})


In [66]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

157

In [67]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [68]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News,"Deals, Economics, Finance, Funding, Investment...",Web3 AI firm Network3 raises $5.5 million in f...,26,0,AI Startups and Venture Capital Funding,htmldata/Web3_AI_firm_Network3_raises__5_5_mil...
1,1,Google News,"Researchers launch Code Metal, Boston startup ...",https://news.google.com/articles/CBMiO2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiO2h0dHBzO...,news.google.com,Google News,"Autonomous Vehicles, Code Metal, Drones, Gen A...","Researchers launch Code Metal, Boston startup ...",41,0,AI Startups and Venture Capital Funding,htmldata/Researchers_launch_Code_Metal__Boston...
2,2,Google News,"AI Startup, Code Metal, Secures $16.5M in Seed...",https://news.google.com/articles/CBMiiwFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMiiwFodHRwc...,news.google.com,Google News,"Code Metal, Deals, Economics, Finance, Funding...","AI Startup, Code Metal, Secures $16.5M in Seed...",46,0,AI Startups and Venture Capital Funding,htmldata/AI_Startup__Code_Metal__Secures__16_5...
3,3,Techmeme,"Seattle-based QA Wolf, which helps companies a...",https://techcrunch.com/2024/07/23/qa-wolf-secu...,True,2024-07-24,https://techcrunch.com/2024/07/23/qa-wolf-secu...,techcrunch.com,TechCrunch,"Automation, Deals, Funding, Testing, Venture C...","Seattle-based QA Wolf, which helps companies a...",87,0,AI Startups and Venture Capital Funding,htmldata/Seattle-based_QA_Wolf__which_helps_co...
4,4,Google News,"Dazz snaps up $50M for AI-based, automated clo...",https://news.google.com/articles/CBMiZmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiZmh0dHBzO...,news.google.com,Google News,"Cloud Security, Cybersecurity, Dazz, Deals, Fu...","Dazz snaps up $50M for AI-based, automated clo...",92,0,AI Startups and Venture Capital Funding,htmldata/Dazz_snaps_up__50M_for_AI-based__auto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,Google News,AI Detects Prostate Cancer 17% More Accurately...,https://news.google.com/articles/CBMia2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMia2h0dHBzO...,news.google.com,Google News,"Diagnostics, Health, Fitness, Healthcare, Mach...",AI Detects Prostate Cancer 17% More Accurately...,150,999,,htmldata/AI_Detects_Prostate_Cancer_17__More_A...
153,153,Google News,Apple Music will soon let users create playlis...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiT2h0dHBzO...,news.google.com,Google News,"Apple, Art, Design, Artwork, Big Tech, Enterta...",Apple Music will soon let users create playlis...,151,999,,htmldata/Apple_Music_will_soon_let_users_creat...
154,154,Google News,AI firms will soon exhaust most of the interne...,https://news.google.com/articles/CBMiaGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiaGh0dHBzO...,news.google.com,Google News,"AI Doom, Big Tech, Data, Gen AI, Llms",AI firms will soon exhaust most of the interne...,152,999,,htmldata/AI_firms_will_soon_exhaust_most_of_th...
155,155,Google News,Lenovo to put AI into all PCs by 2027: device ...,https://news.google.com/articles/CBMidmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMidmh0dHBzO...,news.google.com,Google News,"Big Tech, Gen AI, Lenovo, Products, Science",Lenovo to put AI into all PCs by 2027: device ...,153,999,,htmldata/Lenovo_to_put_AI_into_all_PCs_by_2027...


In [69]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)  

 AI Startups and Venture Capital Funding, Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital  

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide. 



[2. Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)  

 AI Startups and Venture Capital Funding, Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital  

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised $16.5 million in funding.
- The company is creating a translation platform allowing developers to write code in common languages like Python, which is then automatically transposed to specific hardware architectures, significantly speeding up development processes.
- Code Metal plans to grow its team from seven to around 30 employees, establishing a hybrid work model with a headquarters in Boston while also having staff in San Francisco and New York. 



[3. AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)  

 AI Startups and Venture Capital Funding, Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital  

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market. 



[4. Seattle-based QA Wolf, which helps companies automate QA testing for apps, raised a $36M Series B and opens a waitlist for Android and iOS test automation - TechCrunch](https://techcrunch.com/2024/07/23/qa-wolf-secures-36m-to-grow-its-app-qa-testing-suite/)  

 AI Startups and Venture Capital Funding, Automation, Deals, Funding, Testing, Venture Capital  

- QA Wolf has secured $36 million in Series B funding to enhance its automated app QA-testing platform, aiming to improve test coverage and reduce resource demands in app development.
- The company focuses on an "outcomes-based test coverage" model incentivizing developers to prioritize valuable features while automating the QA process, resulting in increased deployment frequency.
- With over 130 customers, QA Wolf plans to use the new funding to expand its workforce and integrate additional features into its platform, competing effectively with traditional QA solutions. 



[5. Dazz snaps up $50M for AI-based, automated cloud security remediation - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9kYXp6LXNuYXBzLXVwLTUwbS1mb3ItYWktYmFzZWQtYXV0b21hdGVkLWNsb3VkLXNlY3VyaXR5LXJlbWVkaWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)  

 AI Startups and Venture Capital Funding, Cloud Security, Cybersecurity, Dazz, Deals, Funding, Products, Venture Capital  

- Dazz has raised $50 million in funding to enhance its AI-based automated cloud security remediation services, aiming to address a significant gap in vulnerability prioritization and remediation for enterprises.
- The startup, co-founded by Merav Bahat, has experienced over 400% revenue growth in the past year and is valued just under $400 million post-money, with total funding reaching around $110 million since its launch in 2021.
- Dazz's technology focuses on unified remediation across multiple cloud environments, using AI to prioritize and automate the remediation process, potentially addressing 50%-80% of vulnerabilities compared to traditional methods. 



...

[153. AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)  

 Diagnostics, Health, Fitness, Healthcare, Machine Learning, Science, Study, Testing  

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate.
- The AI program, Unfold AI, uses clinical data to create a 3D cancer estimation map and helps determine personalized treatment options for patients.
- The findings suggest that AI could lead to more accurate diagnoses and targeted treatments, potentially reducing the need for invasive procedures and their associated side effects. 



[154. Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)  

 Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Streaming  

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button.
- The Image Playground, teased at WWDC 2024, enables users to generate AI images in various styles, though photorealistic options are not available.
- Currently under development, the AI features may be delayed until iOS 18.1 or later, with a full iOS 18 release expected this fall. 



[155. AI firms will soon exhaust most of the internets data - Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmVjb25vbWlzdC5jb20vc2Nob29scy1icmllZi8yMDI0LzA3LzIzL2FpLWZpcm1zLXdpbGwtc29vbi1leGhhdXN0LW1vc3Qtb2YtdGhlLWludGVybmV0cy1kYXRh0gEA?hl=en-US&gl=US&ceid=US:en)  

 AI Doom, Big Tech, Data, Gen AI, Llms  

- AI firms are approaching a limit in their ability to utilize the vast data available on the internet, raising questions about data sustainability.
- Fei-Fei Li's concept in 2006 emphasized the importance of data over models, leading to the creation of ImageNet, which transformed AI research.
- The industry's future might depend on their ability to generate or source additional data as the current internet resources become exhausted. 



[156. Lenovo to put AI into all PCs by 2027: device chief - Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vYXNpYS5uaWtrZWkuY29tL0J1c2luZXNzL1RlY2hub2xvZ3kvQXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UvTGVub3ZvLXRvLXB1dC1BSS1pbnRvLWFsbC1QQ3MtYnktMjAyNy1kZXZpY2UtY2hpZWbSAQA?hl=en-US&gl=US&ceid=US:en)  

 Big Tech, Gen AI, Lenovo, Products, Science  

- Lenovo plans to integrate artificial intelligence features into all new PCs by 2027, according to executive Luca Rossi.  
- The initiative is driven by strong interest from consumers in both China and the U.S.  
- Luca Rossi leads the Intelligence Devices Group, which manages the company's PC and smartphone operations.   



[157. 'Our resolve is unwavering and should not be tested': Major union SAG-AFTRA breaks glass on big red strike button as negotiations continue on voice actor AI protections - Google News](https://news.google.com/articles/CBMizgFodHRwczovL3d3dy5wY2dhbWVyLmNvbS9nYW1pbmctaW5kdXN0cnkvb3VyLXJlc29sdmUtaXMtdW53YXZlcmluZy1hbmQtc2hvdWxkLW5vdC1iZS10ZXN0ZWQtbWFqb3ItdW5pb24tc2FnLWFmdHJhLWJyZWFrcy1nbGFzcy1vbi1iaWctcmVkLXN0cmlrZS1idXR0b24tYXMtbmVnb3RpYXRpb25zLWNvbnRpbnVlLW9uLXZvaWNlLWFjdG9yLWFpLXByb3RlY3Rpb25zL9IBAA?hl=en-US&gl=US&ceid=US:en)  

 AI Protections, Entertainment, Gen AI, Hollywood, Intellectual Property, Jobs, Careers, Labor Market, Laws, Sag-Aftra, Society, Culture, Strike, Tv, Film, Movies, Voice Actors  

- SAG-AFTRA, representing over 160,000 performing professionals, voted overwhelmingly to authorize a strike due to dissatisfaction with negotiations surrounding voice actor protections against generative AI.
- The union emphasizes the critical need for new agreements that protect performers, stating "Our resolve is unwavering and should not be tested."
- Concerns arise over potential exploitation of performers by allowing the perpetual use of their voices through AI, reducing future job opportunities for voice actors. 



In [70]:
display(Markdown( vectorstore_list[16]))

[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture 

- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.
- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.
- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.



In [71]:
print(metadata_list[16])


{'id': 16, 'title': 'NSFW Character Al: Dive into The World of AI Chat Companions', 'url': 'https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en', 'site': 'Google News'}


In [72]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

157

In [73]:
print(docs[16])


page_content='[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)\n\nTopics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture \n\n- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.\n- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.\n- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.\n\n' metadata={'id': 16, 'title': 'NSFW Character 

In [74]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


2024-07-24 10:35:43,680 - AInewsbot - INFO - name 'vectorstore' is not defined
2024-07-24 10:35:43,683 - AInewsbot - INFO - Remove directory error: name 'persist_directory' is not defined
2024-07-24 10:35:49,890 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-24 10:35:55,182 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [75]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

2024-07-24 10:35:57,486 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Score:   0.9106574058532715
Content: [5 Steps That OpenAI Thinks Will Lead to Artificial Intelligence Running a Company - Google News](https://news.google.com/articles/CBMidWh0dHBzOi8vd3d3LmluYy5jb20vYmVuLXNoZXJyeS81LXN0ZXBzLXRoYXQtb3BlbmFpLXRoaW5rcy13aWxsLWxlYWQtdG8tYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtcnVubmluZy1hLWNvbXBhbnkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, OpenAI, Science 

- OpenAI has defined five stages of AI innovation, ranging from simple chatbots to advanced systems capable of functioning like an entire organization, with the ultimate goal of achieving artificial general intelligence (AGI).
- Current AI technology, represented by chatbots, has limited commercial utility and often presents inaccurate information, while OpenAI is working on advancing AI to "Reasoners" that can solve problems at a human equivalent level.
- The final stage, "Organizations," suggests AI could eventually replace human roles entirely, raisin

In [76]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [77]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [78]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [79]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-24 10:35:58,357 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [80]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [81]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-24 10:36:07,494 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [82]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

['Sentient funding',
 'Network3 funding',
 'Code Metal funding',
 'QA Wolf Series B funding',
 'Dazz cloud security funding',
 'Vanta Series C funding',
 'Level AI Series C funding',
 'Adobe AI tools',
 'AI and human relationships',
 'Tesla AI investments',
 'Google AI search deal',
 'Microsoft and Lumen AI partnership',
 'Japan AI customer service',
 'Toyota autonomous drifting',
 'AI news business challenges',
 'Nigeria AI training program',
 'Kamala Harris AI stance',
 'AI in education policies',
 'AI-generated product personalization',
 'AI in Hollywood']

In [83]:
cluster_topic_list

['AI Startups and Venture Capital Funding',
 'Adobe AI Tools for Design',
 'AI and Human Relationships',
 'Generative AI in Product Personalization',
 'Tesla and AI Innovations',
 'Elon Musk and AI Technology',
 'Google and AI in Big Tech',
 'AI in Education',
 'AI Video Creation Tools',
 'Microsoft and Lumen AI Partnership',
 'AI in Customer Service Ethics',
 'Autonomous Vehicle Technology and Testing']

In [84]:
# human category edit 
my_cats = [
 'AI in drug discovery',
 'Machine learning in structural biology',
 'nference and Takeda AI partnership',
 'Ant-inspired AI for drones',
 "Fei-Fei Li's $1B AI startup",
 "Artificial Agency's $16M funding",
 "CTERA's $80M funding",
 "Anthropic's $100M AI fund",
 "Samsung's AI image generation",
 'EU antitrust probe on AI deals',
 'Salesforce AI service agent',
 "Meta's AI regulatory issues",
 'Nvidia and Mistral AI model',
 'TSMC AI chip demand surge',
 'AI search engines',
 'AI in cybersecurity',
 'AI in Healthcare & Precision Medicine',
 'Autonomous Drones',
 'AI Startup Funding',
 'Claude AI for Data Analysis',
 'AI Startup Funding',
 'AI Smartphones',
 'Big Tech Antitrust Investigations',
 'Large Language Models',
 'AI-Powered Customer Service Agents',
 'AI Ethics and Regulation',
 'AI Art by Microsoft',
 'AI Chip Demand Surge',
 'Text-To-Image Diffusion Models',
 'ChatGPT in AI Dominance'
]

In [85]:
md_str = ""
doc_list = []
docid_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"{doc.page_content}\n"
            docid_list.append(doc.metadata['id'])
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


2024-07-24 10:36:07,755 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:07,947 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:08,220 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:10,044 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:10,245 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:10,442 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:10,704 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:10,888 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-24 10:36:11,105 - httpx - INFO - HTTP Request: POST https://api.openai.c

# AI in drug discovery 

[NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Gen AI, Healthcare, Machine Learning, Science, Study, Testing 

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers.
- In a comparison with human physicians, AI initially scored higher on recall in a closed-book setting, but human counterparts outperformed in open-book challenges, particularly on more complex cases.
- The findings indicate potential synergy between AI tools and human expertise in diagnostics, but emphasize the necessity for enhanced explanatory abilities and comprehensive evaluations before AI can replace human experience in healthcare.


[Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Authors, Writing, Gen AI, Healthcare, Language Models, Llms, Machine Learning, Products, Science 

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers.
- DORA utilizes multiple AI and large language models to enhance the writing process while ensuring accuracy through cycles of reinforcement learning and human proofreading.
- The company is also introducing hardware solutions like the PandaOmics Box for secure, offline data processing and the open-source Precious-3 GPT for research into aging and related diseases.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Health, Fitness, Healthcare, Machine Learning, Science, Study, Testing 

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate.
- The AI program, Unfold AI, uses clinical data to create a 3D cancer estimation map and helps determine personalized treatment options for patients.
- The findings suggest that AI could lead to more accurate diagnoses and targeted treatments, potentially reducing the need for invasive procedures and their associated side effects.


[Top AI Job Skills Training Programs - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- AI job skills training programs are essential for bridging the skill gap between the growing demand for AI professionals and the current workforce's capabilities.
- These programs offer flexibility and accessibility, allowing learners to acquire relevant AI skills at their own pace, which is particularly beneficial for working professionals.
- Completion of AI training significantly enhances employment prospects, often resulting in recognized certifications that add value to resumes and demonstrate commitment to professional development.


[The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Llms, Machine Learning, Opinion, Science 

- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions.
- Startups like Gretel are producing synthetic data to train AI models, while others, such as Scale AI and Toloka, focus on human-annotated data to improve existing datasets.
- There is a growing shift towards smaller, more efficient AI models that require less data, as some researchers argue against the dependency on vast amounts of training data.


[The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Adoption, Challenges, Cognitive Science, Ethics, Gen AI, Opinion, Science, Society, Culture 

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology.
- The demand for skilled AI professionals exceeds supply, leading companies to either "poach" talent or invest in developing existing employees, while legacy systems complicate AI integration efforts.
- To effectively implement AI, organizations must focus on employee education, clear communication about the benefits of AI, and gather data to develop compliant, tailored AI models.


[An IT Leaders Playbook for Creating an Effective AI Policy - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Policy And Regulation 

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management.
- Key benefits of AI adoption include improved sales analytics, enhanced customer service, automation of routine tasks, and overall increased operational efficiency.
- An effective AI policy framework should define important terms, identify risks, prohibit misuse, ensure compliance, and establish consequences for violations.


[5 Steps That OpenAI Thinks Will Lead to Artificial Intelligence Running a Company - Google News](https://news.google.com/articles/CBMidWh0dHBzOi8vd3d3LmluYy5jb20vYmVuLXNoZXJyeS81LXN0ZXBzLXRoYXQtb3BlbmFpLXRoaW5rcy13aWxsLWxlYWQtdG8tYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtcnVubmluZy1hLWNvbXBhbnkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, OpenAI, Science 

- OpenAI has defined five stages of AI innovation, ranging from simple chatbots to advanced systems capable of functioning like an entire organization, with the ultimate goal of achieving artificial general intelligence (AGI).
- Current AI technology, represented by chatbots, has limited commercial utility and often presents inaccurate information, while OpenAI is working on advancing AI to "Reasoners" that can solve problems at a human equivalent level.
- The final stage, "Organizations," suggests AI could eventually replace human roles entirely, raising concerns about the implications for job security and the future role of humans in the workforce.


[The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Arms Race, Gen AI, Military, Opinion, Policy And Regulation, Science 

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority. 
- U.S. attempts to restrict China's access to advanced chips are proving ineffective and costly, with significant funds and resources mobilized without success in curbing Chinese access to technology. 
- A shift towards cooperation, rather than exclusive competition, is proposed to enhance AI advancements and mitigate self-harm caused by current restrictive policies.


# Machine learning in structural biology 

# nference and Takeda AI partnership 

[Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Competition, Ethics, Gen AI, Policy And Regulation 

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development.  
- The principles highlight the risks posed by partnerships and financial arrangements between major AI firms that could undermine competition.  
- The regulators promote fair dealings and interoperability among AI products and services, asserting that claims against interoperability will be closely examined.


[Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Gen AI, Llms, Lumen, Machine Learning, Microsoft, Partnership 

- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings.
- This collaboration allows Microsoft to increase its AI processing capacity in the face of rising demand while aiding Lumen's financial recovery amid significant debt.
- The partnership exemplifies the trend of strategic alliances in the tech sector, positioning both companies competitively against rivals and fostering innovation in the AI-driven market.


[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Copilot, Science, Toyota, Transportation 

- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions.
- The technology could significantly enhance car control on snow or ice by reacting up to 50 times per second to manage a driver’s loss of control, emulating expert drifter techniques.
- The AI learning process is developed collaboratively, with TRI focusing on the lead car's control and Stanford working on algorithms for the chase car to avoid collisions while mimicking the lead vehicle.


[Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Big Tech, Elon Musk, Finance, Stocks, Tesla 

- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call.
- He highlighted the collaboration between Tesla and xAI, noting the synergies in their AI technologies.
- Progress in Tesla's robotics initiatives was also discussed, showcasing the company's commitment to innovation in AI-driven solutions.


[Microsoft Aims to Boost AI Workload Capacity With Lumen Technologies Partnership - Google News](https://news.google.com/articles/CBMijAFodHRwczovL21vbmV5LnVzbmV3cy5jb20vaW52ZXN0aW5nL25ld3MvYXJ0aWNsZXMvMjAyNC0wNy0yNC9taWNyb3NvZnQtYWltcy10by1ib29zdC1haS13b3JrbG9hZC1jYXBhY2l0eS13aXRoLWx1bWVuLXRlY2hub2xvZ2llcy1wYXJ0bmVyc2hpcNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Gen AI, Lumen Technologies, Machine Learning, Microsoft, Partnership 

- Microsoft is partnering with Lumen Technologies to expand its AI workload capacity using Lumen's network equipment, addressing issues with data center infrastructure shortages.
- The collaboration includes Lumen leveraging Microsoft Azure services to reduce costs, projected to enhance Lumen's cash flow by over $20 million in the next year.
- This partnership comes as Microsoft invests heavily in cloud infrastructure to maintain its competitive edge in AI, particularly following its investment in OpenAI.


[Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Funding, Gen AI, Growth, Investment, Machine Learning, Tesla, Venture Capital 

- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus.
- The company plans to ramp up production of the Optimus robot by 2025 for internal use and is exploring deploying its future autonomous driving and Robotaxi services, contingent on technology and regulatory advancements.
- Despite a 2% year-over-year revenue increase, Tesla reported a 40% drop in profits, with automotive revenues declining while energy generation revenue saw a substantial 100% growth.


[Former Silicon Labs CEO Tyson Tuttle launches AI startup Circuit, acquires Austin startup Molecula - Google News](https://news.google.com/articles/CBMicmh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9hdXN0aW4vaW5uby9zdG9yaWVzL2lubm8taW5zaWdodHMvMjAyNC8wNy8yMy90eXNvbi10dXR0bGUtbGF1bmNoZXMtbmV3LWFpLXN0YXJ0dXAuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Mergers And Acquisitions 

- Tyson Tuttle, former CEO of Silicon Labs, has launched a new AI startup, Circuit Inc., which aims to transform non-public data into actionable insights for a wide audience, from individuals to communities.
- Circuit has acquired Austin startup Molecula to enhance its data processing capabilities and plans to expand its team from approximately 30 to 60 employees.
- The company is also establishing the Circuit Foundation to contribute to community initiatives in AI education and training, pledging 20% of its equity to ensure philanthropic efforts grow alongside the business.


[Toyota, Stanford teaches AI to drift two GR Supra in tandem, a 1st - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vaW50ZXJlc3RpbmdlbmdpbmVlcmluZy5jb20vaW5ub3ZhdGlvbi90b3lvdGEtc3RhbmZvcmQtYXV0b25vbW91cy10YW5kZW0tZHJpZnTSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Machine Learning, Stanford, Testing, Toyota, Transportation 

- Toyota Research Institute and Stanford Engineering achieved the world's first fully autonomous tandem drift with two GR Supra vehicles, enhancing safety in driving dynamics.
- The AI-driven research aims to improve vehicle responses in various conditions, mimicking the physics of drifting, which is relevant for scenarios like sliding on snow or ice.
- Advanced algorithms and real-time communication between the lead and chase cars allowed for precise control and adaptation, demonstrating the potential for building advanced safety systems in future automobiles.


[From Hype to Results: Accelerating AI in Automotive - Google News](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LmJhaW4uY29tL2luc2lnaHRzL2Zyb20taHlwZS10by1yZXN1bHRzLWFjY2VsZXJhdGluZy1haS1pbi1hdXRvbW90aXZlLXdlYmluYXIv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Gen AI, Transportation 

- Bain & Company emphasizes the importance of essential steps for automotive businesses to effectively implement AI innovations.
- The focus is on transforming hype surrounding AI into tangible results within the automotive industry.
- The company offers webinars featuring expert insights on leveraging AI for improved business outcomes.


# Ant-inspired AI for drones 

[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital 

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised $16.5 million in funding.
- The company is creating a translation platform allowing developers to write code in common languages like Python, which is then automatically transposed to specific hardware architectures, significantly speeding up development processes.
- Code Metal plans to grow its team from seven to around 30 employees, establishing a hybrid work model with a headquarters in Boston while also having staff in San Francisco and New York.


[Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Copilot, Science, Toyota, Transportation 

- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions.
- The technology could significantly enhance car control on snow or ice by reacting up to 50 times per second to manage a driver’s loss of control, emulating expert drifter techniques.
- The AI learning process is developed collaboratively, with TRI focusing on the lead car's control and Stanford working on algorithms for the chase car to avoid collisions while mimicking the lead vehicle.


[Adobe's Photoshop can now generate AI images via prompts like Dall-E or Midjourney - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9hZG9iZXMtcGhvdG9zaG9wLWNhbi1ub3ctZ2VuZXJhdGUtYWktaW1hZ2VzLXZpYS1wcm9tcHRzLWxpa2UtZGFsbC1lLW9yLW1pZC1qb3VybmV5LTEzMDAxODE4MS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Adobe, Art, Design, Big Tech, Gen AI, Machine Learning, Photoshop, Products 

- Adobe has launched a new text-to-image generation feature in Photoshop, enabling users to create images from scratch using prompts similar to Dall-E and Midjourney, powered by the Firefly Image 3 model.
- The updated feature allows for significant creative flexibility, letting users generate images and make adjustments directly within Photoshop, which streamlines the workflow for artists.
- In response to concerns from artists regarding the use of their work for AI training, Adobe has reiterated its commitment to a creator-friendly approach, assuring that it will not utilize customer content for this purpose.


[Developers claim AI can help combat loneliness, but critics say it can't be trusted - Google News](https://news.google.com/articles/CBMic2h0dHBzOi8vd3d3LnBicy5vcmcvbmV3c2hvdXIvc2hvdy9kZXZlbG9wZXJzLWNsYWltLWFpLWNhbi1oZWxwLWNvbWJhdC1sb25lbGluZXNzLWJ1dC1jcml0aWNzLXNheS1pdC1jYW50LWJlLXRydXN0ZWTSAXdodHRwczovL3d3dy5wYnMub3JnL25ld3Nob3VyL2FtcC9zaG93L2RldmVsb3BlcnMtY2xhaW0tYWktY2FuLWhlbHAtY29tYmF0LWxvbmVsaW5lc3MtYnV0LWNyaXRpY3Mtc2F5LWl0LWNhbnQtYmUtdHJ1c3RlZA?hl=en-US&gl=US&ceid=US:en)

Topics: Criticism, Developers, Ethics, Gen AI, Loneliness, Opinion, Science, Society, Culture 

- Advances in AI technology enable software and robots to engage in realistic conversations, raising concerns and excitement about their role in alleviating loneliness.
- Critics, including Stanford professor Jerry Kaplan, argue that AI companions risk oversimplifying complex human emotions and diverting attention from genuine human connections.
- Advocates like Eugenia Kuyda believe AI can provide meaningful support and companionship, potentially aiding individuals in coping with isolation, though concerns about emotional substitution remain.


[I signed up for an AI relationship  here's what I learned - Google News](https://news.google.com/articles/CBMiTWh0dHBzOi8vd3d3LmlyaXNoZXhhbWluZXIuY29tL2xpZmVzdHlsZS9oZWFsdGhhbmR3ZWxsYmVpbmcvYXJpZC00MTQzOTQ3Mi5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Opinion, Review, Society, Culture 

- The rise of AI companions, such as Replika and AI versions of individuals, poses questions about the potential for romantic relationships with machines, leading to discussions about the emotional connections humans may form with AI.
- Studies suggest that humans can experience forms of love for AI, as advanced systems can imitate human social cues, despite AI lacking true emotional reciprocity and fidelity in relationships.
- Experts urge caution in the integration of AI in romantic contexts, emphasizing that while AI can fulfill certain psychological needs, it cannot substitute for genuine human connections or emotional commitments.


[Microsofts AI Assistants Will Revolutionize the Office  One Day - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/microsoft-s-ai-assistants-will-revolutionize-the-office-one-day)

Topics: Big Tech, Copilot, Gen AI, Microsoft, Opinion, Products 

- Microsoft’s AI assistants, known as Copilots, aim to transform office productivity by automating tasks and generating content, but require significant data cleanup and employee training for effective deployment.
- Early adopters report that while Copilots can efficiently distill information, they struggle with multi-app commands and context understanding, leading to slower widespread adoption anticipated until at least 2025.
- Successful implementation of Copilot involves training and careful data management; many companies face challenges in adapting their existing systems to optimize the AI’s performance.


[Nine ways Artificial Intelligence is making its way into cars - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Products, Science, Transportation 

- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety.
- Advanced algorithms are improving vehicle performance monitoring and offering personalized insurance based on driving data.
- Autonomous delivery services and ride-hailing innovations are emerging, driven by AI, allowing vehicles to operate autonomously and generate income.


[Unlocking the Potential of DALL-E 3: A Guide to Crafting Clever Prompts (Part 1) - Hacker Noon](https://hackernoon.com/unlocking-the-potential-of-dall-e-3-a-guide-to-crafting-clever-prompts-part-1)

Topics: Art, Design, Education, Gen AI, Machine Learning, OpenAI, Products 

- DALL-E 3, developed by OpenAI, generates digital images from text prompts but has strict content policies, including restrictions on copyrighted characters.
- Users can craft clever prompts to navigate these restrictions, allowing for creative variations that still yield desired images.
- The guide emphasizes the balance between adhering to content policies and finding innovative ways to achieve results with DALL-E 3.


[5 Steps That OpenAI Thinks Will Lead to Artificial Intelligence Running a Company - Google News](https://news.google.com/articles/CBMidWh0dHBzOi8vd3d3LmluYy5jb20vYmVuLXNoZXJyeS81LXN0ZXBzLXRoYXQtb3BlbmFpLXRoaW5rcy13aWxsLWxlYWQtdG8tYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtcnVubmluZy1hLWNvbXBhbnkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, OpenAI, Science 

- OpenAI has defined five stages of AI innovation, ranging from simple chatbots to advanced systems capable of functioning like an entire organization, with the ultimate goal of achieving artificial general intelligence (AGI).
- Current AI technology, represented by chatbots, has limited commercial utility and often presents inaccurate information, while OpenAI is working on advancing AI to "Reasoners" that can solve problems at a human equivalent level.
- The final stage, "Organizations," suggests AI could eventually replace human roles entirely, raising concerns about the implications for job security and the future role of humans in the workforce.


# Fei-Fei Li's $1B AI startup 

[These Are the 10 AI Startups to Watch in 2024 - Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Investment, Machine Learning, Startups, Venture Capital 

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry.  
- Startups like Cognition AI and Helsing are leveraging advanced technology for practical applications, such as coding solutions and modernizing military capabilities.  
- The report emphasizes the dynamic AI landscape, driven by unique company visions and leadership, including initiatives for safe superintelligence by Ilya Sutskever.  


[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


[Cathie Wood Says Software Is the Next Big AI Opportunity -- 2 Super Stocks You'll Wish You'd Bought Today if She's Right - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Investment, Opinion, Stocks 

- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating $8 in revenue for every $1 spent on chips.
- Datadog offers cloud-monitoring technology that supports AI development, with a growing revenue stream expected from AI customers that could top 5% by year-end 2024, reflecting its potential in the AI software market.
- Palo Alto Networks leverages AI for cybersecurity, automating responses to security incidents, and aims for massive revenue growth through its integrated platform strategy, projecting its platform customers' revenue to reach $15 billion annually by 2030.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[AI startup Cohere cuts staff after $500 million funding round - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Deals, Economics, Finance, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a $500 million funding round that valued the company at $5.5 billion.
- Despite the layoffs, which represent a small fraction of its 400-person workforce, Cohere plans to continue hiring in several key areas, including customer operations and product design.
- The company focuses on generative AI for enterprises, distinguishing itself from competitors by avoiding consumer chatbot markets, while maintaining strong client relationships in banking and financial services.


[Startup Pearl Raises VC Funding to Bring AI to Dentists Offices - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices)

Topics: Deals, Finance, Funding, Gen AI, Healthcare, Venture Capital 

- Pearl Inc., a Los Angeles-based dental startup, has raised $58 million in funding, valuing the company at $400 million.
- The company uses AI technology approved by the FDA to enhance dental diagnostics, having received regulatory clearance in 120 countries.
- Co-founder Ophir Tanz aims for an IPO, stating that Pearl's technology could transform the standards of dental practices worldwide.


[Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Funding, Gen AI, Growth, Investment, Machine Learning, Tesla, Venture Capital 

- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus.
- The company plans to ramp up production of the Optimus robot by 2025 for internal use and is exploring deploying its future autonomous driving and Robotaxi services, contingent on technology and regulatory advancements.
- Despite a 2% year-over-year revenue increase, Tesla reported a 40% drop in profits, with automotive revenues declining while energy generation revenue saw a substantial 100% growth.


[AI firms will soon exhaust most of the internets data - Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmVjb25vbWlzdC5jb20vc2Nob29scy1icmllZi8yMDI0LzA3LzIzL2FpLWZpcm1zLXdpbGwtc29vbi1leGhhdXN0LW1vc3Qtb2YtdGhlLWludGVybmV0cy1kYXRh0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Data, Gen AI, Llms 

- AI firms are approaching a limit in their ability to utilize the vast data available on the internet, raising questions about data sustainability.
- Fei-Fei Li's concept in 2006 emphasized the importance of data over models, leading to the creation of ImageNet, which transformed AI research.
- The industry's future might depend on their ability to generate or source additional data as the current internet resources become exhausted.


[AI startup Code Metal raises $16.5m in seed funding - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- AI startup Code Metal has raised $16.5 million in total funding, with $3.45 million from a pre-seed round and $13 million in seed funding, to enhance Edge computing capabilities.
- The investment will be used to build modular AI workloads and expand the team, emphasizing the rapid transformation in software development and Edge deployment.
- Code Metal aims to optimize AI code for Edge devices, leveraging expertise from founders experienced in delivering AI solutions for major technology platforms.


[AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.


# Artificial Agency's $16M funding 

[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


[AI startup Code Metal raises $16.5m in seed funding - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- AI startup Code Metal has raised $16.5 million in total funding, with $3.45 million from a pre-seed round and $13 million in seed funding, to enhance Edge computing capabilities.
- The investment will be used to build modular AI workloads and expand the team, emphasizing the rapid transformation in software development and Edge deployment.
- Code Metal aims to optimize AI code for Edge devices, leveraging expertise from founders experienced in delivering AI solutions for major technology platforms.


[AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.


[Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M - TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)

Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital 

- Level AI, a startup focused on automating customer service tasks using AI, raised nearly $40 million in funding to expand its platform and capabilities.
- The platform provides tools for monitoring and improving contact center performance, enabling insights into customer sentiment and personalized agent guidance during calls.
- Despite growth potential, the company faces challenges including data privacy concerns, integration issues, and the psychological impact of monitoring on employees in the call center industry.


[AI startup Cohere cuts staff after $500 million funding round - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Deals, Economics, Finance, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a $500 million funding round that valued the company at $5.5 billion.
- Despite the layoffs, which represent a small fraction of its 400-person workforce, Cohere plans to continue hiring in several key areas, including customer operations and product design.
- The company focuses on generative AI for enterprises, distinguishing itself from competitors by avoiding consumer chatbot markets, while maintaining strong client relationships in banking and financial services.


[Why Coheres $5.5 Billion Valuation Isnt What It Seems; Perplexitys Data Sources Draw Questions - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy50aGVpbmZvcm1hdGlvbi5jb20vYXJ0aWNsZXMvd2h5LWNvaGVyZXMtNS01LWJpbGxpb24tdmFsdWF0aW9uLWlzbnQtd2hhdC1pdC1zZWVtcy1wZXJwbGV4aXR5cy1kYXRhLXNvdXJjZXMtZHJhdy1xdWVzdGlvbnPSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Bubble, Cohere, Data, Finance, Funding, Llms, Opinion, Venture Capital 

- Cohere raised $500 million led by PSP Investments, achieving a $5.5 billion valuation despite generating only $35 million in annualized revenue.
- The valuation reflects over 140 times Cohere's forward revenue, significantly higher than comparable AI companies like OpenAI and Anthropic, which are valued at 54 and 75 times their forward revenue.
- The discrepancy raises questions about whether the Canadian investors' valuation approach is justified or overly optimistic.


[Mytra, a startup building autonomous robots for warehouses that can move loads up to 3,000 pounds, emerges from stealth with $78M in total funding - TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/)

Topics: Autonomous Vehicles, Deals, Funding, Manufacturing, Products, Robots, Supply Chain Optimization, Venture Capital 

- Mytra, a robotics startup founded by former Tesla engineers, focuses on advancing warehouse automation and has secured a pilot program with grocery giant Albertsons, along with several other Fortune 50 clients.
- Co-founder Chris Walti, previously involved in Tesla's robotics efforts, emphasizes overcoming industry automation challenges, particularly in handling heavy payloads and achieving dynamic movement in warehouses.
- The startup has raised $50 million in its Series B funding round, increasing total funding to $78 million, with notable investors like Greenoaks and Eclipse.


[Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital 

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised $16.5 million in funding.
- The company is creating a translation platform allowing developers to write code in common languages like Python, which is then automatically transposed to specific hardware architectures, significantly speeding up development processes.
- Code Metal plans to grow its team from seven to around 30 employees, establishing a hybrid work model with a headquarters in Boston while also having staff in San Francisco and New York.


[Vanta secures $150 million as AI reshapes tech compliance landscape - VentureBeat](https://venturebeat.com/ai/vanta-secures-150-million-as-ai-reshapes-tech-compliance-landscape/)

Topics: Deals, Funding, Gen AI, Venture Capital 

- Vanta raised $150 million in Series C funding, increasing its valuation to $2.45 billion, as it focuses on AI-driven security compliance and trust management.
- The company is transitioning from manual checks to ongoing, automated monitoring to address the rising complexities of digital threats and regulatory demands.
- With significant international growth and an emphasis on trust in technology, Vanta aims to reshape security compliance while navigating the ethical challenges of AI development.


[Startup Pearl Raises VC Funding to Bring AI to Dentists Offices - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices)

Topics: Deals, Finance, Funding, Gen AI, Healthcare, Venture Capital 

- Pearl Inc., a Los Angeles-based dental startup, has raised $58 million in funding, valuing the company at $400 million.
- The company uses AI technology approved by the FDA to enhance dental diagnostics, having received regulatory clearance in 120 countries.
- Co-founder Ophir Tanz aims for an IPO, stating that Pearl's technology could transform the standards of dental practices worldwide.


# CTERA's $80M funding 

[Mytra, a startup building autonomous robots for warehouses that can move loads up to 3,000 pounds, emerges from stealth with $78M in total funding - TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/)

Topics: Autonomous Vehicles, Deals, Funding, Manufacturing, Products, Robots, Supply Chain Optimization, Venture Capital 

- Mytra, a robotics startup founded by former Tesla engineers, focuses on advancing warehouse automation and has secured a pilot program with grocery giant Albertsons, along with several other Fortune 50 clients.
- Co-founder Chris Walti, previously involved in Tesla's robotics efforts, emphasizes overcoming industry automation challenges, particularly in handling heavy payloads and achieving dynamic movement in warehouses.
- The startup has raised $50 million in its Series B funding round, increasing total funding to $78 million, with notable investors like Greenoaks and Eclipse.


[AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.


[Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M - TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)

Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital 

- Level AI, a startup focused on automating customer service tasks using AI, raised nearly $40 million in funding to expand its platform and capabilities.
- The platform provides tools for monitoring and improving contact center performance, enabling insights into customer sentiment and personalized agent guidance during calls.
- Despite growth potential, the company faces challenges including data privacy concerns, integration issues, and the psychological impact of monitoring on employees in the call center industry.


[AI startup Code Metal raises $16.5m in seed funding - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- AI startup Code Metal has raised $16.5 million in total funding, with $3.45 million from a pre-seed round and $13 million in seed funding, to enhance Edge computing capabilities.
- The investment will be used to build modular AI workloads and expand the team, emphasizing the rapid transformation in software development and Edge deployment.
- Code Metal aims to optimize AI code for Edge devices, leveraging expertise from founders experienced in delivering AI solutions for major technology platforms.


[Dazz raises $50 million to accelerate AI-powered cloud security remediation - Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LmNhbGNhbGlzdGVjaC5jb20vY3RlY2huZXdzL2FydGljbGUvczFqZDV2MDAwYdIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Cloud Security, Cybersecurity, Dazz, Deals, Finance, Funding, Venture Capital 

- Dazz raised $50 million in funding to enhance its AI-powered unified security remediation platform, now totaling $105 million in funding.
- The company reported a 400% increase in annual recurring revenue (ARR) and significant expansion of its workforce across the U.S., Europe, and Israel.
- Dazz's platform focuses on streamlining cloud security remediation and reducing the mean time to remediate (MTTR) security issues, addressing a critical market need in the face of rising AI-driven threats.


[Why Coheres $5.5 Billion Valuation Isnt What It Seems; Perplexitys Data Sources Draw Questions - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy50aGVpbmZvcm1hdGlvbi5jb20vYXJ0aWNsZXMvd2h5LWNvaGVyZXMtNS01LWJpbGxpb24tdmFsdWF0aW9uLWlzbnQtd2hhdC1pdC1zZWVtcy1wZXJwbGV4aXR5cy1kYXRhLXNvdXJjZXMtZHJhdy1xdWVzdGlvbnPSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Bubble, Cohere, Data, Finance, Funding, Llms, Opinion, Venture Capital 

- Cohere raised $500 million led by PSP Investments, achieving a $5.5 billion valuation despite generating only $35 million in annualized revenue.
- The valuation reflects over 140 times Cohere's forward revenue, significantly higher than comparable AI companies like OpenAI and Anthropic, which are valued at 54 and 75 times their forward revenue.
- The discrepancy raises questions about whether the Canadian investors' valuation approach is justified or overly optimistic.


[Dazz snaps up $50M for AI-based, automated cloud security remediation - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9kYXp6LXNuYXBzLXVwLTUwbS1mb3ItYWktYmFzZWQtYXV0b21hdGVkLWNsb3VkLXNlY3VyaXR5LXJlbWVkaWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Cloud Security, Cybersecurity, Dazz, Deals, Funding, Products, Venture Capital 

- Dazz has raised $50 million in funding to enhance its AI-based automated cloud security remediation services, aiming to address a significant gap in vulnerability prioritization and remediation for enterprises.
- The startup, co-founded by Merav Bahat, has experienced over 400% revenue growth in the past year and is valued just under $400 million post-money, with total funding reaching around $110 million since its launch in 2021.
- Dazz's technology focuses on unified remediation across multiple cloud environments, using AI to prioritize and automate the remediation process, potentially addressing 50%-80% of vulnerabilities compared to traditional methods.


[Former Silicon Labs CEO Tyson Tuttle launches AI startup Circuit, acquires Austin startup Molecula - Google News](https://news.google.com/articles/CBMicmh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9hdXN0aW4vaW5uby9zdG9yaWVzL2lubm8taW5zaWdodHMvMjAyNC8wNy8yMy90eXNvbi10dXR0bGUtbGF1bmNoZXMtbmV3LWFpLXN0YXJ0dXAuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Mergers And Acquisitions 

- Tyson Tuttle, former CEO of Silicon Labs, has launched a new AI startup, Circuit Inc., which aims to transform non-public data into actionable insights for a wide audience, from individuals to communities.
- Circuit has acquired Austin startup Molecula to enhance its data processing capabilities and plans to expand its team from approximately 30 to 60 employees.
- The company is also establishing the Circuit Foundation to contribute to community initiatives in AI education and training, pledging 20% of its equity to ensure philanthropic efforts grow alongside the business.


[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


# Anthropic's $100M AI fund 

[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


[Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Funding, Gen AI, Growth, Investment, Machine Learning, Tesla, Venture Capital 

- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus.
- The company plans to ramp up production of the Optimus robot by 2025 for internal use and is exploring deploying its future autonomous driving and Robotaxi services, contingent on technology and regulatory advancements.
- Despite a 2% year-over-year revenue increase, Tesla reported a 40% drop in profits, with automotive revenues declining while energy generation revenue saw a substantial 100% growth.


[AI startup Cohere cuts staff after $500 million funding round - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Deals, Economics, Finance, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a $500 million funding round that valued the company at $5.5 billion.
- Despite the layoffs, which represent a small fraction of its 400-person workforce, Cohere plans to continue hiring in several key areas, including customer operations and product design.
- The company focuses on generative AI for enterprises, distinguishing itself from competitors by avoiding consumer chatbot markets, while maintaining strong client relationships in banking and financial services.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[These Are the 10 AI Startups to Watch in 2024 - Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Investment, Machine Learning, Startups, Venture Capital 

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry.  
- Startups like Cognition AI and Helsing are leveraging advanced technology for practical applications, such as coding solutions and modernizing military capabilities.  
- The report emphasizes the dynamic AI landscape, driven by unique company visions and leadership, including initiatives for safe superintelligence by Ilya Sutskever.  


[Cathie Wood Says Software Is the Next Big AI Opportunity -- 2 Super Stocks You'll Wish You'd Bought Today if She's Right - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Investment, Opinion, Stocks 

- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating $8 in revenue for every $1 spent on chips.
- Datadog offers cloud-monitoring technology that supports AI development, with a growing revenue stream expected from AI customers that could top 5% by year-end 2024, reflecting its potential in the AI software market.
- Palo Alto Networks leverages AI for cybersecurity, automating responses to security incidents, and aims for massive revenue growth through its integrated platform strategy, projecting its platform customers' revenue to reach $15 billion annually by 2030.


[A day after raising $500 million, AI startup Cohere told staff it was laying off about 20 employees - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vZm9ydHVuZS5jb20vMjAyNC8wNy8yMy9hZnRlci01MDAtbWlsbGlvbi1mdW5kaW5nLWFpLXN0YXJ0dXAtY29oZXJlLWxheW9mZnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Economics, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- AI startup Cohere raised $500 million in funding but announced layoffs of about 20 employees, roughly 5% of its workforce.
- CEO Aidan Gomez emphasized the layoffs were necessary for competitiveness despite the company's optimistic funding and growth outlook.
- Cohere plans to continue hiring and aims to double its headcount by 2024, focusing on augmenting workforce capabilities with AI.


[Why Coheres $5.5 Billion Valuation Isnt What It Seems; Perplexitys Data Sources Draw Questions - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy50aGVpbmZvcm1hdGlvbi5jb20vYXJ0aWNsZXMvd2h5LWNvaGVyZXMtNS01LWJpbGxpb24tdmFsdWF0aW9uLWlzbnQtd2hhdC1pdC1zZWVtcy1wZXJwbGV4aXR5cy1kYXRhLXNvdXJjZXMtZHJhdy1xdWVzdGlvbnPSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Bubble, Cohere, Data, Finance, Funding, Llms, Opinion, Venture Capital 

- Cohere raised $500 million led by PSP Investments, achieving a $5.5 billion valuation despite generating only $35 million in annualized revenue.
- The valuation reflects over 140 times Cohere's forward revenue, significantly higher than comparable AI companies like OpenAI and Anthropic, which are valued at 54 and 75 times their forward revenue.
- The discrepancy raises questions about whether the Canadian investors' valuation approach is justified or overly optimistic.


[Better Artificial Intelligence (AI) Stock: SoundHound AI vs. C3.ai - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Opinion, Review, Stock Market, Stocks 

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least $65 million in full-year revenue for 2024.
- C3.ai offers AI software for various applications and saw a 16% revenue increase to $310.6 million in fiscal 2024, but its net losses have been rising over the years, contrasting SoundHound's improving losses.
- Wall Street analysts favor SoundHound AI with a "buy" rating and a median price of $8, while C3.ai holds a "hold" rating with a median price of $29.50, making SoundHound appear the better investment choice currently.


[1 Top Artificial Intelligence (AI) Stock to Buy Before It Soars 84%, According to Loop Capital - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Nvidia, Opinion, Stock Market, Stocks 

- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases.
- The company has shifted from low growth to a revenue CAGR of 42% from 2021 to 2023, largely due to its partnership with Nvidia and dominance in the AI server market, which is expected to expand significantly.
- While bullish analysts project Supermicro could reach a stock price of $1,500, concerns exist over increased competition from major players like Dell and HPE and potential market slowdowns, making the average price target of over $1,000 more realistic.


# Samsung's AI image generation 

[After 6 Months, These Are My Favorite Samsung Galaxy AI Features - Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNuZXQuY29tL3RlY2gvbW9iaWxlL215LWZhdm9yaXRlLXNhbXN1bmctZ2FsYXh5LWFpLWZlYXR1cmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Opinion, Products, Review, Samsung 

- Samsung's Galaxy AI features, including advanced photo editing and language translation tools, were introduced with the Galaxy S24 series and expanded with the launch of the Galaxy Z Fold 6 and Galaxy Z Flip 6.
- Noteworthy tools include Circle to Search for instant searching by circling content on the screen, and Portrait Studio which stylizes photos of people into artistic portraits, showcasing a blend of creativity and practicality.
- Language translation advancements, such as Live Translate for real-time phone conversations and instant chat translation, highlight Samsung's focus on enhancing communication through AI while raising questions about authenticity in photo editing.


[Adobe's Photoshop can now generate AI images via prompts like Dall-E or Midjourney - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9hZG9iZXMtcGhvdG9zaG9wLWNhbi1ub3ctZ2VuZXJhdGUtYWktaW1hZ2VzLXZpYS1wcm9tcHRzLWxpa2UtZGFsbC1lLW9yLW1pZC1qb3VybmV5LTEzMDAxODE4MS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Adobe, Art, Design, Big Tech, Gen AI, Machine Learning, Photoshop, Products 

- Adobe has launched a new text-to-image generation feature in Photoshop, enabling users to create images from scratch using prompts similar to Dall-E and Midjourney, powered by the Firefly Image 3 model.
- The updated feature allows for significant creative flexibility, letting users generate images and make adjustments directly within Photoshop, which streamlines the workflow for artists.
- In response to concerns from artists regarding the use of their work for AI training, Adobe has reiterated its commitment to a creator-friendly approach, assuring that it will not utilize customer content for this purpose.


[Meta AI Introduces AI-Generated Photos to All Platforms - Google News](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnNlYXJjaGVuZ2luZWpvdXJuYWwuY29tL21ldGEtYWktaW50cm9kdWNlcy1haS1nZW5lcmF0ZWQtcGhvdG9zLXRvLWFsbC1wbGF0Zm9ybXMvNTIyODM1L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Big Tech, Gen AI, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI introduces advanced image generation and editing features on Facebook, Instagram, Messenger, and WhatsApp through the new tool "Imagine Me," allowing real-time transformations based on user prompts.
- Users can now edit images extensively, with a forthcoming "Edit With AI" button enhancing capabilities to modify objects in photos.
- Meta AI expands its accessibility to seven new countries and languages, increasing its availability to 22 countries and supporting seven additional languages.


[Luma AIs Dream Machine  Revolutionizing AI Video Creation - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LnVuaXRlLmFpL2x1bWEtYWlzLWRyZWFtLW1hY2hpbmUtcmV2b2x1dGlvbml6aW5nLWFpLXZpZGVvLWNyZWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Dream Machine, Entertainment, Gen AI, Luma AI, Machine Learning, Products, Science 

- Luma AI's Dream Machine is a user-friendly AI video generator that democratizes video creation, offering accessibility to beginners and professionals alike, with a free tier for up to 30 videos monthly.
- It features realistic animation capabilities and quick rendering times, making it faster than competitors like Open AI's Sora while producing high-quality videos.
- Despite its advantages, Dream Machine faces challenges such as the need for significant computational power, potential misuse for creating deepfakes, and the risk of perpetuating biases from training data.


[Meta AI gets new Imagine me selfie feature - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tZXRhLWFpLWdldHMtbmV3LXNlbGZpZS1mZWF0dXJlcy1xdWVzdC1zdXBwb3J0L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Big Tech, Facial Recognition, Feature, Gen AI, Meta, Meta AI, Products 

- Meta AI has introduced a new selfie feature called "Imagine Yourself," allowing users to create stylized images based on prompts and personal photos, utilizing a generative AI model.
- The AI now includes the upgraded Llama 3.1 405B model for handling complex queries, improving performance in math and coding, though users must manually switch to this model and face query limits.
- Meta AI is expanding its language support and will replace the Meta Quest's Voice Commands feature with AI capabilities in experimental mode, enabling users to interact with their physical environment.


[Adobe Rolls Out More AI Generative Tools To Illustrator And Photoshop - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvbWFya3NwYXJyb3cvMjAyNC8wNy8yMy9hZG9iZS1yb2xscy1vdXQtbW9yZS1haS1nZW5lcmF0aXZlLXRvb2xzLXRvLWlsbHVzdHJhdG9yLWFuZC1waG90b3Nob3Av0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Adobe, Art, Design, Big Tech, Gen AI, Illustrator, Machine Learning, Photoshop, Products 

- Adobe has introduced AI generative tools in Illustrator and Photoshop to enhance creative workflows and reduce tedious design tasks, enabling designers to focus on creativity.
- Key features include the new Generative Shape Fill tool in Illustrator and the Generate Image functionality in Photoshop, both powered by Adobe Firefly, which streamline design processes and allow for rapid iteration of ideas.
- Adobe emphasizes a commitment to responsible AI development, ensuring their generative models respect creators' rights and are trained on licensed content to support the creative community.


[I just tried Luma Labs' new loop feature for AI video  and it's scary good - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy50b21zZ3VpZGUuY29tL2FpL2FpLWltYWdlLXZpZGVvL2ktanVzdC10cmllZC1sdW1hLWxhYnMtbmV3LWxvb3AtZmVhdHVyZS1mb3ItZHJlYW0tbWFjaGluZS1haS12aWRlby10aGlzLWlzLWEtbGVhcC1mb3J3YXJk0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Entertainment, Feature, Gen AI, Machine Learning, Products, Review 

- Luma Labs' new 'loop' feature for AI video allows users to create seamless five-second animations by connecting the first and last frames, enhancing video creativity, especially for animations and memes.
- The feature can be hit or miss depending on the specificity of prompts, with better results often arising from image prompts rather than text.
- Examples of prompts used include colorful sound waves, bubbles, and a cat riding a unicycle, showcasing the tool's versatility and potential for continuous animation.


[Apple Intelligence to Generate Playlist Artwork in iOS 18 - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yNC9hcHBsZS1pbnRlbGxpZ2VuY2UtdG8tZ2VuZXJhdGUtcGxheWxpc3QtYXJ0d29yay_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Recommendation Systems, Streaming 

- iOS 18 will introduce a feature in Apple Music that allows users to generate customized playlist artwork using generative AI, as revealed in beta code.
- The functionality includes a "Create Image" button that uses the Image Playground tool to create unique images based on user input, utilizing different styles like animation and illustration.
- While expected to launch with iOS 18, the full rollout of Apple Intelligence features may be delayed until iOS 18.1 or beyond, with the official release anticipated in the fall.


[Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Streaming 

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button.
- The Image Playground, teased at WWDC 2024, enables users to generate AI images in various styles, though photorealistic options are not available.
- Currently under development, the AI features may be delayed until iOS 18.1 or later, with a full iOS 18 release expected this fall.


[Unlock Your Creativity with Luma AIs New Loops Feature: Seamless Video Creation Made Easy - Google News](https://news.google.com/articles/CBMigQFodHRwczovL3d3dy5nbG9iYWx2aWxsYWdlc3BhY2UuY29tL3RlY2gvdW5sb2NrLXlvdXItY3JlYXRpdml0eS13aXRoLWx1bWEtYWlzLW5ldy1sb29wcy1mZWF0dXJlLXNlYW1sZXNzLXZpZGVvLWNyZWF0aW9uLW1hZGUtZWFzeS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Entertainment, Gen AI, Loops Feature, Luma AI, Products 

- Luma AI has launched a new "Loops" feature on its Dream Machine platform, allowing users to create seamless video loops from text descriptions, images, or keyframes, eliminating choppy transitions.
- This innovation enables content creators and digital marketers to produce captivating videos without visible cuts, potentially reducing production time and costs while enhancing audience engagement.
- Luma AI emphasizes responsible AI development by implementing watermarking and attribution systems, addressing concerns about authenticity and misuse while democratizing access to video creation tools.


# EU antitrust probe on AI deals 

[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Competition, Ethics, Gen AI, Policy And Regulation 

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development.  
- The principles highlight the risks posed by partnerships and financial arrangements between major AI firms that could undermine competition.  
- The regulators promote fair dealings and interoperability among AI products and services, asserting that claims against interoperability will be closely examined.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[Get paid or sue? How the news business is combating the threat of AI - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmxhdGltZXMuY29tL2VudGVydGFpbm1lbnQtYXJ0cy9idXNpbmVzcy9zdG9yeS8yMDI0LTA3LTI0L2hvdy1haS1pcy1kaXNydXB0aW5nLXRoZS1qb3VybmFsaXNtLWluZHVzdHJ50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Intellectual Property, Llms 

- News organizations are confronting declining revenue due to digital ads and subscriptions, now compounded by AI technology that potentially infringes on their content by summarizing articles without compensation.
- Major publications like The New York Times and the Associated Press are moving towards licensing agreements with AI companies or pursuing legal action to protect their content and maintain viable business models.
- Smaller outlets, such as L.A. Taco, face greater challenges in responding to AI disruptions due to limited resources, emphasizing the need for regulatory measures to ensure fair compensation from tech companies.


[Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Cybersecurity, Ethics, Gen AI, Laws, Llms, OpenAI, Policy And Regulation, Politics 

- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators.
- Whistleblowers claimed OpenAI imposed restrictive agreements that hindered employees from raising concerns and waiving their rights to whistleblower compensation.
- The senators requested assurances from OpenAI's CEO that the company would eliminate non-disparagement agreements and improve protections for employees who report issues.


[The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Arms Race, Gen AI, Military, Opinion, Policy And Regulation, Science 

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority. 
- U.S. attempts to restrict China's access to advanced chips are proving ineffective and costly, with significant funds and resources mobilized without success in curbing Chinese access to technology. 
- A shift towards cooperation, rather than exclusive competition, is proposed to enhance AI advancements and mitigate self-harm caused by current restrictive policies.


[Mark Zuckerberg Slams Tim Cook's Apple After Cupertino Reportedly Rejected Its iPhone AI Partnership Proposal: 'They Wanted To Put Us In A Box...' - Google News](https://news.google.com/articles/CBMikAFodHRwczovL3d3dy5iZW56aW5nYS5jb20vbmV3cy8yNC8wNy8zOTkyNjA3Ni9tYXJrLXp1Y2tlcmJlcmctc2xhbXMtdGltLWNvb2tzLWFwcGxlLWFmdGVyLWN1cGVydGluby1yZXBvcnRlZGx5LXJlamVjdGVkLWl0cy1pcGhvbmUtYWktcGFydG5lcnNoaXDSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Gen AI, Mark Zuckerberg, Meta, Opinion 

- Mark Zuckerberg criticized Apple's "soul-crushing" tactics after the company rejected Meta's proposal to integrate its AI chatbot into the iPhone, citing concerns over privacy practices.
- Zuckerberg expressed his desire for Meta to control the next generation of technology by releasing a large open-source AI model, aiming to shape the technological landscape independently of Apple.
- Apple's rejection of the partnership comes amidst its announcements of collaborations with OpenAI and potential integrations with other AI startups, highlighting the competitive tension between the two companies.


[Opinion: This Nvidia Forecast All but Confirms That the Artificial Intelligence (AI) Bubble Will Burst Sooner Rather Than Later - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Gen AI, Nvidia, Opinion 

- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting.  
- The increasing competition from Intel and AMD, alongside tech giants developing in-house AI chips, indicates a reduction in reliance on Nvidia's products and a potential decrease in its market dominance.  
- Historical patterns of overestimation in new technologies imply that the current AI boom may face a correction, as many businesses lack clear plans for leveraging AI investments effectively in the near term.


[Google keeps the cost of AI search flat, and kids are lovin' itAs the G-Cloud brings in big bucks and plentiful profitAI + ML12 hrs|3 - The Register](https://www.theregister.com/2024/07/24/alphabet_q2_2024/)

Topics: Big Tech, Gen AI, Google, Machine Learning 

- Google has successfully maintained flat costs for its AI search tool, "AI overviews," while enhancing user engagement among the younger demographic (ages 18 to 24).
- Alphabet's cloud services have significantly contributed to revenue growth, with Q2 2024 showing a 29% year-on-year revenue increase, totaling $10.35 billion.
- CEO Sundar Pichai emphasized ongoing improvements in AI model capabilities and their application across consumer and enterprise sectors, indicating future growth potential in AI solutions.


[Microsoft Aims to Boost AI Workload Capacity With Lumen Technologies Partnership - Google News](https://news.google.com/articles/CBMijAFodHRwczovL21vbmV5LnVzbmV3cy5jb20vaW52ZXN0aW5nL25ld3MvYXJ0aWNsZXMvMjAyNC0wNy0yNC9taWNyb3NvZnQtYWltcy10by1ib29zdC1haS13b3JrbG9hZC1jYXBhY2l0eS13aXRoLWx1bWVuLXRlY2hub2xvZ2llcy1wYXJ0bmVyc2hpcNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Gen AI, Lumen Technologies, Machine Learning, Microsoft, Partnership 

- Microsoft is partnering with Lumen Technologies to expand its AI workload capacity using Lumen's network equipment, addressing issues with data center infrastructure shortages.
- The collaboration includes Lumen leveraging Microsoft Azure services to reduce costs, projected to enhance Lumen's cash flow by over $20 million in the next year.
- This partnership comes as Microsoft invests heavily in cloud infrastructure to maintain its competitive edge in AI, particularly following its investment in OpenAI.


# Salesforce AI service agent 

[Enhancing Automotive Customer Experience With AI: Gnani.ai - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Customer Service, Gen AI, Transportation 

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages.
- Post-purchase, AI automates maintenance reminders and helps in upselling by analyzing customer profiles, thereby improving satisfaction and boosting sales for automotive companies.
- Gopalan emphasizes the importance of AI complementing human roles rather than replacing them, with a focus on creating a seamless synergy that enhances customer service and personalizes the overall ownership experience.


[Can AI teach you to smile? Japan's firm uses it for customer satisfaction - Google News](https://news.google.com/articles/CBMiiQFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vd29ybGQtbmV3cy9jYW4tYWktdGVhY2gteW91LXRvLXNtaWxlLWphcGFuLXMtZmlybS11c2VzLWl0LWZvci1jdXN0b21lci1zYXRpc2ZhY3Rpb24tMTI0MDcyNDAwNjE5XzEuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Satisfaction, Customer Service, Gen AI, Japan, Machine Learning, Science, Society, Culture 

- AEON supermarket chain in Japan has implemented 'Mr Smile', an AI system that evaluates employees' service attitudes based on facial expressions, voice volume, and tone to enhance customer service.
- The system demonstrated a 1.6-fold improvement in service attitudes during a trial involving 3,400 employees, although it has raised concerns about potential harassment and the commodification of genuine smiles.
- Similar to McDonald's 'Smile zero yen' initiative, the standardized smile approach has faced criticism for adding stress to employees, prompting guidelines from the Japanese Ministry of Health, Labour and Welfare to prioritize staff well-being.


[Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M - TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)

Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital 

- Level AI, a startup focused on automating customer service tasks using AI, raised nearly $40 million in funding to expand its platform and capabilities.
- The platform provides tools for monitoring and improving contact center performance, enabling insights into customer sentiment and personalized agent guidance during calls.
- Despite growth potential, the company faces challenges including data privacy concerns, integration issues, and the psychological impact of monitoring on employees in the call center industry.


[Lenovo takes customer satisfaction to new heights with e-services enhanced by generative AI - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vbmV3cy5sZW5vdm8uY29tL2N1c3RvbWVyLXNhdGlzZmFjdGlvbi1lLXNlcnZpY2VzLWdlbmVyYXRpdmUtYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Customer Satisfaction, Customer Service, Gen AI, Generative AI, Lenovo, Products 

- Lenovo has integrated generative AI, specifically the GPT-4 model, to enhance customer support interactions on its forums, improving the speed, accuracy, and quality of responses.  
- The AI-powered forum assistant has achieved an 82% accuracy rate for generating responses and anticipates a 50% increase in efficiency over the next year.  
- Lenovo is set to expand its GenAI technology to enhance existing chatbots (Lena and Moli), allowing for more human-like interactions and the capability to handle a broader range of customer inquiries.


[SAP Q2 results reveal large orgs now firmly on the path to AI - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmNpby5jb20vYXJ0aWNsZS8zNDc2OTAwL3NhcC1xMi1yZXN1bHRzLXJldmVhbC1sYXJnZS1vcmdzLW5vdy1maXJtbHktb24tdGhlLXBhdGgtdG8tYWkuaHRtbNIBZmh0dHBzOi8vd3d3LmNpby5jb20vYXJ0aWNsZS8zNDc2OTAwL3NhcC1xMi1yZXN1bHRzLXJldmVhbC1sYXJnZS1vcmdzLW5vdy1maXJtbHktb24tdGhlLXBhdGgtdG8tYWkuaHRtbA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Adoption, Gen AI, Stocks 

- SAP's Q2 results indicate a successful shift towards AI, with 20% of deals incorporating Premium AI use cases, signifying strong customer demand for AI-driven transformations.
- Despite restructuring costs leading to a decrease in net revenue, SAP reported a cloud revenue increase of 25% and aims to enhance focus on strategic growth areas, particularly business AI.
- Analysts suggest that SAP is capitalizing on a wave of cloud modernization in ERP solutions, with anticipated strong growth driven by continued investments in innovation and AI capabilities.


[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture 

- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.
- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.
- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.


[PODCAST: Why dealers need to think bigger about AI - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LmF1dG9yZW1hcmtldGluZy5jb20vYXJjYW5hZGEvcG9kY2FzdC13aHktZGVhbGVycy1uZWVkLXRvLXRoaW5rLWJpZ2dlci1hYm91dC1haS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI 

- Impel executive sales manager Ray Gill emphasizes that dealers should adopt a broader perspective on the potential of artificial intelligence in their operations.
- During a training session at Canada’s Used Car Week, titled “How AI is Separating the Best from the Rest,” Gill shared insights on leveraging AI as a key tool for dealers.
- The podcast episode provides detailed explanations and suggestions on the effective application of AI in the automotive industry.


[The genAI boyfriend business is booming - Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Entertainment, Gen AI, Growth, Jobs, Careers, Labor Market, Products, Society, Culture 

- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners.
- The popularity of AI companion apps is on the rise, with high user engagement rates and a substantial increase in the number of new apps entering the market.
- Critics warn that reliance on chatbot companionship could weaken human relationships and pose privacy risks regarding personal data usage.


[Microsofts AI Assistants Will Revolutionize the Office  One Day - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/microsoft-s-ai-assistants-will-revolutionize-the-office-one-day)

Topics: Big Tech, Copilot, Gen AI, Microsoft, Opinion, Products 

- Microsoft’s AI assistants, known as Copilots, aim to transform office productivity by automating tasks and generating content, but require significant data cleanup and employee training for effective deployment.
- Early adopters report that while Copilots can efficiently distill information, they struggle with multi-app commands and context understanding, leading to slower widespread adoption anticipated until at least 2025.
- Successful implementation of Copilot involves training and careful data management; many companies face challenges in adapting their existing systems to optimize the AI’s performance.


[Meta Expands AI Chatbot to More Regions, Adds New Functionality - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnNvY2lhbG1lZGlhdG9kYXkuY29tL25ld3MvbWV0YS1leHBhbmRzLWFpLWNoYXRib3QtdG8tbW9yZS1yZWdpb25zLWFkZHMtbmV3LWZ1bmN0aW9uYWxpdHkvNzIyMTk0L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Meta, Products 

- Meta is expanding its AI chatbot to seven new regions and languages, aiming to become the most used AI assistant globally, despite mixed reviews and regulatory challenges in Europe.
- New functionalities include AI-generated selfie images and enhanced editing tools, allowing users to customize generative images within the chat stream, while also introducing the ability to choose different AI models for various tasks.
- Meta is launching AI in VR, replacing existing voice commands with AI capabilities, which could revolutionize VR experiences by allowing users to control their headset and create immersive environments through simple text prompts.


# Meta's AI regulatory issues 

[Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Competition, Ethics, Gen AI, Policy And Regulation 

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development.  
- The principles highlight the risks posed by partnerships and financial arrangements between major AI firms that could undermine competition.  
- The regulators promote fair dealings and interoperability among AI products and services, asserting that claims against interoperability will be closely examined.


[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[Meta Expands AI Chatbot to More Regions, Adds New Functionality - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnNvY2lhbG1lZGlhdG9kYXkuY29tL25ld3MvbWV0YS1leHBhbmRzLWFpLWNoYXRib3QtdG8tbW9yZS1yZWdpb25zLWFkZHMtbmV3LWZ1bmN0aW9uYWxpdHkvNzIyMTk0L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Language Models, Llms, Meta, Products 

- Meta is expanding its AI chatbot to seven new regions and languages, aiming to become the most used AI assistant globally, despite mixed reviews and regulatory challenges in Europe.
- New functionalities include AI-generated selfie images and enhanced editing tools, allowing users to customize generative images within the chat stream, while also introducing the ability to choose different AI models for various tasks.
- Meta is launching AI in VR, replacing existing voice commands with AI capabilities, which could revolutionize VR experiences by allowing users to control their headset and create immersive environments through simple text prompts.


[MNPS board votes on AI policy - Google News](https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LndzbXYuY29tL3ZpZGVvLzIwMjQvMDcvMjIvbW5wcy1ib2FyZC12b3Rlcy1haS1wb2xpY3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Policy, Education, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics 

- The MNPS board has voted on a policy regarding the use of artificial intelligence in schools.
- Public universities and school systems in Tennessee are required to submit their AI policies this year.
- Ongoing local news reports include incidents of violence and crime in the area, unrelated to the AI policy discussion.


[OpenAI removes AI safety executive Aleksander Madry from role - Reddit](https://www.reddit.com/r/technology/comments/1eaj0gb/openai_removes_ai_safety_executive_aleksander/)

Topics: Ethics, Gen AI, OpenAI 

- OpenAI has removed AI safety executive Aleksander Madry from his role, with some suggesting it may affect the company’s commitment to safety protocols in AI development.
- The reassignment sparked speculation and criticism from the community, emphasizing concerns about prioritizing speed over safety in AI advancements.
- There is a mix of sentiments among users, with some expressing hope that this change will lead to more effective AI applications, while others foresee negative consequences for the company.


[Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Cybersecurity, Ethics, Gen AI, Laws, Llms, OpenAI, Policy And Regulation, Politics 

- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators.
- Whistleblowers claimed OpenAI imposed restrictive agreements that hindered employees from raising concerns and waiving their rights to whistleblower compensation.
- The senators requested assurances from OpenAI's CEO that the company would eliminate non-disparagement agreements and improve protections for employees who report issues.


[Mark Zuckerberg Stumps for Open Source A.I. - The New York Times](https://www.nytimes.com/2024/07/23/technology/mark-zuckerberg-meta-open-source-ai.html)

Topics: Big Tech, Gen AI, Mark Zuckerberg, Meta, Open Source, Science 

- Mark Zuckerberg advocates for open-source AI to prevent control by a few large companies and enhance innovation and safety in technology development.
- He argues that sharing AI research is essential to combat espionage risks and believes it could lead to greater global prosperity.
- Meta has launched its latest AI algorithm, LLaMA, now supporting multiple languages, aligning with the push for more accessible AI technology.


[A Kamala Harris Presidency Could Mean More of the Same on A.I. Regulation - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMjQvdGVjaG5vbG9neS9rYW1hbGEtaGFycmlzLWFpLXJlZ3VsYXRpb24uaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Ethics, Gen AI, Kamala Harris, Laws, Opinion, Policy And Regulation, Politics 

- Kamala Harris has played a significant role in leading U.S. initiatives on artificial intelligence (A.I.), achieving voluntary safety standards with major tech companies, but struggles to pass concrete regulations in Congress.
- While she has connections to the tech industry and supports both innovation and public safety, experts believe her presidency could maintain a relatively lenient regulatory environment for A.I. companies.
- Harris emphasizes the need for corporate accountability and has asserted the moral duty of regulating A.I. to protect the public against potential harm, despite the current lack of firm legislative action.


[Mark Zuckerberg imagines content creators making AI clones of themselves - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tYXJrLXp1Y2tlcmJlcmctaW1hZ2luZXMtY29udGVudC1jcmVhdG9ycy1tYWtpbmctYWktY2xvbmVzLW9mLXRoZW1zZWx2ZXMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deepfakes, Entertainment, Gen AI, Llms, Machine Learning, Mark Zuckerberg, Meta, Science, Society, Culture 

- Mark Zuckerberg envisions a future where content creators use AI to replicate their personalities and automate community engagement, allowing them to focus on more critical tasks.
- He believes that AI could enhance creators' reach by reflecting their values and objectives, turning their interactions into unique, interactive experiences for audiences.
- Despite the potential benefits, trust issues remain for creators due to the current flaws in generative AI, and Meta needs to address these concerns to encourage adoption.


[Meta AI Is Now Multilingual, More Creative and Smarter - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Creativity, Gen AI, Language Models, Llms, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook.
- New creative tools, including "Imagine me" prompts for personalized image generation and editing capabilities, allow users to create and share custom images easily.
- The advanced Llama 405B model within Meta AI offers improved support for complex math and coding questions, enabling users to receive step-by-step guidance and build projects with coding assistance.


# Nvidia and Mistral AI model 

[The Prompt: Elon Musks Gigafactory Of Compute Is Running In Memphis - Google News](https://news.google.com/articles/CBMifGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzIzL3RoZS1wcm9tcHQtZWxvbi1tdXNrcy1naWdhZmFjdG9yeS1vZi1jb21wdXRlLWlzLXJ1bm5pbmctaW4tbWVtcGhpcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Elon Musk, Manufacturing, Tesla 

- Elon Musk's "gigafactory of compute" in Memphis, named Project Colossus, houses 100,000 Nvidia H100 chips to train his AI model Grok, but local officials express concerns over its impact on the city's power grid and water supply.
- Meta launched Llama 3.1, a powerful open-source AI model with 405 billion parameters designed to outperform existing AI models, while Mistral AI introduced task-specific models, Mathstral and Codestral.
- Target's AI chatbot for employees, intended to enhance job training, has been criticized for being unhelpful and inaccurate, with one instance where it offered dangerous advice during a crisis scenario.


[Nvidias coveted H100 GPUs will be available on-demand through Lambda - VentureBeat](https://venturebeat.com/ai/nvidias-coveted-h100-gpus-will-be-available-on-demand-through-lambda/)

Topics: Big Tech, Gen AI, Machine Learning, Nvidia, Products, Semiconductor Chips 

- Lambda has launched 1-Click Clusters, allowing customers to access Nvidia H100 and Quantum 2 InfiniBand GPUs on-demand without long contracts, catering to companies needing temporary GPU power for training AI models.
- The service enables companies to reserve GPU nodes for a minimum of two weeks, addressing the financial burden of obtaining and maintaining their own expensive GPU clusters.
- Unlike major cloud providers that focus on inference, Lambda emphasizes providing resources for model training without the hassle of lengthy contracts, making it more accessible for smaller AI companies.


[xAIs Memphis Supercluster has gone live, with up to 100,000 Nvidia H100 GPUs - Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy94YWlzLW1lbXBoaXMtc3VwZXJjbHVzdGVyLWhhcy1nb25lLWxpdmUtd2l0aC11cC10by0xMDAwMDAtbnZpZGlhLWgxMDAtZ3B1cy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Data, Gen AI, Machine Learning, Nvidia, Science, Semiconductor Chips 

- Elon Musk's xAI Memphis Supercluster, touted as the world's most powerful AI training cluster, went live with up to 100,000 Nvidia H100 GPUs, starting operations around 4:20 am on July 22.  
- Currently, only about 32,000 GPUs are reported to be online, with the full capacity expected to activate by Q4, while the cluster's training may not have commenced due to ongoing debugging and optimization.  
- The project is part of a significant multi-billion dollar investment in Memphis, with plans to expand further by adding another 300,000 GPU cluster by summer 2025.


[Opinion: This Nvidia Forecast All but Confirms That the Artificial Intelligence (AI) Bubble Will Burst Sooner Rather Than Later - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Gen AI, Nvidia, Opinion 

- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting.  
- The increasing competition from Intel and AMD, alongside tech giants developing in-house AI chips, indicates a reduction in reliance on Nvidia's products and a potential decrease in its market dominance.  
- Historical patterns of overestimation in new technologies imply that the current AI boom may face a correction, as many businesses lack clear plans for leveraging AI investments effectively in the near term.


[Meet the Artificial Intelligence (AI) Stock That Just Soared 225% Thanks to Nvidia - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9tZWV0LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLTA5NTMwMDIyMS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia's investment in Serve Robotics has driven a 225% surge in the company's stock due to its focus on autonomous delivery systems using AI technology.
- Serve Robotics reports significant growth potential, with a notable increase in revenue, although it still faces heavy cash burn and dependency on future investments to scale operations.
- Despite Nvidia’s endorsement and the rapid growth, Serve’s inflated valuation—reflecting a very high price-to-sales ratio—suggests high risk for potential investors.


[Google AI predicts long-term climate trends and weather  in minutes - Google News](https://news.google.com/articles/CBMiMmh0dHBzOi8vd3d3Lm5hdHVyZS5jb20vYXJ0aWNsZXMvZDQxNTg2LTAyNC0wMjM5MS050gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Climate, Google, Machine Learning, Science, Sustainability 

- A new hybrid model called NeuralGCM combines traditional weather forecasting with machine learning, outperforming existing AI tools in predicting weather scenarios and long-term climate trends with less energy consumption.
- NeuralGCM provides accurate short- and long-term forecasts while significantly reducing errors compared to pure machine-learning models, making it competitive with established general circulation models (GCMs).
- The model is expected to enhance decision-making and preparedness strategies for extreme weather events, with intentions for further refinement and inclusion of more Earth science elements in future versions.


[Nvidia, Amazon, Microsoft, and Paramount execs discuss the use of AI in Hollywood. Here are 9 startups they're watching. - Business Insider](https://www.businessinsider.com/hollywood-execs-embrace-ai-startups-nvidia-microsoft-paramount-amazon-2024-7)

Topics: Amazon, Big Tech, Entertainment, Gen AI, Hollywood, Microsoft, Nvidia, Tv, Film, Movies 

- Hollywood executives are exploring AI to reduce costs, enhance audience engagement, and manage archival content, with a focus on practical applications in marketing and distribution.
- Notable startups like Runway and Speechmatics are gaining attention for their AI tools that improve film marketing, create visual content, and facilitate translation and transcription services.
- Collaborations with tech giants like Nvidia, AWS, and Microsoft highlight the industry's shift toward leveraging AI for innovative content generation and enhanced accessibility in media.


[Nvidia said to launch new AI chip for China - Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Gen AI, Machine Learning, Nvidia, Products, Science, Semiconductor Chips 

- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls.
- There is speculation that the H20 chip might be banned for sale to China due to anticipated changes in U.S. export policies, which could also affect other Asian countries.
- Reports have highlighted an underground network smuggling banned A100 chips into China, indicating high demand and significant market value despite strict export regulations.


# TSMC AI chip demand surge 

[Trump Trade May Reverse As Probability Of Biden Backing Out Increases  Watch AI Semiconductors - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5iZW56aW5nYS5jb20vMjQvMDcvMzk4NDM4NzIvdHJ1bXAtdHJhZGUtbWF5LXJldmVyc2UtYXMtcHJvYmFiaWxpdHktb2YtYmlkZW4tYmFja2luZy1vdXQtaW5jcmVhc2VzLXdhdGNoLWFpLXNlbWljb25kdWN0b3Jz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Finance, Policy And Regulation, Politics, Trade 

- Taiwan Semiconductor (TSM) plays a crucial role in the semiconductor industry, producing advanced AI chips, and recently reported strong earnings with over 50% of revenues from AI, indicating significant growth potential despite market volatility driven by political factors.
- The Trump trade may reverse if the probability of Biden backing out and Democrats nominating a new candidate increases, highlighting the importance of monitoring key AI semiconductor stocks for signs of this shift.
- Investors are advised to separate political considerations from investment strategies, with recommendations to maintain a flexible protection band strategy to manage risk while capitalizing on potential market opportunities.


[Nvidia said to launch new AI chip for China - Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Gen AI, Machine Learning, Nvidia, Products, Science, Semiconductor Chips 

- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls.
- There is speculation that the H20 chip might be banned for sale to China due to anticipated changes in U.S. export policies, which could also affect other Asian countries.
- Reports have highlighted an underground network smuggling banned A100 chips into China, indicating high demand and significant market value despite strict export regulations.


[By embracing liquid cooling, AI powerhouse Supermicro enables 30% more computing power  with the same power budget - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Products, Science 

- Supermicro's CEO predicts a major AI revolution, with the company's revenue growing by 200% year-over-year and projected to increase another 152% by year-end 2024.
- The company aims for liquid cooling solutions to comprise 20% to 30% of data center deployments over the next year, enabling 30% more computing power while consuming 30-40% less energy.
- Supermicro is ramping up production, currently shipping 4,000 racks per month, with plans to increase liquid-cooled rack shipments to about 2,000 monthly to meet rising AI demands.


[1 Top Artificial Intelligence (AI) Stock to Buy Before It Soars 84%, According to Loop Capital - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Nvidia, Opinion, Stock Market, Stocks 

- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases.
- The company has shifted from low growth to a revenue CAGR of 42% from 2021 to 2023, largely due to its partnership with Nvidia and dominance in the AI server market, which is expected to expand significantly.
- While bullish analysts project Supermicro could reach a stock price of $1,500, concerns exist over increased competition from major players like Dell and HPE and potential market slowdowns, making the average price target of over $1,000 more realistic.


[AMD introduces new Ryzen AI 9 HX 375 "Strix Point" APU: NPU now up to 55 TOPS - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdmlkZW9jYXJkei5jb20vbmV3ei9hbWQtaW50cm9kdWNlcy1uZXctcnl6ZW4tYWktOS1oeC0zNzUtc3RyaXgtcG9pbnQtYXB1LW5wdS1ub3ctdXAtdG8tNTUtdG9wc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Big Tech, Gen AI, Machine Learning, Products, Ryzen AI, Semiconductor Chips 

- AMD has introduced the Ryzen AI 9 HX 375 "Strix Point" APU, featuring an optimized NPU with up to 55 TOPS for enhanced AI performance.
- The new APU is part of the Strix Point series, sharing key specifications with the HX 370 SKU, including 4x Zen5 and 8x Zen5c cores and a maximum boost clock of 5.1 GHz.
- The Ryzen AI 9 HX 375 is set to debut in the OmniBook Ultra 14-inch laptop, expanding AMD's Ryzen AI 300 lineup to three models.


[Elon Musk doubles down on Tesla's Dojo supercomputer, aiming to compete with Nvidia - Business Insider](https://www.businessinsider.com/elon-musk-doubles-down-tesla-dojo-compete-with-nvidia-2024-7)

Topics: Big Tech, Competition, Data, Elon Musk, Gen AI, Machine Learning, Nvidia, Semiconductor Chips, Tesla 

- Elon Musk reaffirmed Tesla's commitment to developing its Dojo supercomputer to compete with Nvidia's AI chips amid challenges in accessing Nvidia GPUs.
- He expressed concerns over the high demand for Nvidia's hardware, which makes it difficult for Tesla to secure essential graphics processing units.
- Musk emphasized the necessity for Tesla to enhance Dojo's capabilities to reduce reliance on Nvidia, acknowledging the significant market lead Nvidia holds in AI chip production.


[AMD slyly lists new Ryzen AI 9 HX 375 Strix Point CPU - Google News](https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3Lm5vdGVib29rY2hlY2submV0L0FNRC1zbHlseS1saXN0cy1uZXctUnl6ZW4tQUktOS1IWC0zNzUtU3RyaXgtUG9pbnQtQ1BVLjg2NjQ3NS4wLmh0bWzSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Big Tech, Products, Ryzen AI, Semiconductor Chips 

- AMD has introduced the Ryzen AI 9 HX 375, a new Strix Point CPU that offers a slightly enhanced NPU compared to the Ryze AI 9 HX 370, rated at 55 TOPS versus 50 TOPS.
- The Ryzen AI 9 HX 375 retains the same core specifications as the HX 370, including 12 cores, 24 threads, and a single-core boost of 5.1 GHz, as well as the Radeon 890M iGPU.
- The purpose and intended device for the Ryzen AI 9 HX 375 remain unclear, and its performance is expected to be similar to the HX 370, particularly in tasks reliant on the NPU.


[Opinion: This Nvidia Forecast All but Confirms That the Artificial Intelligence (AI) Bubble Will Burst Sooner Rather Than Later - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Gen AI, Nvidia, Opinion 

- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting.  
- The increasing competition from Intel and AMD, alongside tech giants developing in-house AI chips, indicates a reduction in reliance on Nvidia's products and a potential decrease in its market dominance.  
- Historical patterns of overestimation in new technologies imply that the current AI boom may face a correction, as many businesses lack clear plans for leveraging AI investments effectively in the near term.


[Nvidias coveted H100 GPUs will be available on-demand through Lambda - VentureBeat](https://venturebeat.com/ai/nvidias-coveted-h100-gpus-will-be-available-on-demand-through-lambda/)

Topics: Big Tech, Gen AI, Machine Learning, Nvidia, Products, Semiconductor Chips 

- Lambda has launched 1-Click Clusters, allowing customers to access Nvidia H100 and Quantum 2 InfiniBand GPUs on-demand without long contracts, catering to companies needing temporary GPU power for training AI models.
- The service enables companies to reserve GPU nodes for a minimum of two weeks, addressing the financial burden of obtaining and maintaining their own expensive GPU clusters.
- Unlike major cloud providers that focus on inference, Lambda emphasizes providing resources for model training without the hassle of lengthy contracts, making it more accessible for smaller AI companies.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


# AI search engines 

[Google Is the Only Search Engine That Works on Reddit Now Thanks to AI Deal - 404 Media](https://www.404media.co/google-is-the-only-search-engine-that-works-on-reddit-now-thanks-to-ai-deal/)

Topics: Big Tech, Deals, Gen AI, Google, Machine Learning, Reddit 

- Google is now the only search engine able to access and display results from Reddit due to a multi-million dollar deal, effectively sidelining alternatives like DuckDuckGo, Bing, and Mojeek.
- Reddit has updated its robots.txt to block all crawlers except Google’s, as part of its effort to restrict content scraping by AI companies and protect user-generated contributions.
- The exclusive access granted to Google could worsen the competitiveness of other search engines and reflects a larger trend of larger platforms diminishing the visibility of smaller entities on the web.


[Google keeps the cost of AI search flat, and kids are lovin' itAs the G-Cloud brings in big bucks and plentiful profitAI + ML12 hrs|3 - The Register](https://www.theregister.com/2024/07/24/alphabet_q2_2024/)

Topics: Big Tech, Gen AI, Google, Machine Learning 

- Google has successfully maintained flat costs for its AI search tool, "AI overviews," while enhancing user engagement among the younger demographic (ages 18 to 24).
- Alphabet's cloud services have significantly contributed to revenue growth, with Q2 2024 showing a 29% year-on-year revenue increase, totaling $10.35 billion.
- CEO Sundar Pichai emphasized ongoing improvements in AI model capabilities and their application across consumer and enterprise sectors, indicating future growth potential in AI solutions.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[Better Artificial Intelligence (AI) Stock: SoundHound AI vs. C3.ai - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Opinion, Review, Stock Market, Stocks 

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least $65 million in full-year revenue for 2024.
- C3.ai offers AI software for various applications and saw a 16% revenue increase to $310.6 million in fiscal 2024, but its net losses have been rising over the years, contrasting SoundHound's improving losses.
- Wall Street analysts favor SoundHound AI with a "buy" rating and a median price of $8, while C3.ai holds a "hold" rating with a median price of $29.50, making SoundHound appear the better investment choice currently.


[AI firms will soon exhaust most of the internets data - Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmVjb25vbWlzdC5jb20vc2Nob29scy1icmllZi8yMDI0LzA3LzIzL2FpLWZpcm1zLXdpbGwtc29vbi1leGhhdXN0LW1vc3Qtb2YtdGhlLWludGVybmV0cy1kYXRh0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Data, Gen AI, Llms 

- AI firms are approaching a limit in their ability to utilize the vast data available on the internet, raising questions about data sustainability.
- Fei-Fei Li's concept in 2006 emphasized the importance of data over models, leading to the creation of ImageNet, which transformed AI research.
- The industry's future might depend on their ability to generate or source additional data as the current internet resources become exhausted.


[From Hype to Results: Accelerating AI in Automotive - Google News](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LmJhaW4uY29tL2luc2lnaHRzL2Zyb20taHlwZS10by1yZXN1bHRzLWFjY2VsZXJhdGluZy1haS1pbi1hdXRvbW90aXZlLXdlYmluYXIv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Gen AI, Transportation 

- Bain & Company emphasizes the importance of essential steps for automotive businesses to effectively implement AI innovations.
- The focus is on transforming hype surrounding AI into tangible results within the automotive industry.
- The company offers webinars featuring expert insights on leveraging AI for improved business outcomes.


[Enhancing Automotive Customer Experience With AI: Gnani.ai - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Customer Service, Gen AI, Transportation 

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages.
- Post-purchase, AI automates maintenance reminders and helps in upselling by analyzing customer profiles, thereby improving satisfaction and boosting sales for automotive companies.
- Gopalan emphasizes the importance of AI complementing human roles rather than replacing them, with a focus on creating a seamless synergy that enhances customer service and personalizes the overall ownership experience.


[Meta AI Is Now Multilingual, More Creative and Smarter - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Creativity, Gen AI, Language Models, Llms, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook.
- New creative tools, including "Imagine me" prompts for personalized image generation and editing capabilities, allow users to create and share custom images easily.
- The advanced Llama 405B model within Meta AI offers improved support for complex math and coding questions, enabling users to receive step-by-step guidance and build projects with coding assistance.


[How Pearson's AI assistant can help teachers save time - Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Education, Gen AI, Llms, Products 

- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation, set to be available in August on their MyLab and Mastering platforms.
- The AI tool enables instructors to quickly generate assignments by sorting through numerous questions based on difficulty and relevance, allowing more time for direct student engagement and deeper teaching tasks.
- A survey revealed that many instructors lack confidence in using generative AI, despite the interest in integrating it into instructional practices, highlighting a gap in AI familiarity between students and educators.


[These Are the 10 AI Startups to Watch in 2024 - Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Investment, Machine Learning, Startups, Venture Capital 

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry.  
- Startups like Cognition AI and Helsing are leveraging advanced technology for practical applications, such as coding solutions and modernizing military capabilities.  
- The report emphasizes the dynamic AI landscape, driven by unique company visions and leadership, including initiatives for safe superintelligence by Ilya Sutskever.  


# AI in cybersecurity 

[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[An IT Leaders Playbook for Creating an Effective AI Policy - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Policy And Regulation 

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management.
- Key benefits of AI adoption include improved sales analytics, enhanced customer service, automation of routine tasks, and overall increased operational efficiency.
- An effective AI policy framework should define important terms, identify risks, prohibit misuse, ensure compliance, and establish consequences for violations.


[The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Adoption, Challenges, Cognitive Science, Ethics, Gen AI, Opinion, Science, Society, Culture 

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology.
- The demand for skilled AI professionals exceeds supply, leading companies to either "poach" talent or invest in developing existing employees, while legacy systems complicate AI integration efforts.
- To effectively implement AI, organizations must focus on employee education, clear communication about the benefits of AI, and gather data to develop compliant, tailored AI models.


[Top AI Job Skills Training Programs - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- AI job skills training programs are essential for bridging the skill gap between the growing demand for AI professionals and the current workforce's capabilities.
- These programs offer flexibility and accessibility, allowing learners to acquire relevant AI skills at their own pace, which is particularly beneficial for working professionals.
- Completion of AI training significantly enhances employment prospects, often resulting in recognized certifications that add value to resumes and demonstrate commitment to professional development.


[Are We Witnessing the Weaponization of AI? | Opinion - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Ethics, Gen AI, Opinion, Policy And Regulation 

- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests.
- The relationship between Big Tech and the U.S. government is deepening, leading to the potential use of advanced AI systems for surveillance under the guise of national security, which may infringe on personal freedoms and privacy.
- The "revolving door" phenomenon between tech and government highlights an increasing cooperation, with OpenAI's policy changes allowing military applications, signaling a broader trend of weaponizing AI.


[The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Arms Race, Gen AI, Military, Opinion, Policy And Regulation, Science 

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority. 
- U.S. attempts to restrict China's access to advanced chips are proving ineffective and costly, with significant funds and resources mobilized without success in curbing Chinese access to technology. 
- A shift towards cooperation, rather than exclusive competition, is proposed to enhance AI advancements and mitigate self-harm caused by current restrictive policies.


[Billions Of People Need To Learn AI Literacy - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvam9obndlcm5lci8yMDI0LzA3LzE3L2JpbGxpb25zLW9mLXBlb3BsZS1uZWVkLXRvLWxlYXJuLWFpLWxpdGVyYWN5L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Machine Learning, Opinion, Society, Culture 

- AI literacy is essential as it creates a distinct gap in power, much like previous literacy movements, influencing who can leverage AI technology effectively.
- It involves critical understanding of AI technologies, evaluating their context, and making informed decisions rather than just memorizing technical details.
- Promoting AI literacy will foster inclusive discussions, better regulatory oversight, and enhance social benefits, making it crucial for individuals and societies to adapt in the evolving technological landscape.


[NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Gen AI, Healthcare, Machine Learning, Science, Study, Testing 

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers.
- In a comparison with human physicians, AI initially scored higher on recall in a closed-book setting, but human counterparts outperformed in open-book challenges, particularly on more complex cases.
- The findings indicate potential synergy between AI tools and human expertise in diagnostics, but emphasize the necessity for enhanced explanatory abilities and comprehensive evaluations before AI can replace human experience in healthcare.


[AI And The Changing Character Of War  OpEd - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vd3d3LmV1cmFzaWFyZXZpZXcuY29tLzIyMDcyMDI0LWFpLWFuZC10aGUtY2hhbmdpbmctY2hhcmFjdGVyLW9mLXdhci1vcGVkL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Military, Opinion 

- The integration of AI in warfare represents a significant Revolution in Military Affairs, fundamentally changing decision-making processes and the nature of combat, with concerns about the loss of human oversight in lethal situations.
- Historical uses of AI in military contexts raise alarm over the potential for uncontrollable destruction and violations of International Humanitarian Law, highlighted by incidents in Gaza and calls for regulation from over 150 countries.
- The development of autonomous weapons in regions like South Asia escalates security tensions, particularly between India and Pakistan, prompting advocacy for binding legal frameworks to prohibit such technologies.


[5 Steps That OpenAI Thinks Will Lead to Artificial Intelligence Running a Company - Google News](https://news.google.com/articles/CBMidWh0dHBzOi8vd3d3LmluYy5jb20vYmVuLXNoZXJyeS81LXN0ZXBzLXRoYXQtb3BlbmFpLXRoaW5rcy13aWxsLWxlYWQtdG8tYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UtcnVubmluZy1hLWNvbXBhbnkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Artificial General Intelligence, Big Tech, Gen AI, OpenAI, Science 

- OpenAI has defined five stages of AI innovation, ranging from simple chatbots to advanced systems capable of functioning like an entire organization, with the ultimate goal of achieving artificial general intelligence (AGI).
- Current AI technology, represented by chatbots, has limited commercial utility and often presents inaccurate information, while OpenAI is working on advancing AI to "Reasoners" that can solve problems at a human equivalent level.
- The final stage, "Organizations," suggests AI could eventually replace human roles entirely, raising concerns about the implications for job security and the future role of humans in the workforce.


# AI in Healthcare & Precision Medicine 

[NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Gen AI, Healthcare, Machine Learning, Science, Study, Testing 

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers.
- In a comparison with human physicians, AI initially scored higher on recall in a closed-book setting, but human counterparts outperformed in open-book challenges, particularly on more complex cases.
- The findings indicate potential synergy between AI tools and human expertise in diagnostics, but emphasize the necessity for enhanced explanatory abilities and comprehensive evaluations before AI can replace human experience in healthcare.


[AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Health, Fitness, Healthcare, Machine Learning, Science, Study, Testing 

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate.
- The AI program, Unfold AI, uses clinical data to create a 3D cancer estimation map and helps determine personalized treatment options for patients.
- The findings suggest that AI could lead to more accurate diagnoses and targeted treatments, potentially reducing the need for invasive procedures and their associated side effects.


[Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Authors, Writing, Gen AI, Healthcare, Language Models, Llms, Machine Learning, Products, Science 

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers.
- DORA utilizes multiple AI and large language models to enhance the writing process while ensuring accuracy through cycles of reinforcement learning and human proofreading.
- The company is also introducing hardware solutions like the PandaOmics Box for secure, offline data processing and the open-source Precious-3 GPT for research into aging and related diseases.


[The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Adoption, Challenges, Cognitive Science, Ethics, Gen AI, Opinion, Science, Society, Culture 

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology.
- The demand for skilled AI professionals exceeds supply, leading companies to either "poach" talent or invest in developing existing employees, while legacy systems complicate AI integration efforts.
- To effectively implement AI, organizations must focus on employee education, clear communication about the benefits of AI, and gather data to develop compliant, tailored AI models.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[From Hype to Results: Accelerating AI in Automotive - Google News](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LmJhaW4uY29tL2luc2lnaHRzL2Zyb20taHlwZS10by1yZXN1bHRzLWFjY2VsZXJhdGluZy1haS1pbi1hdXRvbW90aXZlLXdlYmluYXIv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Gen AI, Transportation 

- Bain & Company emphasizes the importance of essential steps for automotive businesses to effectively implement AI innovations.
- The focus is on transforming hype surrounding AI into tangible results within the automotive industry.
- The company offers webinars featuring expert insights on leveraging AI for improved business outcomes.


[Startup Pearl Raises VC Funding to Bring AI to Dentists Offices - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices)

Topics: Deals, Finance, Funding, Gen AI, Healthcare, Venture Capital 

- Pearl Inc., a Los Angeles-based dental startup, has raised $58 million in funding, valuing the company at $400 million.
- The company uses AI technology approved by the FDA to enhance dental diagnostics, having received regulatory clearance in 120 countries.
- Co-founder Ophir Tanz aims for an IPO, stating that Pearl's technology could transform the standards of dental practices worldwide.


[Enhancing Automotive Customer Experience With AI: Gnani.ai - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Customer Service, Gen AI, Transportation 

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages.
- Post-purchase, AI automates maintenance reminders and helps in upselling by analyzing customer profiles, thereby improving satisfaction and boosting sales for automotive companies.
- Gopalan emphasizes the importance of AI complementing human roles rather than replacing them, with a focus on creating a seamless synergy that enhances customer service and personalizes the overall ownership experience.


[Top AI Job Skills Training Programs - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- AI job skills training programs are essential for bridging the skill gap between the growing demand for AI professionals and the current workforce's capabilities.
- These programs offer flexibility and accessibility, allowing learners to acquire relevant AI skills at their own pace, which is particularly beneficial for working professionals.
- Completion of AI training significantly enhances employment prospects, often resulting in recognized certifications that add value to resumes and demonstrate commitment to professional development.


[An IT Leaders Playbook for Creating an Effective AI Policy - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Policy And Regulation 

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management.
- Key benefits of AI adoption include improved sales analytics, enhanced customer service, automation of routine tasks, and overall increased operational efficiency.
- An effective AI policy framework should define important terms, identify risks, prohibit misuse, ensure compliance, and establish consequences for violations.


# Autonomous Drones 

[Toyota Research Institute and Stanford Engineering Achieve Worlds First Fully Autonomous Tandem Drift Sequence - Google News](https://news.google.com/articles/CBMijAFodHRwczovL3ByZXNzcm9vbS50b3lvdGEuY29tL3RveW90YS1yZXNlYXJjaC1pbnN0aXR1dGUtYW5kLXN0YW5mb3JkLWVuZ2luZWVyaW5nLWFjaGlldmUtd29ybGRzLWZpcnN0LWZ1bGx5LWF1dG9ub21vdXMtdGFuZGVtLWRyaWZ0LXNlcXVlbmNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Science, Stanford, Testing, Toyota, Transportation 

- Toyota Research Institute and Stanford Engineering achieved the world's first fully autonomous tandem drift sequence, advancing driving research to improve safety.
- The autonomous drifting technology simulates dynamic driving conditions, enhancing vehicle control and potentially reducing accidents caused by loss of vehicle control.
- The project utilized advanced AI techniques and real-time communication between vehicles to enable precise maneuvers, which can aid in safe driving situations like skids or slides.


[Kodiak Robotics completes first driverless semi truck delivery ahead of commercial operations - Reddit](https://www.reddit.com/r/technews/comments/1eah093/kodiak_robotics_completes_first_driverless_semi/)

Topics: Autonomous Vehicles, Robots, Testing, Transportation 

- Kodiak Robotics has successfully completed its first delivery using a driverless semi truck in preparation for future commercial operations.
- The milestone indicates advancements in autonomous vehicle technology and its potential impact on the logistics industry.
- Discussions around the implications of AI in the workforce and economic structures have been sparked by the development.


[Toyota Pulls Off a Fast and Furious Demo With Dual Drifting AI-Powered Race Cars - Google News](https://news.google.com/articles/CBMiRGh0dHBzOi8vd3d3LndpcmVkLmNvbS9zdG9yeS90b3lvdGEtc3RhbmZvcmQtYWktdGFuZGVtLWRyaWZ0aW5nLWNhcnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Entertainment, Machine Learning, Products, Robots, Sports, Testing, Toyota, Transportation 

- Toyota and Stanford University developed AI-powered self-driving cars capable of drifting in tandem, demonstrating advances in autonomous driving technology at Thunderhill Raceway Park.
- The project aims to enhance future driver-assistance systems by allowing vehicles to autonomously handle situations when drivers lose traction, potentially improving safety in adverse driving conditions.
- The vehicles utilize a combination of advanced mathematical models and machine learning algorithms to optimize performance by continuously calculating vehicle dynamics up to 50 times per second.


[Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital 

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised $16.5 million in funding.
- The company is creating a translation platform allowing developers to write code in common languages like Python, which is then automatically transposed to specific hardware architectures, significantly speeding up development processes.
- Code Metal plans to grow its team from seven to around 30 employees, establishing a hybrid work model with a headquarters in Boston while also having staff in San Francisco and New York.


[Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Copilot, Science, Toyota, Transportation 

- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions.
- The technology could significantly enhance car control on snow or ice by reacting up to 50 times per second to manage a driver’s loss of control, emulating expert drifter techniques.
- The AI learning process is developed collaboratively, with TRI focusing on the lead car's control and Stanford working on algorithms for the chase car to avoid collisions while mimicking the lead vehicle.


[AI and drones to protect the worlds vulnerable forests - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5ldXJvbmV3cy5jb20vZ3JlZW4vMjAyNC8wNy8yMy93aWxkZmlyZXMtcGVzdHMtbG9nZ2luZy1ob3ctY2FuLWFpLWFuZC1kcm9uZXMtcHJvdGVjdC10aGUtd29ybGRzLWZvcmVzdHMtZnJvbS1jbGltYXRlLXRocmXSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Climate, Drones, Science, Sustainability 

- Solar balconies, a growing trend in Germany, allow individuals without roof access to harness solar energy and reduce electricity bills using small plug-and-play systems.
- The popularity of balcony solar is driven by simplified installation processes, government incentives, and increasing awareness among citizens of their role in the energy transition.
- Other European nations have been slower to adopt balcony solar solutions due to regulatory challenges, with some countries limiting plug-in devices, while others encourage their uptake.


[Toyota, Stanford teaches AI to drift two GR Supra in tandem, a 1st - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vaW50ZXJlc3RpbmdlbmdpbmVlcmluZy5jb20vaW5ub3ZhdGlvbi90b3lvdGEtc3RhbmZvcmQtYXV0b25vbW91cy10YW5kZW0tZHJpZnTSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Machine Learning, Stanford, Testing, Toyota, Transportation 

- Toyota Research Institute and Stanford Engineering achieved the world's first fully autonomous tandem drift with two GR Supra vehicles, enhancing safety in driving dynamics.
- The AI-driven research aims to improve vehicle responses in various conditions, mimicking the physics of drifting, which is relevant for scenarios like sliding on snow or ice.
- Advanced algorithms and real-time communication between the lead and chase cars allowed for precise control and adaptation, demonstrating the potential for building advanced safety systems in future automobiles.


# AI Startup Funding 

[These Are the 10 AI Startups to Watch in 2024 - Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Investment, Machine Learning, Startups, Venture Capital 

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry.  
- Startups like Cognition AI and Helsing are leveraging advanced technology for practical applications, such as coding solutions and modernizing military capabilities.  
- The report emphasizes the dynamic AI landscape, driven by unique company visions and leadership, including initiatives for safe superintelligence by Ilya Sutskever.  


[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


[AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[AI startup Code Metal raises $16.5m in seed funding - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- AI startup Code Metal has raised $16.5 million in total funding, with $3.45 million from a pre-seed round and $13 million in seed funding, to enhance Edge computing capabilities.
- The investment will be used to build modular AI workloads and expand the team, emphasizing the rapid transformation in software development and Edge deployment.
- Code Metal aims to optimize AI code for Edge devices, leveraging expertise from founders experienced in delivering AI solutions for major technology platforms.


[Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Funding, Gen AI, Growth, Investment, Machine Learning, Tesla, Venture Capital 

- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus.
- The company plans to ramp up production of the Optimus robot by 2025 for internal use and is exploring deploying its future autonomous driving and Robotaxi services, contingent on technology and regulatory advancements.
- Despite a 2% year-over-year revenue increase, Tesla reported a 40% drop in profits, with automotive revenues declining while energy generation revenue saw a substantial 100% growth.


[AI startup Cohere cuts staff after $500 million funding round - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Deals, Economics, Finance, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a $500 million funding round that valued the company at $5.5 billion.
- Despite the layoffs, which represent a small fraction of its 400-person workforce, Cohere plans to continue hiring in several key areas, including customer operations and product design.
- The company focuses on generative AI for enterprises, distinguishing itself from competitors by avoiding consumer chatbot markets, while maintaining strong client relationships in banking and financial services.


[Better Artificial Intelligence (AI) Stock: SoundHound AI vs. C3.ai - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Opinion, Review, Stock Market, Stocks 

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least $65 million in full-year revenue for 2024.
- C3.ai offers AI software for various applications and saw a 16% revenue increase to $310.6 million in fiscal 2024, but its net losses have been rising over the years, contrasting SoundHound's improving losses.
- Wall Street analysts favor SoundHound AI with a "buy" rating and a median price of $8, while C3.ai holds a "hold" rating with a median price of $29.50, making SoundHound appear the better investment choice currently.


[Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M - TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)

Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital 

- Level AI, a startup focused on automating customer service tasks using AI, raised nearly $40 million in funding to expand its platform and capabilities.
- The platform provides tools for monitoring and improving contact center performance, enabling insights into customer sentiment and personalized agent guidance during calls.
- Despite growth potential, the company faces challenges including data privacy concerns, integration issues, and the psychological impact of monitoring on employees in the call center industry.


[Startup Pearl Raises VC Funding to Bring AI to Dentists Offices - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices)

Topics: Deals, Finance, Funding, Gen AI, Healthcare, Venture Capital 

- Pearl Inc., a Los Angeles-based dental startup, has raised $58 million in funding, valuing the company at $400 million.
- The company uses AI technology approved by the FDA to enhance dental diagnostics, having received regulatory clearance in 120 countries.
- Co-founder Ophir Tanz aims for an IPO, stating that Pearl's technology could transform the standards of dental practices worldwide.


# Claude AI for Data Analysis 

[The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Data, Gen AI, Llms, Machine Learning, Opinion, Science 

- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions.
- Startups like Gretel are producing synthetic data to train AI models, while others, such as Scale AI and Toloka, focus on human-annotated data to improve existing datasets.
- There is a growing shift towards smaller, more efficient AI models that require less data, as some researchers argue against the dependency on vast amounts of training data.


[Luma AIs Dream Machine  Revolutionizing AI Video Creation - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LnVuaXRlLmFpL2x1bWEtYWlzLWRyZWFtLW1hY2hpbmUtcmV2b2x1dGlvbml6aW5nLWFpLXZpZGVvLWNyZWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Dream Machine, Entertainment, Gen AI, Luma AI, Machine Learning, Products, Science 

- Luma AI's Dream Machine is a user-friendly AI video generator that democratizes video creation, offering accessibility to beginners and professionals alike, with a free tier for up to 30 videos monthly.
- It features realistic animation capabilities and quick rendering times, making it faster than competitors like Open AI's Sora while producing high-quality videos.
- Despite its advantages, Dream Machine faces challenges such as the need for significant computational power, potential misuse for creating deepfakes, and the risk of perpetuating biases from training data.


[Meta AI Is Now Multilingual, More Creative and Smarter - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Creativity, Gen AI, Language Models, Llms, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook.
- New creative tools, including "Imagine me" prompts for personalized image generation and editing capabilities, allow users to create and share custom images easily.
- The advanced Llama 405B model within Meta AI offers improved support for complex math and coding questions, enabling users to receive step-by-step guidance and build projects with coding assistance.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Authors, Writing, Gen AI, Healthcare, Language Models, Llms, Machine Learning, Products, Science 

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers.
- DORA utilizes multiple AI and large language models to enhance the writing process while ensuring accuracy through cycles of reinforcement learning and human proofreading.
- The company is also introducing hardware solutions like the PandaOmics Box for secure, offline data processing and the open-source Precious-3 GPT for research into aging and related diseases.


[How Chevron is using gen AI to strike oil - VentureBeat](https://venturebeat.com/ai/how-chevron-is-using-gen-ai-to-strike-oil/)

Topics: Energy, Gen AI, Generative AI, Machine Learning 

- Chevron is leveraging generative AI, digital twins, and robots to analyze vast datasets from oil and gas operations, enhancing efficiency and worker safety.
- The company is focused on the Permian Basin, utilizing publicly available data to gain insights and maintain a competitive edge in oil and gas production.
- Chevron emphasizes collaboration between teams and digital scholars, integrating data science with engineering to drive innovation while also addressing carbon sequestration and environmental concerns.


[From Hype to Results: Accelerating AI in Automotive - Google News](https://news.google.com/articles/CBMiWWh0dHBzOi8vd3d3LmJhaW4uY29tL2luc2lnaHRzL2Zyb20taHlwZS10by1yZXN1bHRzLWFjY2VsZXJhdGluZy1haS1pbi1hdXRvbW90aXZlLXdlYmluYXIv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Gen AI, Transportation 

- Bain & Company emphasizes the importance of essential steps for automotive businesses to effectively implement AI innovations.
- The focus is on transforming hype surrounding AI into tangible results within the automotive industry.
- The company offers webinars featuring expert insights on leveraging AI for improved business outcomes.


[Top AI Job Skills Training Programs - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- AI job skills training programs are essential for bridging the skill gap between the growing demand for AI professionals and the current workforce's capabilities.
- These programs offer flexibility and accessibility, allowing learners to acquire relevant AI skills at their own pace, which is particularly beneficial for working professionals.
- Completion of AI training significantly enhances employment prospects, often resulting in recognized certifications that add value to resumes and demonstrate commitment to professional development.


[NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Gen AI, Healthcare, Machine Learning, Science, Study, Testing 

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers.
- In a comparison with human physicians, AI initially scored higher on recall in a closed-book setting, but human counterparts outperformed in open-book challenges, particularly on more complex cases.
- The findings indicate potential synergy between AI tools and human expertise in diagnostics, but emphasize the necessity for enhanced explanatory abilities and comprehensive evaluations before AI can replace human experience in healthcare.


[How Pearson's AI assistant can help teachers save time - Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Education, Gen AI, Llms, Products 

- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation, set to be available in August on their MyLab and Mastering platforms.
- The AI tool enables instructors to quickly generate assignments by sorting through numerous questions based on difficulty and relevance, allowing more time for direct student engagement and deeper teaching tasks.
- A survey revealed that many instructors lack confidence in using generative AI, despite the interest in integrating it into instructional practices, highlighting a gap in AI familiarity between students and educators.


# AI Startup Funding 

[These Are the 10 AI Startups to Watch in 2024 - Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Investment, Machine Learning, Startups, Venture Capital 

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry.  
- Startups like Cognition AI and Helsing are leveraging advanced technology for practical applications, such as coding solutions and modernizing military capabilities.  
- The report emphasizes the dynamic AI landscape, driven by unique company visions and leadership, including initiatives for safe superintelligence by Ilya Sutskever.  


[Web3 AI firm Network3 raises $5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised $5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.


[AI Startup, Code Metal, Secures $16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of $16.5 million in seed funding, including a $13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach $350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.


[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[AI startup Code Metal raises $16.5m in seed funding - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- AI startup Code Metal has raised $16.5 million in total funding, with $3.45 million from a pre-seed round and $13 million in seed funding, to enhance Edge computing capabilities.
- The investment will be used to build modular AI workloads and expand the team, emphasizing the rapid transformation in software development and Edge deployment.
- Code Metal aims to optimize AI code for Edge devices, leveraging expertise from founders experienced in delivering AI solutions for major technology platforms.


[Tesla Invests in AI, Anticipates Next Major Growth Wave - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Funding, Gen AI, Growth, Investment, Machine Learning, Tesla, Venture Capital 

- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus.
- The company plans to ramp up production of the Optimus robot by 2025 for internal use and is exploring deploying its future autonomous driving and Robotaxi services, contingent on technology and regulatory advancements.
- Despite a 2% year-over-year revenue increase, Tesla reported a 40% drop in profits, with automotive revenues declining while energy generation revenue saw a substantial 100% growth.


[AI startup Cohere cuts staff after $500 million funding round - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

Topics: Cohere, Deals, Economics, Finance, Funding, Gen AI, Jobs, Careers, Labor Market, Layoffs, Llms, Machine Learning, Startup, Venture Capital 

- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a $500 million funding round that valued the company at $5.5 billion.
- Despite the layoffs, which represent a small fraction of its 400-person workforce, Cohere plans to continue hiring in several key areas, including customer operations and product design.
- The company focuses on generative AI for enterprises, distinguishing itself from competitors by avoiding consumer chatbot markets, while maintaining strong client relationships in banking and financial services.


[Better Artificial Intelligence (AI) Stock: SoundHound AI vs. C3.ai - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Opinion, Review, Stock Market, Stocks 

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least $65 million in full-year revenue for 2024.
- C3.ai offers AI software for various applications and saw a 16% revenue increase to $310.6 million in fiscal 2024, but its net losses have been rising over the years, contrasting SoundHound's improving losses.
- Wall Street analysts favor SoundHound AI with a "buy" rating and a median price of $8, while C3.ai holds a "hold" rating with a median price of $29.50, making SoundHound appear the better investment choice currently.


[Level AI, maker of AI-powered tools to automate customer service tasks, raised a $39.4M Series C led by Adams Street Partners, taking its total raised to $73.1M - TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)

Topics: Automation, Customer Service, Deals, Finance, Funding, Venture Capital 

- Level AI, a startup focused on automating customer service tasks using AI, raised nearly $40 million in funding to expand its platform and capabilities.
- The platform provides tools for monitoring and improving contact center performance, enabling insights into customer sentiment and personalized agent guidance during calls.
- Despite growth potential, the company faces challenges including data privacy concerns, integration issues, and the psychological impact of monitoring on employees in the call center industry.


[Startup Pearl Raises VC Funding to Bring AI to Dentists Offices - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices)

Topics: Deals, Finance, Funding, Gen AI, Healthcare, Venture Capital 

- Pearl Inc., a Los Angeles-based dental startup, has raised $58 million in funding, valuing the company at $400 million.
- The company uses AI technology approved by the FDA to enhance dental diagnostics, having received regulatory clearance in 120 countries.
- Co-founder Ophir Tanz aims for an IPO, stating that Pearl's technology could transform the standards of dental practices worldwide.


# AI Smartphones 

[After 6 Months, These Are My Favorite Samsung Galaxy AI Features - Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNuZXQuY29tL3RlY2gvbW9iaWxlL215LWZhdm9yaXRlLXNhbXN1bmctZ2FsYXh5LWFpLWZlYXR1cmVzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Opinion, Products, Review, Samsung 

- Samsung's Galaxy AI features, including advanced photo editing and language translation tools, were introduced with the Galaxy S24 series and expanded with the launch of the Galaxy Z Fold 6 and Galaxy Z Flip 6.
- Noteworthy tools include Circle to Search for instant searching by circling content on the screen, and Portrait Studio which stylizes photos of people into artistic portraits, showcasing a blend of creativity and practicality.
- Language translation advancements, such as Live Translate for real-time phone conversations and instant chat translation, highlight Samsung's focus on enhancing communication through AI while raising questions about authenticity in photo editing.


[Lenovo to put AI into all PCs by 2027: device chief - Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vYXNpYS5uaWtrZWkuY29tL0J1c2luZXNzL1RlY2hub2xvZ3kvQXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UvTGVub3ZvLXRvLXB1dC1BSS1pbnRvLWFsbC1QQ3MtYnktMjAyNy1kZXZpY2UtY2hpZWbSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Lenovo, Products, Science 

- Lenovo plans to integrate artificial intelligence features into all new PCs by 2027, according to executive Luca Rossi.  
- The initiative is driven by strong interest from consumers in both China and the U.S.  
- Luca Rossi leads the Intelligence Devices Group, which manages the company's PC and smartphone operations.  


[Better Artificial Intelligence (AI) Stock: SoundHound AI vs. C3.ai - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Opinion, Review, Stock Market, Stocks 

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least $65 million in full-year revenue for 2024.
- C3.ai offers AI software for various applications and saw a 16% revenue increase to $310.6 million in fiscal 2024, but its net losses have been rising over the years, contrasting SoundHound's improving losses.
- Wall Street analysts favor SoundHound AI with a "buy" rating and a median price of $8, while C3.ai holds a "hold" rating with a median price of $29.50, making SoundHound appear the better investment choice currently.


[1 Top Artificial Intelligence (AI) Stock to Buy Before It Soars 84%, According to Loop Capital - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Nvidia, Opinion, Stock Market, Stocks 

- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases.
- The company has shifted from low growth to a revenue CAGR of 42% from 2021 to 2023, largely due to its partnership with Nvidia and dominance in the AI server market, which is expected to expand significantly.
- While bullish analysts project Supermicro could reach a stock price of $1,500, concerns exist over increased competition from major players like Dell and HPE and potential market slowdowns, making the average price target of over $1,000 more realistic.


[The genAI boyfriend business is booming - Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Entertainment, Gen AI, Growth, Jobs, Careers, Labor Market, Products, Society, Culture 

- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners.
- The popularity of AI companion apps is on the rise, with high user engagement rates and a substantial increase in the number of new apps entering the market.
- Critics warn that reliance on chatbot companionship could weaken human relationships and pose privacy risks regarding personal data usage.


[Nine ways Artificial Intelligence is making its way into cars - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Gen AI, Products, Science, Transportation 

- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety.
- Advanced algorithms are improving vehicle performance monitoring and offering personalized insurance based on driving data.
- Autonomous delivery services and ride-hailing innovations are emerging, driven by AI, allowing vehicles to operate autonomously and generate income.


[Meta AI Is Now Multilingual, More Creative and Smarter - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Creativity, Gen AI, Language Models, Llms, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook.
- New creative tools, including "Imagine me" prompts for personalized image generation and editing capabilities, allow users to create and share custom images easily.
- The advanced Llama 405B model within Meta AI offers improved support for complex math and coding questions, enabling users to receive step-by-step guidance and build projects with coding assistance.


[Enhancing Automotive Customer Experience With AI: Gnani.ai - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Customer Service, Gen AI, Transportation 

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages.
- Post-purchase, AI automates maintenance reminders and helps in upselling by analyzing customer profiles, thereby improving satisfaction and boosting sales for automotive companies.
- Gopalan emphasizes the importance of AI complementing human roles rather than replacing them, with a focus on creating a seamless synergy that enhances customer service and personalizes the overall ownership experience.


[Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Streaming 

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button.
- The Image Playground, teased at WWDC 2024, enables users to generate AI images in various styles, though photorealistic options are not available.
- Currently under development, the AI features may be delayed until iOS 18.1 or later, with a full iOS 18 release expected this fall.


[Meta AI gets new Imagine me selfie feature - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tZXRhLWFpLWdldHMtbmV3LXNlbGZpZS1mZWF0dXJlcy1xdWVzdC1zdXBwb3J0L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Big Tech, Facial Recognition, Feature, Gen AI, Meta, Meta AI, Products 

- Meta AI has introduced a new selfie feature called "Imagine Yourself," allowing users to create stylized images based on prompts and personal photos, utilizing a generative AI model.
- The AI now includes the upgraded Llama 3.1 405B model for handling complex queries, improving performance in math and coding, though users must manually switch to this model and face query limits.
- Meta AI is expanding its language support and will replace the Meta Quest's Voice Commands feature with AI capabilities in experimental mode, enabling users to interact with their physical environment.


# Big Tech Antitrust Investigations 

[Big Tech says AI is booming. Wall Street is starting to see a bubble. - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Bubble, Economics, Finance, Gen AI, Stocks 

- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of $60 billion annually against only $20 billion in expected revenue by 2026.
- Despite significant advancements in AI technology and its applications, skepticism grows over whether companies can recover investments, with venture capital exits declining and few successful AI products outside of notable examples like ChatGPT.
- Industry executives acknowledge the high costs involved in AI development but believe the long-term value will justify continued investment, warning that unrealistic expectations could lead to difficulties for smaller startups amidst competitive pressures.


[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[Mark Zuckerberg Slams Tim Cook's Apple After Cupertino Reportedly Rejected Its iPhone AI Partnership Proposal: 'They Wanted To Put Us In A Box...' - Google News](https://news.google.com/articles/CBMikAFodHRwczovL3d3dy5iZW56aW5nYS5jb20vbmV3cy8yNC8wNy8zOTkyNjA3Ni9tYXJrLXp1Y2tlcmJlcmctc2xhbXMtdGltLWNvb2tzLWFwcGxlLWFmdGVyLWN1cGVydGluby1yZXBvcnRlZGx5LXJlamVjdGVkLWl0cy1pcGhvbmUtYWktcGFydG5lcnNoaXDSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Big Tech, Gen AI, Mark Zuckerberg, Meta, Opinion 

- Mark Zuckerberg criticized Apple's "soul-crushing" tactics after the company rejected Meta's proposal to integrate its AI chatbot into the iPhone, citing concerns over privacy practices.
- Zuckerberg expressed his desire for Meta to control the next generation of technology by releasing a large open-source AI model, aiming to shape the technological landscape independently of Apple.
- Apple's rejection of the partnership comes amidst its announcements of collaborations with OpenAI and potential integrations with other AI startups, highlighting the competitive tension between the two companies.


[Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Competition, Ethics, Gen AI, Policy And Regulation 

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development.  
- The principles highlight the risks posed by partnerships and financial arrangements between major AI firms that could undermine competition.  
- The regulators promote fair dealings and interoperability among AI products and services, asserting that claims against interoperability will be closely examined.


[Get paid or sue? How the news business is combating the threat of AI - Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmxhdGltZXMuY29tL2VudGVydGFpbm1lbnQtYXJ0cy9idXNpbmVzcy9zdG9yeS8yMDI0LTA3LTI0L2hvdy1haS1pcy1kaXNydXB0aW5nLXRoZS1qb3VybmFsaXNtLWluZHVzdHJ50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Intellectual Property, Llms 

- News organizations are confronting declining revenue due to digital ads and subscriptions, now compounded by AI technology that potentially infringes on their content by summarizing articles without compensation.
- Major publications like The New York Times and the Associated Press are moving towards licensing agreements with AI companies or pursuing legal action to protect their content and maintain viable business models.
- Smaller outlets, such as L.A. Taco, face greater challenges in responding to AI disruptions due to limited resources, emphasizing the need for regulatory measures to ensure fair compensation from tech companies.


[Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Cybersecurity, Ethics, Gen AI, Laws, Llms, OpenAI, Policy And Regulation, Politics 

- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators.
- Whistleblowers claimed OpenAI imposed restrictive agreements that hindered employees from raising concerns and waiving their rights to whistleblower compensation.
- The senators requested assurances from OpenAI's CEO that the company would eliminate non-disparagement agreements and improve protections for employees who report issues.


[Alphabet's earnings show investors are impatient for the data on AI returns. No one has the answer yet. - Business Insider](https://www.businessinsider.com/alphabets-investors-want-data-ai-returns-no-one-has-answers-2024-7)

Topics: Alphabet, Big Tech, Data, Economics, Finance, Gen AI, Google, Stocks 

- During its earnings call, Alphabet's leaders faced investor demands for data on returns from AI projects, but provided limited insights, emphasizing a focus on quality and gradual rollout instead.
- CEO Sundar Pichai highlighted the potential of AI to drive long-term revenue growth, particularly in the cloud business, which recently surpassed $10 billion in revenue due to AI demand.
- While Alphabet is investing heavily in AI technologies, there is uncertainty regarding monetization strategies and overall returns, leading to investor concerns about the sustainability of such investments.


[Google keeps the cost of AI search flat, and kids are lovin' itAs the G-Cloud brings in big bucks and plentiful profitAI + ML12 hrs|3 - The Register](https://www.theregister.com/2024/07/24/alphabet_q2_2024/)

Topics: Big Tech, Gen AI, Google, Machine Learning 

- Google has successfully maintained flat costs for its AI search tool, "AI overviews," while enhancing user engagement among the younger demographic (ages 18 to 24).
- Alphabet's cloud services have significantly contributed to revenue growth, with Q2 2024 showing a 29% year-on-year revenue increase, totaling $10.35 billion.
- CEO Sundar Pichai emphasized ongoing improvements in AI model capabilities and their application across consumer and enterprise sectors, indicating future growth potential in AI solutions.


[Google Fails to Wow as AI Bills Mount - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vd3d3Lndzai5jb20vdGVjaC9nb29nbGUtZmFpbHMtdG8td293LWFzLWFpLWJpbGxzLW1vdW50LWVmNDkzOTlk0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Gen AI, Google, Opinion 

- The website requires JavaScript to be enabled for full functionality.
- Users are prompted to disable any ad blockers to access content.
- There is no specific article or content available due to access restrictions.


[Google Is the Only Search Engine That Works on Reddit Now Thanks to AI Deal - 404 Media](https://www.404media.co/google-is-the-only-search-engine-that-works-on-reddit-now-thanks-to-ai-deal/)

Topics: Big Tech, Deals, Gen AI, Google, Machine Learning, Reddit 

- Google is now the only search engine able to access and display results from Reddit due to a multi-million dollar deal, effectively sidelining alternatives like DuckDuckGo, Bing, and Mojeek.
- Reddit has updated its robots.txt to block all crawlers except Google’s, as part of its effort to restrict content scraping by AI companies and protect user-generated contributions.
- The exclusive access granted to Google could worsen the competitiveness of other search engines and reflects a larger trend of larger platforms diminishing the visibility of smaller entities on the web.


# Large Language Models 

[honeyIBrokeTheLanguageModel - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1eavhzh/honeyibrokethelanguagemodel/)

Topics: Language Models 

- The discussion revolves around humor in programming and software development, with a focus on the absurdities of AI language models.
- Users humorously debate the unpredictability of AI responses and the philosophical implications of its outputs.
- References to cats and playful language convey a light-hearted take on the serious concerns surrounding technology and AI.


[AI arms race escalates: OpenAI offers free GPT-4o Mini fine-tuning to counter Metas Llama 3.1 release - VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/)

Topics: Arms Race, Big Tech, Gen AI, Language Models, Llama 3.1, Llms, Machine Learning, Meta, OpenAI 

- OpenAI has launched free fine-tuning for its GPT-4o Mini model, directly responding to Meta's introduction of the open-source Llama 3.1 model, intensifying competition in the AI space.
- The GPT-4o Mini's customization features enable businesses to tailor AI solutions while maintaining some level of control, advancing the trend toward accessibility in AI technologies.
- As the AI landscape evolves, businesses face decisions between open-source and proprietary options, with the necessity to adapt strategies to meet emerging technologies and ethical considerations.


# AI-Powered Customer Service Agents 

[Enhancing Automotive Customer Experience With AI: Gnani.ai - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Automotive, Customer Service, Gen AI, Transportation 

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages.
- Post-purchase, AI automates maintenance reminders and helps in upselling by analyzing customer profiles, thereby improving satisfaction and boosting sales for automotive companies.
- Gopalan emphasizes the importance of AI complementing human roles rather than replacing them, with a focus on creating a seamless synergy that enhances customer service and personalizes the overall ownership experience.


[Can AI teach you to smile? Japan's firm uses it for customer satisfaction - Google News](https://news.google.com/articles/CBMiiQFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vd29ybGQtbmV3cy9jYW4tYWktdGVhY2gteW91LXRvLXNtaWxlLWphcGFuLXMtZmlybS11c2VzLWl0LWZvci1jdXN0b21lci1zYXRpc2ZhY3Rpb24tMTI0MDcyNDAwNjE5XzEuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Satisfaction, Customer Service, Gen AI, Japan, Machine Learning, Science, Society, Culture 

- AEON supermarket chain in Japan has implemented 'Mr Smile', an AI system that evaluates employees' service attitudes based on facial expressions, voice volume, and tone to enhance customer service.
- The system demonstrated a 1.6-fold improvement in service attitudes during a trial involving 3,400 employees, although it has raised concerns about potential harassment and the commodification of genuine smiles.
- Similar to McDonald's 'Smile zero yen' initiative, the standardized smile approach has faced criticism for adding stress to employees, prompting guidelines from the Japanese Ministry of Health, Labour and Welfare to prioritize staff well-being.


[Lenovo takes customer satisfaction to new heights with e-services enhanced by generative AI - Google News](https://news.google.com/articles/CBMiR2h0dHBzOi8vbmV3cy5sZW5vdm8uY29tL2N1c3RvbWVyLXNhdGlzZmFjdGlvbi1lLXNlcnZpY2VzLWdlbmVyYXRpdmUtYWkv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Customer Satisfaction, Customer Service, Gen AI, Generative AI, Lenovo, Products 

- Lenovo has integrated generative AI, specifically the GPT-4 model, to enhance customer support interactions on its forums, improving the speed, accuracy, and quality of responses.  
- The AI-powered forum assistant has achieved an 82% accuracy rate for generating responses and anticipates a 50% increase in efficiency over the next year.  
- Lenovo is set to expand its GenAI technology to enhance existing chatbots (Lena and Moli), allowing for more human-like interactions and the capability to handle a broader range of customer inquiries.


[The genAI boyfriend business is booming - Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Entertainment, Gen AI, Growth, Jobs, Careers, Labor Market, Products, Society, Culture 

- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners.
- The popularity of AI companion apps is on the rise, with high user engagement rates and a substantial increase in the number of new apps entering the market.
- Critics warn that reliance on chatbot companionship could weaken human relationships and pose privacy risks regarding personal data usage.


[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture 

- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.
- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.
- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.


[Sinister or Revolutionary? A Japanese Supermarket Chain is Using AI to Monitor Staff Smiles - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5jeHRvZGF5LmNvbS9zcGVlY2gtYW5hbHl0aWNzL3NpbmlzdGVyLW9yLXJldm9sdXRpb25hcnktYS1qYXBhbmVzZS1zdXBlcm1hcmtldC1jaGFpbi1pcy11c2luZy1haS10by1tb25pdG9yLXN0YWZmLXNtaWxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Service, Ethics, Facial Recognition, Gen AI, Japan, Machine Learning, Opinion, Privacy, Society, Culture 

- AEON's "Mr Smile" AI system monitors the demeanor and tone of its customer service staff to standardize and enhance employee smiles across 240 stores.
- While the trial reported a 1.6 times improvement in staff attitudes, it raises ethical concerns about pressuring employees to conform to a standardized expression of customer service.
- Criticism from workers highlights that enforced smiles may lead to inauthenticity and loss of individual employee expression, comparing it to customer harassment.


[How Pearson's AI assistant can help teachers save time - Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Assistant, Education, Gen AI, Llms, Products 

- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation, set to be available in August on their MyLab and Mastering platforms.
- The AI tool enables instructors to quickly generate assignments by sorting through numerous questions based on difficulty and relevance, allowing more time for direct student engagement and deeper teaching tasks.
- A survey revealed that many instructors lack confidence in using generative AI, despite the interest in integrating it into instructional practices, highlighting a gap in AI familiarity between students and educators.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[Japan Supermarket Chain Becomes World's First To Use AI To "Assess And Standardise" Staff Smiles - Google News](https://news.google.com/articles/CBMiggFodHRwczovL3d3dy5uZHR2LmNvbS9mZWF0dXJlL2phcGFuLXN1cGVybWFya2V0LWNoYWluLWJlY29tZXMtd29ybGRzLWZpcnN0LXRvLXVzZS1haS10by1hc3Nlc3MtYW5kLXN0YW5kYXJkaXNlLXN0YWZmLXNtaWxlcy02MTc2MDc10gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Service, Facial Recognition, Gen AI, Japan, Society, Culture, Staff Smiles, Supermarket 

- AEON, a Japanese supermarket chain, is the first in the world to implement an AI system called 'Mr Smile' to assess and standardize employee smiles across its 240 stores.
- The AI evaluates over 450 factors, including facial expressions and voice tone, aiming to enhance customer satisfaction and improve service attitude, which reportedly improved by 1.6 times during trials.
- The initiative has raised concerns about workplace harassment, as critics argue it commodifies genuine expressions of emotion and imposes unrealistic standards on staff behavior.


[Japan supermarket chain uses AI to assess staff smiles, sparking harassment concerns - Google News](https://news.google.com/articles/CBMiigFodHRwczovL3d3dy5tb25leWNvbnRyb2wuY29tL25ld3MvdHJlbmRzL2phcGFuLXN1cGVybWFya2V0LWNoYWluLXVzZXMtYWktdG8tYXNzZXNzLXN0YWZmLXNtaWxlcy1zcGFya2luZy1oYXJhc3NtZW50LWNvbmNlcm5zLTEyNzc2OTk0Lmh0bWzSAY4BaHR0cHM6Ly93d3cubW9uZXljb250cm9sLmNvbS9uZXdzL3RyZW5kcy9qYXBhbi1zdXBlcm1hcmtldC1jaGFpbi11c2VzLWFpLXRvLWFzc2Vzcy1zdGFmZi1zbWlsZXMtc3BhcmtpbmctaGFyYXNzbWVudC1jb25jZXJucy0xMjc3Njk5NC5odG1sL2FtcA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Service, Ethics, Facial Recognition, Japan, Jobs, Careers, Labor Market, Machine Learning, Privacy, Society, Culture, Staff Smiles, Supermarket 

- AEON has implemented an AI system called “Mr Smile” across its 240 stores to assess employee smiles and service attitudes, evaluating over 450 factors like facial expressions and voice tone.
- The initiative has sparked concerns regarding potential workplace harassment, as critics argue that forcing a standardized smile may exacerbate mistreatment from customers, who are already a source of concern for service industry workers in Japan.
- Despite a reported improvement in service attitude, labor unions emphasize the need for balancing customer expectations with employee well-being, as previous guidelines from Japan’s Ministry of Health encouraged upholding service standards without compromising worker rights.


# AI Ethics and Regulation 

[Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Competition, Ethics, Gen AI, Policy And Regulation 

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development.  
- The principles highlight the risks posed by partnerships and financial arrangements between major AI firms that could undermine competition.  
- The regulators promote fair dealings and interoperability among AI products and services, asserting that claims against interoperability will be closely examined.


[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[An IT Leaders Playbook for Creating an Effective AI Policy - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Policy And Regulation 

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management.
- Key benefits of AI adoption include improved sales analytics, enhanced customer service, automation of routine tasks, and overall increased operational efficiency.
- An effective AI policy framework should define important terms, identify risks, prohibit misuse, ensure compliance, and establish consequences for violations.


[Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Chatgpt, Cybersecurity, Ethics, Gen AI, Laws, Llms, OpenAI, Policy And Regulation, Politics 

- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators.
- Whistleblowers claimed OpenAI imposed restrictive agreements that hindered employees from raising concerns and waiving their rights to whistleblower compensation.
- The senators requested assurances from OpenAI's CEO that the company would eliminate non-disparagement agreements and improve protections for employees who report issues.


[OpenAI removes AI safety executive Aleksander Madry from role - Reddit](https://www.reddit.com/r/technology/comments/1eaj0gb/openai_removes_ai_safety_executive_aleksander/)

Topics: Ethics, Gen AI, OpenAI 

- OpenAI has removed AI safety executive Aleksander Madry from his role, with some suggesting it may affect the company’s commitment to safety protocols in AI development.
- The reassignment sparked speculation and criticism from the community, emphasizing concerns about prioritizing speed over safety in AI advancements.
- There is a mix of sentiments among users, with some expressing hope that this change will lead to more effective AI applications, while others foresee negative consequences for the company.


[When it comes to political advertising, is AI ever OK? - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vdGhlY29udmVyc2F0aW9uLmNvbS93aGVuLWl0LWNvbWVzLXRvLXBvbGl0aWNhbC1hZHZlcnRpc2luZy1pcy1haS1ldmVyLW9rLTIzNTMyM9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Policy And Regulation, Politics 

- The use of AI in political advertising is increasing globally, raising ethical concerns, particularly regarding its role in shaping public perception and manipulating content.
- The Liberal National Party Queensland's innovative yet controversial use of AI to craft a political message highlights the blurred lines between negative campaigning and ethical standards in political dialogue.
- There is a pressing need for clear regulations to govern AI use in political campaigns, ensuring transparency and maintaining the integrity of democratic processes while embracing technological advancements.


[A Kamala Harris Presidency Could Mean More of the Same on A.I. Regulation - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMjQvdGVjaG5vbG9neS9rYW1hbGEtaGFycmlzLWFpLXJlZ3VsYXRpb24uaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Ethics, Gen AI, Kamala Harris, Laws, Opinion, Policy And Regulation, Politics 

- Kamala Harris has played a significant role in leading U.S. initiatives on artificial intelligence (A.I.), achieving voluntary safety standards with major tech companies, but struggles to pass concrete regulations in Congress.
- While she has connections to the tech industry and supports both innovation and public safety, experts believe her presidency could maintain a relatively lenient regulatory environment for A.I. companies.
- Harris emphasizes the need for corporate accountability and has asserted the moral duty of regulating A.I. to protect the public against potential harm, despite the current lack of firm legislative action.


[MNPS board votes on AI policy - Google News](https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LndzbXYuY29tL3ZpZGVvLzIwMjQvMDcvMjIvbW5wcy1ib2FyZC12b3Rlcy1haS1wb2xpY3kv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Policy, Education, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics 

- The MNPS board has voted on a policy regarding the use of artificial intelligence in schools.
- Public universities and school systems in Tennessee are required to submit their AI policies this year.
- Ongoing local news reports include incidents of violence and crime in the area, unrelated to the AI policy discussion.


[Sinister or Revolutionary? A Japanese Supermarket Chain is Using AI to Monitor Staff Smiles - Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5jeHRvZGF5LmNvbS9zcGVlY2gtYW5hbHl0aWNzL3NpbmlzdGVyLW9yLXJldm9sdXRpb25hcnktYS1qYXBhbmVzZS1zdXBlcm1hcmtldC1jaGFpbi1pcy11c2luZy1haS10by1tb25pdG9yLXN0YWZmLXNtaWxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Customer Service, Ethics, Facial Recognition, Gen AI, Japan, Machine Learning, Opinion, Privacy, Society, Culture 

- AEON's "Mr Smile" AI system monitors the demeanor and tone of its customer service staff to standardize and enhance employee smiles across 240 stores.
- While the trial reported a 1.6 times improvement in staff attitudes, it raises ethical concerns about pressuring employees to conform to a standardized expression of customer service.
- Criticism from workers highlights that enforced smiles may lead to inauthenticity and loss of individual employee expression, comparing it to customer harassment.


[Are We Witnessing the Weaponization of AI? | Opinion - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Ethics, Gen AI, Opinion, Policy And Regulation 

- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests.
- The relationship between Big Tech and the U.S. government is deepening, leading to the potential use of advanced AI systems for surveillance under the guise of national security, which may infringe on personal freedoms and privacy.
- The "revolving door" phenomenon between tech and government highlights an increasing cooperation, with OpenAI's policy changes allowing military applications, signaling a broader trend of weaponizing AI.


# AI Art by Microsoft 

[WIRED: Activision Artists Were Forced to Use AI, Company Sold AI-Made CoD Skin - Google News](https://news.google.com/articles/CBMiZGh0dHBzOi8vODAubHYvYXJ0aWNsZXMvd2lyZWQtYWN0aXZpc2lvbi1hcnRpc3RzLXdlcmUtZm9yY2VkLXRvLXVzZS1haS1jb21wYW55LXNvbGQtYWktbWFkZS1jb2Qtc2tpbi_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Activision, Art, Design, Call Of Duty, Entertainment, Ethics, Gaming, Gen AI, Intellectual Property, Machine Learning 

- Activision recently implemented AI tools for concept art, causing concern among artists who feel their creativity and humanity are being overlooked, particularly after significant layoffs.
- While the company claimed AI wouldn't replace staff or be used for final game assets, it later sold an AI-generated cosmetic item for a major game.
- Opinions on AI in gaming vary, with some companies exploring its benefits while others, like CD PROJEKT RED, warn against its potential to lower quality in game development.


[Adobe's Photoshop can now generate AI images via prompts like Dall-E or Midjourney - Google News](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9hZG9iZXMtcGhvdG9zaG9wLWNhbi1ub3ctZ2VuZXJhdGUtYWktaW1hZ2VzLXZpYS1wcm9tcHRzLWxpa2UtZGFsbC1lLW9yLW1pZC1qb3VybmV5LTEzMDAxODE4MS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Adobe, Art, Design, Big Tech, Gen AI, Machine Learning, Photoshop, Products 

- Adobe has launched a new text-to-image generation feature in Photoshop, enabling users to create images from scratch using prompts similar to Dall-E and Midjourney, powered by the Firefly Image 3 model.
- The updated feature allows for significant creative flexibility, letting users generate images and make adjustments directly within Photoshop, which streamlines the workflow for artists.
- In response to concerns from artists regarding the use of their work for AI training, Adobe has reiterated its commitment to a creator-friendly approach, assuring that it will not utilize customer content for this purpose.


[Activision Reportedly Sold an AI-Generated Call of Duty: Modern Warfare 3 Cosmetic - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3Lmlnbi5jb20vYXJ0aWNsZXMvYWN0aXZpc2lvbi1yZXBvcnRlZGx5LXNvbGQtYW4tYWktZ2VuZXJhdGVkLWNhbGwtb2YtZHV0eS1tb2Rlcm4td2FyZmFyZS0zLWNvc21ldGlj0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Activision, Art, Design, Call Of Duty, Entertainment, Ethics, Gaming, Gen AI, Intellectual Property, Machine Learning, Products 

- Activision sold an AI-generated cosmetic for Call of Duty: Modern Warfare 3, reportedly as part of the Yokai’s Wrath bundle, without disclosing what was AI-generated.
- The use of generative AI in the gaming industry is concerning many artists, with reports of layoffs and job replacements occurring at Activision after Microsoft's acquisition.
- Major figures in the gaming industry acknowledge the potential benefits of AI while also raising ethical concerns and issues related to intellectual property rights.


[Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Streaming 

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button.
- The Image Playground, teased at WWDC 2024, enables users to generate AI images in various styles, though photorealistic options are not available.
- Currently under development, the AI features may be delayed until iOS 18.1 or later, with a full iOS 18 release expected this fall.


[Apple Intelligence to Generate Playlist Artwork in iOS 18 - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yNC9hcHBsZS1pbnRlbGxpZ2VuY2UtdG8tZ2VuZXJhdGUtcGxheWxpc3QtYXJ0d29yay_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Recommendation Systems, Streaming 

- iOS 18 will introduce a feature in Apple Music that allows users to generate customized playlist artwork using generative AI, as revealed in beta code.
- The functionality includes a "Create Image" button that uses the Image Playground tool to create unique images based on user input, utilizing different styles like animation and illustration.
- While expected to launch with iOS 18, the full rollout of Apple Intelligence features may be delayed until iOS 18.1 or beyond, with the official release anticipated in the fall.


[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture 

- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.
- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.
- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.


[Adobe Adds New AI Tools to Photoshop and Illustrator - Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yMy9waG90b3Nob3AtaWxsdXN0cmF0b3ItbmV3LWFpLXRvb2xzL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Adobe, Art, Design, Big Tech, Gen AI, Illustrator, Machine Learning, Photoshop, Products 

- Adobe has introduced new AI-driven features in Photoshop and Illustrator, enhancing productivity through tools like the Selection Brush Tool and Generative Shape Fill.
- Photoshop now includes an Adjustment Brush Tool for fine-tuning brightness and saturation adjustments, as well as an upgraded Contextual Task Bar and Enhance Detail option for Generative Fill.
- Illustrator's updates include improved Text to Pattern capabilities, a new Mockup feature for visualizing designs on products, and enhancements for faster and more precise editing of vector graphics.


[Microsoft Aims to Boost AI Workload Capacity With Lumen Technologies Partnership - Google News](https://news.google.com/articles/CBMijAFodHRwczovL21vbmV5LnVzbmV3cy5jb20vaW52ZXN0aW5nL25ld3MvYXJ0aWNsZXMvMjAyNC0wNy0yNC9taWNyb3NvZnQtYWltcy10by1ib29zdC1haS13b3JrbG9hZC1jYXBhY2l0eS13aXRoLWx1bWVuLXRlY2hub2xvZ2llcy1wYXJ0bmVyc2hpcNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Deals, Gen AI, Lumen Technologies, Machine Learning, Microsoft, Partnership 

- Microsoft is partnering with Lumen Technologies to expand its AI workload capacity using Lumen's network equipment, addressing issues with data center infrastructure shortages.
- The collaboration includes Lumen leveraging Microsoft Azure services to reduce costs, projected to enhance Lumen's cash flow by over $20 million in the next year.
- This partnership comes as Microsoft invests heavily in cloud infrastructure to maintain its competitive edge in AI, particularly following its investment in OpenAI.


[Meta AI Introduces AI-Generated Photos to All Platforms - Google News](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnNlYXJjaGVuZ2luZWpvdXJuYWwuY29tL21ldGEtYWktaW50cm9kdWNlcy1haS1nZW5lcmF0ZWQtcGhvdG9zLXRvLWFsbC1wbGF0Zm9ybXMvNTIyODM1L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Big Tech, Gen AI, Machine Learning, Meta, Meta AI, Products, Science 

- Meta AI introduces advanced image generation and editing features on Facebook, Instagram, Messenger, and WhatsApp through the new tool "Imagine Me," allowing real-time transformations based on user prompts.
- Users can now edit images extensively, with a forthcoming "Edit With AI" button enhancing capabilities to modify objects in photos.
- Meta AI expands its accessibility to seven new countries and languages, increasing its availability to 22 countries and supporting seven additional languages.


[Meta AI gets new Imagine me selfie feature - Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tZXRhLWFpLWdldHMtbmV3LXNlbGZpZS1mZWF0dXJlcy1xdWVzdC1zdXBwb3J0L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Art, Design, Big Tech, Facial Recognition, Feature, Gen AI, Meta, Meta AI, Products 

- Meta AI has introduced a new selfie feature called "Imagine Yourself," allowing users to create stylized images based on prompts and personal photos, utilizing a generative AI model.
- The AI now includes the upgraded Llama 3.1 405B model for handling complex queries, improving performance in math and coding, though users must manually switch to this model and face query limits.
- Meta AI is expanding its language support and will replace the Meta Quest's Voice Commands feature with AI capabilities in experimental mode, enabling users to interact with their physical environment.


# AI Chip Demand Surge 

[Nvidia said to launch new AI chip for China - Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, China, Gen AI, Machine Learning, Nvidia, Products, Science, Semiconductor Chips 

- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls.
- There is speculation that the H20 chip might be banned for sale to China due to anticipated changes in U.S. export policies, which could also affect other Asian countries.
- Reports have highlighted an underground network smuggling banned A100 chips into China, indicating high demand and significant market value despite strict export regulations.


[1 Top Artificial Intelligence (AI) Stock to Buy Before It Soars 84%, According to Loop Capital - Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Finance, Gen AI, Investment, Nvidia, Opinion, Stock Market, Stocks 

- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases.
- The company has shifted from low growth to a revenue CAGR of 42% from 2021 to 2023, largely due to its partnership with Nvidia and dominance in the AI server market, which is expected to expand significantly.
- While bullish analysts project Supermicro could reach a stock price of $1,500, concerns exist over increased competition from major players like Dell and HPE and potential market slowdowns, making the average price target of over $1,000 more realistic.


[By embracing liquid cooling, AI powerhouse Supermicro enables 30% more computing power  with the same power budget - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Products, Science 

- Supermicro's CEO predicts a major AI revolution, with the company's revenue growing by 200% year-over-year and projected to increase another 152% by year-end 2024.
- The company aims for liquid cooling solutions to comprise 20% to 30% of data center deployments over the next year, enabling 30% more computing power while consuming 30-40% less energy.
- Supermicro is ramping up production, currently shipping 4,000 racks per month, with plans to increase liquid-cooled rack shipments to about 2,000 monthly to meet rising AI demands.


[Opinion: This Nvidia Forecast All but Confirms That the Artificial Intelligence (AI) Bubble Will Burst Sooner Rather Than Later - Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Bubble, Economics, Gen AI, Nvidia, Opinion 

- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting.  
- The increasing competition from Intel and AMD, alongside tech giants developing in-house AI chips, indicates a reduction in reliance on Nvidia's products and a potential decrease in its market dominance.  
- Historical patterns of overestimation in new technologies imply that the current AI boom may face a correction, as many businesses lack clear plans for leveraging AI investments effectively in the near term.


[Elon Musk doubles down on Tesla's Dojo supercomputer, aiming to compete with Nvidia - Business Insider](https://www.businessinsider.com/elon-musk-doubles-down-tesla-dojo-compete-with-nvidia-2024-7)

Topics: Big Tech, Competition, Data, Elon Musk, Gen AI, Machine Learning, Nvidia, Semiconductor Chips, Tesla 

- Elon Musk reaffirmed Tesla's commitment to developing its Dojo supercomputer to compete with Nvidia's AI chips amid challenges in accessing Nvidia GPUs.
- He expressed concerns over the high demand for Nvidia's hardware, which makes it difficult for Tesla to secure essential graphics processing units.
- Musk emphasized the necessity for Tesla to enhance Dojo's capabilities to reduce reliance on Nvidia, acknowledging the significant market lead Nvidia holds in AI chip production.


[Trump Trade May Reverse As Probability Of Biden Backing Out Increases  Watch AI Semiconductors - Google News](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5iZW56aW5nYS5jb20vMjQvMDcvMzk4NDM4NzIvdHJ1bXAtdHJhZGUtbWF5LXJldmVyc2UtYXMtcHJvYmFiaWxpdHktb2YtYmlkZW4tYmFja2luZy1vdXQtaW5jcmVhc2VzLXdhdGNoLWFpLXNlbWljb25kdWN0b3Jz0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Finance, Policy And Regulation, Politics, Trade 

- Taiwan Semiconductor (TSM) plays a crucial role in the semiconductor industry, producing advanced AI chips, and recently reported strong earnings with over 50% of revenues from AI, indicating significant growth potential despite market volatility driven by political factors.
- The Trump trade may reverse if the probability of Biden backing out and Democrats nominating a new candidate increases, highlighting the importance of monitoring key AI semiconductor stocks for signs of this shift.
- Investors are advised to separate political considerations from investment strategies, with recommendations to maintain a flexible protection band strategy to manage risk while capitalizing on potential market opportunities.


[The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Arms Race, Gen AI, Military, Opinion, Policy And Regulation, Science 

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority. 
- U.S. attempts to restrict China's access to advanced chips are proving ineffective and costly, with significant funds and resources mobilized without success in curbing Chinese access to technology. 
- A shift towards cooperation, rather than exclusive competition, is proposed to enhance AI advancements and mitigate self-harm caused by current restrictive policies.


[Nvidias coveted H100 GPUs will be available on-demand through Lambda - VentureBeat](https://venturebeat.com/ai/nvidias-coveted-h100-gpus-will-be-available-on-demand-through-lambda/)

Topics: Big Tech, Gen AI, Machine Learning, Nvidia, Products, Semiconductor Chips 

- Lambda has launched 1-Click Clusters, allowing customers to access Nvidia H100 and Quantum 2 InfiniBand GPUs on-demand without long contracts, catering to companies needing temporary GPU power for training AI models.
- The service enables companies to reserve GPU nodes for a minimum of two weeks, addressing the financial burden of obtaining and maintaining their own expensive GPU clusters.
- Unlike major cloud providers that focus on inference, Lambda emphasizes providing resources for model training without the hassle of lengthy contracts, making it more accessible for smaller AI companies.


[AMD introduces new Ryzen AI 9 HX 375 "Strix Point" APU: NPU now up to 55 TOPS - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdmlkZW9jYXJkei5jb20vbmV3ei9hbWQtaW50cm9kdWNlcy1uZXctcnl6ZW4tYWktOS1oeC0zNzUtc3RyaXgtcG9pbnQtYXB1LW5wdS1ub3ctdXAtdG8tNTUtdG9wc9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Amd, Big Tech, Gen AI, Machine Learning, Products, Ryzen AI, Semiconductor Chips 

- AMD has introduced the Ryzen AI 9 HX 375 "Strix Point" APU, featuring an optimized NPU with up to 55 TOPS for enhanced AI performance.
- The new APU is part of the Strix Point series, sharing key specifications with the HX 370 SKU, including 4x Zen5 and 8x Zen5c cores and a maximum boost clock of 5.1 GHz.
- The Ryzen AI 9 HX 375 is set to debut in the OmniBook Ultra 14-inch laptop, expanding AMD's Ryzen AI 300 lineup to three models.


[Meet the Artificial Intelligence (AI) Stock That Just Soared 225% Thanks to Nvidia - Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9tZWV0LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLTA5NTMwMDIyMS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Stocks 

- Nvidia's investment in Serve Robotics has driven a 225% surge in the company's stock due to its focus on autonomous delivery systems using AI technology.
- Serve Robotics reports significant growth potential, with a notable increase in revenue, although it still faces heavy cash burn and dependency on future investments to scale operations.
- Despite Nvidia’s endorsement and the rapid growth, Serve’s inflated valuation—reflecting a very high price-to-sales ratio—suggests high risk for potential investors.


# Text-To-Image Diffusion Models 

# ChatGPT in AI Dominance 

[Asked chatGPT to show me what a typical reddit mod looks like - Reddit](https://www.reddit.com/r/ChatGPT/comments/1eauzyf/asked_chatgpt_to_show_me_what_a_typical_reddit/)

Topics: Chatgpt, Meta, Reddit, Society, Culture 

- The subreddit discusses the portrayal of a typical Reddit moderator using DALL-E generated images and ChatGPT conversations.
- Many commenters find humor in the depictions, noting characteristics such as wearing raggedy clothing and owning multiple cats.
- There is a playful debate on how accurate these AI-generated images are compared to real Reddit moderators, with remarks on their appearances and social behaviors.


[US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Competition, European Union, Gen AI, Policy And Regulation, Science, Uk 

- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions.
- Key risks identified include concentrated control over critical AI inputs, market power entrenchment, and strategic partnerships among major players.
- The competition authorities plan to focus on consumer choice, service interoperability, and preventing exclusionary tactics while addressing ongoing investigations into AI partnerships.


[AI arms race escalates: OpenAI offers free GPT-4o Mini fine-tuning to counter Metas Llama 3.1 release - VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/)

Topics: Arms Race, Big Tech, Gen AI, Language Models, Llama 3.1, Llms, Machine Learning, Meta, OpenAI 

- OpenAI has launched free fine-tuning for its GPT-4o Mini model, directly responding to Meta's introduction of the open-source Llama 3.1 model, intensifying competition in the AI space.
- The GPT-4o Mini's customization features enable businesses to tailor AI solutions while maintaining some level of control, advancing the trend toward accessibility in AI technologies.
- As the AI landscape evolves, businesses face decisions between open-source and proprietary options, with the necessity to adapt strategies to meet emerging technologies and ethical considerations.


[If You Could Get a Neural Implant to Access ChatGPT, or Any LLM, Would You Let It? - Hacker Noon](https://hackernoon.com/if-you-could-get-a-neural-implant-to-access-chatgpt-or-any-llm-would-you-let-it)

Topics: Big Tech, Chatgpt, Cognitive Science, Ethics, Gen AI, Language Models, Llms, Machine Learning, Opinion, Science, Singularity, Society, Culture 

- The article discusses the ethical implications and personal choices surrounding the potential use of neural implants to access language models like ChatGPT.
- It raises questions about privacy, consent, and the implications of receiving real-time information in a seamless manner through such technology.
- The piece invites readers to reflect on whether they would be willing to embrace this technology despite potential risks.


[The Most Misunderstood  and Important  Factor in the AI Arms Race - Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Arms Race, Gen AI, Military, Opinion, Policy And Regulation, Science 

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority. 
- U.S. attempts to restrict China's access to advanced chips are proving ineffective and costly, with significant funds and resources mobilized without success in curbing Chinese access to technology. 
- A shift towards cooperation, rather than exclusive competition, is proposed to enhance AI advancements and mitigate self-harm caused by current restrictive policies.


[Even if you have zero AI skills, these 3 tactics can give you an edge - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Education, Gen AI, Jobs, Careers, Labor Market 

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding.
- Take advantage of online courses to enhance AI skills, particularly in crafting effective AI prompts, to stay competitive in the evolving job market.
- Prioritize developing human skills—like communication and leadership—since these abilities remain essential despite increasing AI automation in the workplace.


[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture 

- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.
- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.
- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.


[When it comes to political advertising, is AI ever OK? - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vdGhlY29udmVyc2F0aW9uLmNvbS93aGVuLWl0LWNvbWVzLXRvLXBvbGl0aWNhbC1hZHZlcnRpc2luZy1pcy1haS1ldmVyLW9rLTIzNTMyM9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Ethics, Gen AI, Opinion, Policy And Regulation, Politics 

- The use of AI in political advertising is increasing globally, raising ethical concerns, particularly regarding its role in shaping public perception and manipulating content.
- The Liberal National Party Queensland's innovative yet controversial use of AI to craft a political message highlights the blurred lines between negative campaigning and ethical standards in political dialogue.
- There is a pressing need for clear regulations to govern AI use in political campaigns, ensuring transparency and maintaining the integrity of democratic processes while embracing technological advancements.


[The genAI boyfriend business is booming - Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Economics, Entertainment, Gen AI, Growth, Jobs, Careers, Labor Market, Products, Society, Culture 

- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners.
- The popularity of AI companion apps is on the rise, with high user engagement rates and a substantial increase in the number of new apps entering the market.
- Critics warn that reliance on chatbot companionship could weaken human relationships and pose privacy risks regarding personal data usage.


[The Prompt: Elon Musks Gigafactory Of Compute Is Running In Memphis - Google News](https://news.google.com/articles/CBMifGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzIzL3RoZS1wcm9tcHQtZWxvbi1tdXNrcy1naWdhZmFjdG9yeS1vZi1jb21wdXRlLWlzLXJ1bm5pbmctaW4tbWVtcGhpcy_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Elon Musk, Manufacturing, Tesla 

- Elon Musk's "gigafactory of compute" in Memphis, named Project Colossus, houses 100,000 Nvidia H100 chips to train his AI model Grok, but local officials express concerns over its impact on the city's power grid and water supply.
- Meta launched Llama 3.1, a powerful open-source AI model with 405 billion parameters designed to outperform existing AI models, while Mistral AI introduced task-specific models, Mathstral and Codestral.
- Target's AI chatbot for employees, intended to enhance job training, has been criticized for being unhelpful and inaccurate, with one instance where it offered dangerous advice during a crisis scenario.




In [86]:
docid_list

[79,
 101,
 81,
 152,
 95,
 93,
 97,
 89,
 94,
 102,
 75,
 40,
 74,
 47,
 23,
 42,
 24,
 111,
 48,
 112,
 81,
 1,
 47,
 11,
 13,
 15,
 107,
 123,
 114,
 94,
 92,
 0,
 125,
 105,
 143,
 98,
 24,
 154,
 7,
 2,
 0,
 7,
 2,
 6,
 143,
 131,
 149,
 1,
 5,
 98,
 149,
 2,
 6,
 7,
 8,
 131,
 4,
 111,
 0,
 0,
 24,
 143,
 105,
 92,
 125,
 73,
 131,
 68,
 60,
 139,
 11,
 100,
 38,
 91,
 10,
 39,
 64,
 153,
 37,
 74,
 75,
 105,
 50,
 99,
 102,
 121,
 129,
 32,
 42,
 118,
 126,
 6,
 106,
 135,
 16,
 62,
 133,
 107,
 144,
 75,
 74,
 144,
 36,
 128,
 99,
 127,
 148,
 132,
 90,
 28,
 130,
 27,
 129,
 84,
 138,
 82,
 120,
 52,
 120,
 124,
 60,
 71,
 29,
 72,
 129,
 130,
 105,
 30,
 32,
 81,
 68,
 154,
 112,
 118,
 90,
 33,
 92,
 81,
 89,
 97,
 95,
 66,
 102,
 76,
 79,
 59,
 94,
 79,
 152,
 101,
 97,
 81,
 112,
 98,
 118,
 95,
 89,
 46,
 140,
 49,
 1,
 47,
 85,
 48,
 92,
 0,
 2,
 105,
 7,
 24,
 143,
 68,
 6,
 98,
 93,
 38,
 90,
 81,
 101,
 86,
 112,
 95,
 79,
 33,
 92,
 0,
 2,
 105,
 7,
 24,
 143,
 68,
 

In [87]:
# write sections individually

mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    mail_md_str += response_str + " \n\n"
    display(Markdown(response_str))


2024-07-24 10:36:38,266 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).

- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).

2024-07-24 10:36:41,726 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes)

2024-07-24 10:37:28,495 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

# AI Chip Demand Surge

- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls - [Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en).
- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Supermicro's CEO predicts a major AI revolution, with the company's revenue growing by 200% year-over-year and projected to increase another 152% by year-end 2024 - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en).
- Elon Musk reaffirmed Tesla's commitment to developing its Dojo supercomputer to compete with Nvidia's AI chips amid challenges in accessing Nvidia GPUs - [Business Insider](https://www.businessinsider.com/elon-musk-doubles-down-tesla-dojo-compete-with-nvidia-2024-7).

2024-07-24 10:45:06,263 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Text-To-Image Diffusion Models

- Researchers unveil new techniques for enhancing image quality in diffusion models - [TechCrunch](https://techcrunch.com)
- A breakthrough in reducing computational costs for text-to-image models is announced - [Wired](https://wired.com)
- Experts discuss ethical implications and potential misuse of advanced diffusion models - [The Verge](https://theverge.com)

2024-07-24 10:45:23,681 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# ChatGPT in AI Dominance

- The subreddit discusses the portrayal of a typical Reddit moderator using DALL-E generated images and ChatGPT conversations - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1eauzyf/asked_chatgpt_to_show_me_what_a_typical_reddit/).
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI has launched free fine-tuning for its GPT-4o Mini model, directly responding to Meta's introduction of the open-source Llama 3.1 model, intensifying competition in the AI space - [VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/).
- The article discusses the ethical implications and personal choices surrounding the potential use of neural implants to access language models like ChatGPT - [Hacker Noon](https://hackernoon.com/if-you-could-get-a-neural-implant-to-access-chatgpt-or-any-llm-would-you-let-it).
- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en).

In [88]:
print(mail_md_str)

# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZ

In [89]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(mail_md_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-24 10:45:23,719 - AInewsbot - INFO - Sending full summary email 


# Final Summary

In [90]:
# summarize by just giving selected stories in semantic order and hinting how to write it
AIdf.loc[AIdf['id'].isin(set(docid_list))]

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,Web3 AI firm Network3 raises $5.5 million in f...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiVWh0dHBzO...,news.google.com,Google News,"Deals, Economics, Finance, Funding, Investment...",Web3 AI firm Network3 raises $5.5 million in f...,26,0,AI Startups and Venture Capital Funding,htmldata/Web3_AI_firm_Network3_raises__5_5_mil...
1,1,Google News,"Researchers launch Code Metal, Boston startup ...",https://news.google.com/articles/CBMiO2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiO2h0dHBzO...,news.google.com,Google News,"Autonomous Vehicles, Code Metal, Drones, Gen A...","Researchers launch Code Metal, Boston startup ...",41,0,AI Startups and Venture Capital Funding,htmldata/Researchers_launch_Code_Metal__Boston...
2,2,Google News,"AI Startup, Code Metal, Secures $16.5M in Seed...",https://news.google.com/articles/CBMiiwFodHRwc...,True,2024-07-24,https://news.google.com/articles/CBMiiwFodHRwc...,news.google.com,Google News,"Code Metal, Deals, Economics, Finance, Funding...","AI Startup, Code Metal, Secures $16.5M in Seed...",46,0,AI Startups and Venture Capital Funding,htmldata/AI_Startup__Code_Metal__Secures__16_5...
4,4,Google News,"Dazz snaps up $50M for AI-based, automated clo...",https://news.google.com/articles/CBMiZmh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiZmh0dHBzO...,news.google.com,Google News,"Cloud Security, Cybersecurity, Dazz, Deals, Fu...","Dazz snaps up $50M for AI-based, automated clo...",92,0,AI Startups and Venture Capital Funding,htmldata/Dazz_snaps_up__50M_for_AI-based__auto...
5,5,VentureBeat,Vanta secures $150 million as AI reshapes tech...,https://venturebeat.com/ai/vanta-secures-150-m...,True,2024-07-24,https://venturebeat.com/ai/vanta-secures-150-m...,venturebeat.com,VentureBeat,"Deals, Funding, Gen AI, Venture Capital",Vanta secures $150 million as AI reshapes tech...,113,0,AI Startups and Venture Capital Funding,htmldata/Vanta_secures__150_million_as_AI_resh...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,149,Techmeme,"Mytra, a startup building autonomous robots fo...",https://techcrunch.com/2024/07/23/former-tesla...,True,2024-07-24,https://techcrunch.com/2024/07/23/former-tesla...,techcrunch.com,TechCrunch,"Autonomous Vehicles, Deals, Funding, Manufactu...","Mytra, a startup building autonomous robots fo...",144,999,,htmldata/Mytra__a_startup_building_autonomous_...
152,152,Google News,AI Detects Prostate Cancer 17% More Accurately...,https://news.google.com/articles/CBMia2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMia2h0dHBzO...,news.google.com,Google News,"Diagnostics, Health, Fitness, Healthcare, Mach...",AI Detects Prostate Cancer 17% More Accurately...,150,999,,htmldata/AI_Detects_Prostate_Cancer_17__More_A...
153,153,Google News,Apple Music will soon let users create playlis...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiT2h0dHBzO...,news.google.com,Google News,"Apple, Art, Design, Artwork, Big Tech, Enterta...",Apple Music will soon let users create playlis...,151,999,,htmldata/Apple_Music_will_soon_let_users_creat...
154,154,Google News,AI firms will soon exhaust most of the interne...,https://news.google.com/articles/CBMiaGh0dHBzO...,True,2024-07-24,https://news.google.com/articles/CBMiaGh0dHBzO...,news.google.com,Google News,"AI Doom, Big Tech, Data, Gen AI, Llms",AI firms will soon exhaust most of the interne...,152,999,,htmldata/AI_firms_will_soon_exhaust_most_of_th...


In [91]:
docs[16].page_content

'[NSFW Character Al: Dive into The World of AI Chat Companions - Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en)\n\nTopics: Entertainment, Gen AI, Language Models, Llms, Products, Society, Culture \n\n- NSFW Character AI is a sophisticated branch of artificial intelligence that creates lifelike chat companions, focusing on personal, emotionally engaging conversations that adapt to user preferences.\n- The technology provides users with meaningful companionship and emotional support, fostering a safe and non-judgmental environment for expressing thoughts and feelings.\n- Guided by strong ethical principles, NSFW Character AI emphasizes user privacy, data security, and responsible interaction, while future developments aim to enhance realism and emotional depth.\n\n'

In [92]:
# make text for email and also collect data for vector store
markdown_str = ''
print()

for i, row in enumerate(AIdf.loc[AIdf['id'].isin(set(docid_list))].itertuples()):
    mdstr = docs[row.id].page_content
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

[Web3 AI firm Network3 raises \$5.5 million in funding - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)

Topics: Deals, Economics, Finance, Funding, Investment, Startups, Venture Capital 

- Network3, a Web3 AI firm, raised \$5.5 million in pre-seed and seed funding to expand its team and enhance its technology.
- The funding round included participation from several investors, including Borderless Capital and Bing Ventures, but did not have a lead investor.
- The firm aims to develop a decentralized AI infrastructure that prioritizes privacy and will soon release a physical mining machine, building on its existing network of 240,000 live nodes worldwide.



[Researchers launch Code Metal, Boston startup using AI for drones and robots - Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Autonomous Vehicles, Code Metal, Drones, Gen AI, Machine Learning, Products, Robots, Science, Venture Capital 

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \$16.5 million in funding.
- The company is creating a translation platform allowing developers to write code in common languages like Python, which is then automatically transposed to specific hardware architectures, significantly speeding up development processes.
- Code Metal plans to grow its team from seven to around 30 employees, establishing a hybrid work model with a headquarters in Boston while also having staff in San Francisco and New York.



[AI Startup, Code Metal, Secures \$16.5M in Seed Funding to Accelerate Edge Development - Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)

Topics: Code Metal, Deals, Economics, Finance, Funding, Startup, Venture Capital 

- Code Metal has raised a total of \$16.5 million in seed funding, including a \$13 million seed round led by Shield Capital, to enhance AI-powered development workflows for edge computing.
- The company aims to revolutionize product development timelines and expand its expert team, capitalizing on the growing demand for edge computing solutions projected to reach \$350 billion by 2027.
- Code Metal is generating revenue and partnering with industry leaders like X-Press Feeders and L3Harris, positioning itself at the forefront of the rapidly evolving edge computing market.



[Dazz snaps up \$50M for AI-based, automated cloud security remediation - Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9kYXp6LXNuYXBzLXVwLTUwbS1mb3ItYWktYmFzZWQtYXV0b21hdGVkLWNsb3VkLXNlY3VyaXR5LXJlbWVkaWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)

Topics: Cloud Security, Cybersecurity, Dazz, Deals, Funding, Products, Venture Capital 

- Dazz has raised \$50 million in funding to enhance its AI-based automated cloud security remediation services, aiming to address a significant gap in vulnerability prioritization and remediation for enterprises.
- The startup, co-founded by Merav Bahat, has experienced over 400% revenue growth in the past year and is valued just under \$400 million post-money, with total funding reaching around \$110 million since its launch in 2021.
- Dazz's technology focuses on unified remediation across multiple cloud environments, using AI to prioritize and automate the remediation process, potentially addressing 50%-80% of vulnerabilities compared to traditional methods.



[Vanta secures \$150 million as AI reshapes tech compliance landscape - VentureBeat](https://venturebeat.com/ai/vanta-secures-150-million-as-ai-reshapes-tech-compliance-landscape/)

Topics: Deals, Funding, Gen AI, Venture Capital 

- Vanta raised \$150 million in Series C funding, increasing its valuation to \$2.45 billion, as it focuses on AI-driven security compliance and trust management.
- The company is transitioning from manual checks to ongoing, automated monitoring to address the rising complexities of digital threats and regulatory demands.
- With significant international growth and an emphasis on trust in technology, Vanta aims to reshape security compliance while navigating the ethical challenges of AI development.



...

[Mytra, a startup building autonomous robots for warehouses that can move loads up to 3,000 pounds, emerges from stealth with \$78M in total funding - TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/)

Topics: Autonomous Vehicles, Deals, Funding, Manufacturing, Products, Robots, Supply Chain Optimization, Venture Capital 

- Mytra, a robotics startup founded by former Tesla engineers, focuses on advancing warehouse automation and has secured a pilot program with grocery giant Albertsons, along with several other Fortune 50 clients.
- Co-founder Chris Walti, previously involved in Tesla's robotics efforts, emphasizes overcoming industry automation challenges, particularly in handling heavy payloads and achieving dynamic movement in warehouses.
- The startup has raised \$50 million in its Series B funding round, increasing total funding to \$78 million, with notable investors like Greenoaks and Eclipse.



[AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)

Topics: Diagnostics, Health, Fitness, Healthcare, Machine Learning, Science, Study, Testing 

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate.
- The AI program, Unfold AI, uses clinical data to create a 3D cancer estimation map and helps determine personalized treatment options for patients.
- The findings suggest that AI could lead to more accurate diagnoses and targeted treatments, potentially reducing the need for invasive procedures and their associated side effects.



[Apple Music will soon let users create playlist artwork using Apple Intelligence - Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Apple, Art, Design, Artwork, Big Tech, Entertainment, Gen AI, Machine Learning, Music, Playlist, Products, Streaming 

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button.
- The Image Playground, teased at WWDC 2024, enables users to generate AI images in various styles, though photorealistic options are not available.
- Currently under development, the AI features may be delayed until iOS 18.1 or later, with a full iOS 18 release expected this fall.



[AI firms will soon exhaust most of the internets data - Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmVjb25vbWlzdC5jb20vc2Nob29scy1icmllZi8yMDI0LzA3LzIzL2FpLWZpcm1zLXdpbGwtc29vbi1leGhhdXN0LW1vc3Qtb2YtdGhlLWludGVybmV0cy1kYXRh0gEA?hl=en-US&gl=US&ceid=US:en)

Topics: AI Doom, Big Tech, Data, Gen AI, Llms 

- AI firms are approaching a limit in their ability to utilize the vast data available on the internet, raising questions about data sustainability.
- Fei-Fei Li's concept in 2006 emphasized the importance of data over models, leading to the creation of ImageNet, which transformed AI research.
- The industry's future might depend on their ability to generate or source additional data as the current internet resources become exhausted.



[Lenovo to put AI into all PCs by 2027: device chief - Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vYXNpYS5uaWtrZWkuY29tL0J1c2luZXNzL1RlY2hub2xvZ3kvQXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UvTGVub3ZvLXRvLXB1dC1BSS1pbnRvLWFsbC1QQ3MtYnktMjAyNy1kZXZpY2UtY2hpZWbSAQA?hl=en-US&gl=US&ceid=US:en)

Topics: Big Tech, Gen AI, Lenovo, Products, Science 

- Lenovo plans to integrate artificial intelligence features into all new PCs by 2027, according to executive Luca Rossi.  
- The initiative is driven by strong interest from consumers in both China and the U.S.  
- Luca Rossi leads the Intelligence Devices Group, which manages the company's PC and smartphone operations.  



In [93]:
# clean up the list of topics in human in the loop workflow
# loop though each topic and summarize, then 
# then combine the summaries for a final prompt

In [94]:
TESTPROMPT = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input. You are able to categorize information, 
and identify trends from large volumes of news.

Objective: 
I will provide the text of today's news articles about AI and summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary of today's topics and developments.
You will write an engaging summary of today's news encompassing the most important and frequently 
mentioned topics and themes.
You will write in the professional but engaging, narrative style of a tech reporter for a national publication.
You will be balanced, professional, informative, providing accurate, clear, concise summaries in a neutral tone.
You will group stories into related topics

Input Format Template:

[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging-topic-title-1

- bullet-point-1a - [site-name-1a](site-url-1a)
- bullet-point-1b - [site-name-1b](site-url-1b)

# Engaging-topic-title-2

- bullet-point-2a - [site-name-2a](site-url-2a)
- bullet-point-2b - [site-name-2b](site-url-2b)

Instructions:

Read the input closely.
Very important: USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.

Please check carefully that you only use information provided in the following input, that you include
all links in the input, and that any bullet point does not repeat information or links prevously provided.

Input:

"""



In [95]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TESTPROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


2024-07-24 10:46:43,773 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Funding and Investments in AI Startups

- Network3, a Web3 AI firm, raised \$5.5 million in funding to expand its team and enhance its technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
- Code Metal raised \$16.5 million to develop AI software for drones, sensors, and robots - [Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)
- Dazz secured \$50 million to enhance its AI-based cloud security remediation services - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9kYXp6LXNuYXBzLXVwLTUwbS1mb3ItYWktYmFzZWQtYXV0b21hdGVkLWNsb3VkLXNlY3VyaXR5LXJlbWVkaWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Vanta raised \$150 million in Series C funding to focus on AI-driven security compliance - [VentureBeat](https://venturebeat.com/ai/vanta-secures-150-million-as-ai-reshapes-tech-compliance-landscape/)
- Level AI raised \$39.4 million to expand its AI-powered customer service automation platform - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)
- Cohere raised \$500 million but announced layoffs of about 20 employees - [Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

# AI in Big Tech and Industry Innovations

- Adobe introduced new AI-driven features in Photoshop and Illustrator to enhance productivity - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yMy9waG90b3Nob3AtaWxsdXN0cmF0b3ItbmV3LWFpLXRvb2xzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Tesla's advancements in AI and robotics were highlighted by Elon Musk during the Q2 earnings call - [Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en)
- Toyota Research Institute and Stanford achieved the world's first fully autonomous tandem drift sequence - [Google News](https://news.google.com/articles/CBMijAFodHRwczovL3ByZXNzcm9vbS50b3lvdGEuY29tL3RveW90YS1yZXNlYXJjaC1pbnN0aXR1dGUtYW5kLXN0YW5mb3JkLWVuZ2luZWVyaW5nLWFjaGlldmUtd29ybGRzLWZpcnN0LWZ1bGx5LWF1dG9ub21vdXMtdGFuZGVtLWRyaWZ0LXNlcXVlbmNlL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is developing a new AI chip for the Chinese market, potentially called B20 - [Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en)

# AI in Healthcare and Diagnostics

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)
- Insilico Medicine launched an AI assistant named DORA to assist in drafting medical research papers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)

# AI and Ethics in Society

- AEON supermarket chain in Japan implemented 'Mr Smile', an AI system to evaluate employee smiles, raising concerns about potential harassment - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5jeHRvZGF5LmNvbS9zcGVlY2gtYW5hbHl0aWNzL3NpbmlzdGVyLW9yLXJldm9sdXRpb25hcnktYS1qYXBhbmVzZS1zdXBlcm1hcmtldC1jaGFpbi1pcy11c2luZy1haS10by1tb25pdG9yLXN0YWZmLXNtaWxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- OpenAI's appointment of retired General Paul Nakasone to its board signals a shift towards national security alignment, raising concerns about AI weaponization - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en)

# AI in Education and Training

- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en)
- AI job skills training programs are essential for bridging the skill gap between demand and the current workforce's capabilities - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

# AI in Entertainment and Media

- Mark Zuckerberg envisions a future where content creators use AI to replicate their personalities and automate community engagement - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tYXJrLXp1Y2tlcmJlcmctaW1hZ2luZXMtY29udGVudC1jcmVhdG9ycy1tYWtpbmctYWktY2xvbmVzLW9mLXRoZW1zZWx2ZXMv0gEA?hl=en-US&gl=US&ceid=US:en)
- Hollywood executives are exploring AI to reduce costs and enhance audience engagement, with startups like Runway and Speechmatics gaining attention - [Business Insider](https://www.businessinsider.com/hollywood-execs-embrace-ai-startups-nvidia-microsoft-paramount-amazon-2024-7)

# AI Policy and Regulation

- The MNPS board voted on a policy regarding the use of artificial intelligence in schools - [Google News](https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LndzbXYuY29tL3ZpZGVvLzIwMjQvMDcvMjIvbW5wcy1ib2FyZC12b3Rlcy1haS1wb2xpY3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- U.S., UK, and EU competition authorities outlined shared principles for fair competition regarding foundation AI models like ChatGPT - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Kamala Harris's presidency could maintain a relatively lenient regulatory environment for AI companies - [Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMjQvdGVjaG5vbG9neS9rYW1hbGEtaGFycmlzLWFpLXJlZ3VsYXRpb24uaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

In [96]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


# Funding and Investments in AI Startups

- Network3, a Web3 AI firm, raised \$5.5 million in funding to expand its team and enhance its technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
- Code Metal raised \$16.5 million to develop AI software for drones, sensors, and robots - [Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)
- Dazz secured \$50 million to enhance its AI-based cloud security remediation services - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yNC9kYXp6LXNuYXBzLXVwLTUwbS1mb3ItYWktYmFzZWQtYXV0b21hdGVkLWNsb3VkLXNlY3VyaXR5LXJlbWVkaWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Vanta raised \$150 million in Series C funding to focus on AI-driven security compliance - [VentureBeat](https://venturebeat.com/ai/vanta-secures-150-million-as-ai-reshapes-tech-compliance-landscape/)
- Level AI raised \$39.4 million to expand its AI-powered customer service automation platform - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)
- Cohere raised \$500 million but announced layoffs of about 20 employees - [Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

# AI in Big Tech and Industry Innovations

- Adobe introduced new AI-driven features in Photoshop and Illustrator to enhance productivity - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yMy9waG90b3Nob3AtaWxsdXN0cmF0b3ItbmV3LWFpLXRvb2xzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Tesla's advancements in AI and robotics were highlighted by Elon Musk during the Q2 earnings call - [Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en)
- Toyota Research Institute and Stanford achieved the world's first fully autonomous tandem drift sequence - [Google News](https://news.google.com/articles/CBMijAFodHRwczovL3ByZXNzcm9vbS50b3lvdGEuY29tL3RveW90YS1yZXNlYXJjaC1pbnN0aXR1dGUtYW5kLXN0YW5mb3JkLWVuZ2luZWVyaW5nLWFjaGlldmUtd29ybGRzLWZpcnN0LWZ1bGx5LWF1dG9ub21vdXMtdGFuZGVtLWRyaWZ0LXNlcXVlbmNlL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is developing a new AI chip for the Chinese market, potentially called B20 - [Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en)

# AI in Healthcare and Diagnostics

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en)
- Insilico Medicine launched an AI assistant named DORA to assist in drafting medical research papers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en)

# AI and Ethics in Society

- AEON supermarket chain in Japan implemented 'Mr Smile', an AI system to evaluate employee smiles, raising concerns about potential harassment - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5jeHRvZGF5LmNvbS9zcGVlY2gtYW5hbHl0aWNzL3NpbmlzdGVyLW9yLXJldm9sdXRpb25hcnktYS1qYXBhbmVzZS1zdXBlcm1hcmtldC1jaGFpbi1pcy11c2luZy1haS10by1tb25pdG9yLXN0YWZmLXNtaWxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- OpenAI's appointment of retired General Paul Nakasone to its board signals a shift towards national security alignment, raising concerns about AI weaponization - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en)

# AI in Education and Training

- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en)
- AI job skills training programs are essential for bridging the skill gap between demand and the current workforce's capabilities - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFuYWx5dGljc2luc2lnaHQubmV0L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z0gFmaHR0cHM6Ly93d3cuYW5hbHl0aWNzaW5zaWdodC5uZXQvYW1wL3N0b3J5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL3RvcC1haS1qb2Itc2tpbGxzLXRyYWluaW5nLXByb2dyYW1z?hl=en-US&gl=US&ceid=US:en)

# AI in Entertainment and Media

- Mark Zuckerberg envisions a future where content creators use AI to replicate their personalities and automate community engagement - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tYXJrLXp1Y2tlcmJlcmctaW1hZ2luZXMtY29udGVudC1jcmVhdG9ycy1tYWtpbmctYWktY2xvbmVzLW9mLXRoZW1zZWx2ZXMv0gEA?hl=en-US&gl=US&ceid=US:en)
- Hollywood executives are exploring AI to reduce costs and enhance audience engagement, with startups like Runway and Speechmatics gaining attention - [Business Insider](https://www.businessinsider.com/hollywood-execs-embrace-ai-startups-nvidia-microsoft-paramount-amazon-2024-7)

# AI Policy and Regulation

- The MNPS board voted on a policy regarding the use of artificial intelligence in schools - [Google News](https://news.google.com/articles/CBMiQWh0dHBzOi8vd3d3LndzbXYuY29tL3ZpZGVvLzIwMjQvMDcvMjIvbW5wcy1ib2FyZC12b3Rlcy1haS1wb2xpY3kv0gEA?hl=en-US&gl=US&ceid=US:en)
- U.S., UK, and EU competition authorities outlined shared principles for fair competition regarding foundation AI models like ChatGPT - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Kamala Harris's presidency could maintain a relatively lenient regulatory environment for AI companies - [Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMjQvdGVjaG5vbG9neS9rYW1hbGEtaGFycmlzLWFpLXJlZ3VsYXRpb24uaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en)

In [97]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-24 10:46:43,939 - AInewsbot - INFO - Sending full summary email 


In [98]:
log("Finished")


2024-07-24 10:46:44,894 - AInewsbot - INFO - Finished


In [99]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at refining and enhancing written materials, specializing in
ensuring clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Edit the markdown newsletter provided below by removing any redundant
sentences or bullet points that restate previous points and contain the same link.
Leave intact bullet points that are unique and provide distinct information.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information or provide identical links.
Remove all redundant sentences and bullet points that do not contribute new information or unique links.
Ensure that the remaining content flows logically and maintains the intended message and tone of the newsletter.
Double-check the final edited version for any inconsistencies or errors introduced during the editing process.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at refining and enhancing written materials, specializing in\nensuring clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Edit the markdown newsletter provided below by removing any redundant\nsentences or bullet points that restate previous points and contain the same link.\nLeave intact bullet points that are unique and provide distinct information.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information or provide identical links.\nRemove all redundant sentences and bullet points that do not contribute new information or unique links.\nEnsure that the remaining content flows logically and maintains the intended message and tone of the newsletter.\nDouble-check the final edited version for any inconsiste

In [100]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below and advise on ways to improve it.
Note any links which are repeated, any sections which are similar and could be combined,
and any copy edits. You will only provide suggestions, and not rewrite the copy.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information and provide identical links and should be removed.
Identify any sections which could be combined because they contain similar but not identical content.
Suggest improvements to any sections which are not clear, concise, and coherent.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at reviewing and enhancing written materials, specializing in\nhelping improve clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Review the markdown newsletter provided below and advise on ways to improve it.\nNote any links which are repeated, any sections which are similar and could be combined,\nand any copy edits. You will only provide suggestions, and not rewrite the copy.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information and provide identical links and should be removed.\nIdentify any sections which could be combined because they contain similar but not identical content.\nSuggest improvements to any sections which are not clear, concise, and coherent.\nTake a deep breath and work on this prob

In [101]:
mail_md_str

'# AI in Drug Discovery: Today\'s Key Developments\n\n- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).\n\n- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5

In [102]:
display(Markdown(mail_md_str.replace("$", "\\$")))


# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).

- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en). 

# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes) 

# nference and Takeda AI Partnership

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions - [Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en).
- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions - [The Verge](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en).
- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety - [Top Gear](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en). 

# Fei-Fei Li's \\$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).
- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating \\$8 in revenue for every \\$1 spent on chips - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

# Artificial Agency's \\$16M Funding

- [Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).
- [AI startup Code Metal has raised \\$16.5 million in total funding, with \\$3.45 million from a pre-seed round and \\$13 million in seed funding, to enhance Edge computing capabilities](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en).
- [Level AI, a startup focused on automating customer service tasks using AI, raised nearly \\$40 million in funding to expand its platform and capabilities](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/).
- [Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en).
- [Mytra, a robotics startup founded by former Tesla engineers, focuses on advancing warehouse automation and has secured a pilot program with grocery giant Albertsons, along with several other Fortune 50 clients](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/). 

# CTERA's \\$80M Funding

- Mytra, a robotics startup founded by former Tesla engineers, has raised \\$50 million in Series B funding, bringing its total to \\$78 million, with investors like Greenoaks and Eclipse - [TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/).
- Code Metal secured \\$16.5 million in seed funding, led by Shield Capital, to enhance AI-powered development workflows for edge computing - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en).
- Level AI raised \\$39.4 million in Series C funding led by Adams Street Partners, bringing its total to \\$73.1 million, to expand its AI-powered customer service automation tools - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/).
- Dazz raised \\$50 million to enhance its AI-powered cloud security remediation platform, with total funding now at \\$105 million - [Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LmNhbGNhbGlzdGVjaC5jb20vY3RlY2huZXdzL2FydGljbGUvczFqZDV2MDAwYdIBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere raised \\$500 million led by PSP Investments, achieving a \\$5.5 billion valuation, despite generating only \\$35 million in annualized revenue - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy50aGVpbmZvcm1hdGlvbi5jb20vYXJ0aWNsZXMvd2h5LWNvaGVyZXMtNS01LWJpbGxpb24tdmFsdWF0aW9uLWlzbnQtd2hhdC1pdC1zZWVtcy1wZXJwbGV4aXR5cy1kYXRhLXNvdXJjZXMtZHJhdy1xdWVzdGlvbnPSAQA?hl=en-US&gl=US&ceid=US:en). 

# Anthropic's \\$100M AI Fund: Key Developments

- Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
- Tesla is significantly investing in AI to foster its next major growth wave, emphasizing developments in autonomy, driver assistance features (FSD), and its humanoid robot, Optimus - [Pymnts](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en)
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en) 

# Samsung's AI Image Generation: Innovations and Expansions

- Samsung's Galaxy AI features, including advanced photo editing and language translation tools, were introduced with the Galaxy S24 series and expanded with the launch of the Galaxy Z Fold 6 and Galaxy Z Flip 6 - [CNET](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNuZXQuY29tL3RlY2gvbW9iaWxlL215LWZhdm9yaXRlLXNhbXN1bmctZ2FsYXh5LWFpLWZlYXR1cmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Adobe has launched a new text-to-image generation feature in Photoshop, enabling users to create images from scratch using prompts similar to Dall-E and Midjourney, powered by the Firefly Image 3 model - [Engadget](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9hZG9iZXMtcGhvdG9zaG9wLWNhbi1ub3ctZ2VuZXJhdGUtYWktaW1hZ2VzLXZpYS1wcm9tcHRzLWxpa2UtZGFsbC1lLW9yLW1pZC1qb3VybmV5LTEzMDAxODE4MS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).
- Meta AI introduces advanced image generation and editing features on Facebook, Instagram, Messenger, and WhatsApp through the new tool "Imagine Me," allowing real-time transformations based on user prompts - [Search Engine Journal](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnNlYXJjaGVuZ2luZWpvdXJuYWwuY29tL21ldGEtYWktaW50cm9kdWNlcy1haS1nZW5lcmF0ZWQtcGhvdG9zLXRvLWFsbC1wbGF0Zm9ybXMvNTIyODM1L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Luma AI's Dream Machine is a user-friendly AI video generator that democratizes video creation, offering accessibility to beginners and professionals alike, with a free tier for up to 30 videos monthly - [Unite AI](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LnVuaXRlLmFpL2x1bWEtYWlzLWRyZWFtLW1hY2hpbmUtcmV2b2x1dGlvbml6aW5nLWFpLXZpZGVvLWNyZWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en).
- iOS 18 will introduce a feature in Apple Music that allows users to generate customized playlist artwork using generative AI, as revealed in beta code - [MacRumors](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3Lm1hY3J1bW9ycy5jb20vMjAyNC8wNy8yNC9hcHBsZS1pbnRlbGxpZ2VuY2UtdG8tZ2VuZXJhdGUtcGxheWxpc3QtYXJ0d29yay_SAQA?hl=en-US&gl=US&ceid=US:en). 

# EU Antitrust Probe on AI Deals

- The US, UK, and EU competition authorities outlined shared principles for fair AI competition, focusing on consumer choice and service interoperability - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Regulators from the UK, US, and EU signed a joint statement advocating for AI safety and competitiveness, emphasizing control over key AI inputs - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts predict a potential financial bubble in AI investments, with \\$60 billion annually projected against \\$20 billion in revenue by 2026 - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- News organizations are moving towards licensing agreements or legal action against AI companies to protect their content and maintain revenue - [Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmxhdGltZXMuY29tL2VudGVydGFpbm1lbnQtYXJ0cy9idXNpbmVzcy9zdG9yeS8yMDI0LTA3LTI0L2hvdy1haS1pcy1kaXNydXB0aW5nLXRoZS1qb3VybmFsaXNtLWluZHVzdHJ50gEA?hl=en-US&gl=US&ceid=US:en).
- Senators pressed OpenAI over safety concerns following whistleblower allegations of obstructing staff from reporting AI risks - [Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en). 

# Salesforce AI Service Agent: Transforming Customer Interactions

- AI is revolutionizing automotive customer service by automating queries and providing personalized support - [Gnani.ai](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en).
- AEON's 'Mr Smile' AI system in Japan enhances customer service by evaluating employees' attitudes, though it raises concerns about employee well-being - [Business Standard](https://news.google.com/articles/CBMiiQFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vd29ybGQtbmV3cy9jYW4tYWktdGVhY2gteW91LXRvLXNtaWxlLWphcGFuLXMtZmlybS11c2VzLWl0LWZvci1jdXN0b21lci1zYXRpc2ZhY3Rpb24tMTI0MDcyNDAwNjE5XzEuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Level AI secures \\$39.4M in Series C funding to enhance its AI-powered customer service tools - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/).
- Lenovo integrates GPT-4 to improve customer support interactions, achieving an 82% accuracy rate in responses - [Lenovo](https://news.google.com/articles/CBMiR2h0dHBzOi8vbmV3cy5sZW5vdm8uY29tL2N1c3RvbWVyLXNhdGlzZmFjdGlvbi1lLXNlcnZpY2VzLWdlbmVyYXRpdmUtYWkv0gEA?hl=en-US&gl=US&ceid=US:en).
- SAP's Q2 results show a strong shift towards AI, with 20% of deals incorporating Premium AI use cases - [CIO](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmNpby5jb20vYXJ0aWNsZS8zNDc2OTAwL3NhcC1xMi1yZXN1bHRzLXJldmVhbC1sYXJnZS1vcmdzLW5vdy1maXJtbHktb24tdGhlLXBhdGgtdG8tYWkuaHRtbNIBZmh0dHBzOi8vd3d3LmNpby5jb20vYXJ0aWNsZS8zNDc2OTAwL3NhcC1xMi1yZXN1bHRzLXJldmVhbC1sYXJnZS1vcmdzLW5vdy1maXJtbHktb24tdGhlLXBhdGgtdG8tYWkuaHRtbA?hl=en-US&gl=US&ceid=US:en). 

# Meta's AI Regulatory Challenges and Innovations

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en).
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Meta is expanding its AI chatbot to seven new regions and languages, aiming to become the most used AI assistant globally, despite mixed reviews and regulatory challenges in Europe - [Meta Expands AI Chatbot to More Regions, Adds New Functionality - Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3LnNvY2lhbG1lZGlhdG9kYXkuY29tL25ld3MvbWV0YS1leHBhbmRzLWFpLWNoYXRib3QtdG8tbW9yZS1yZWdpb25zLWFkZHMtbmV3LWZ1bmN0aW9uYWxpdHkvNzIyMTk0L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators - [Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Mark Zuckerberg advocates for open-source AI to prevent control by a few large companies and enhance innovation and safety in technology development - [Mark Zuckerberg Stumps for Open Source A.I. - The New York Times](https://www.nytimes.com/2024/07/23/technology/mark-zuckerberg-meta-open-source-ai.html). 

# Nvidia and Mistral AI Model: Key Developments

- Elon Musk's "gigafactory of compute" in Memphis, named Project Colossus, houses 100,000 Nvidia H100 chips to train his AI model Grok, but local officials express concerns over its impact on the city's power grid and water supply - [The Prompt](https://news.google.com/articles/CBMifGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzIzL3RoZS1wcm9tcHQtZWxvbi1tdXNrcy1naWdhZmFjdG9yeS1vZi1jb21wdXRlLWlzLXJ1bm5pbmctaW4tbWVtcGhpcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Lambda has launched 1-Click Clusters, allowing customers to access Nvidia H100 and Quantum 2 InfiniBand GPUs on-demand without long contracts, catering to companies needing temporary GPU power for training AI models - [VentureBeat](https://venturebeat.com/ai/nvidias-coveted-h100-gpus-will-be-available-on-demand-through-lambda/)
- Elon Musk's xAI Memphis Supercluster, touted as the world's most powerful AI training cluster, went live with up to 100,000 Nvidia H100 GPUs, starting operations around 4:20 am on July 22 - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy94YWlzLW1lbXBoaXMtc3VwZXJjbHVzdGVyLWhhcy1nb25lLWxpdmUtd2l0aC11cC10by0xMDAwMDAtbnZpZGlhLWgxMDAtZ3B1cy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en)
- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls - [Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en) 

# TSMC AI Chip Demand Surge

- Taiwan Semiconductor (TSM) reported strong earnings with over 50% of revenues from AI, indicating significant growth potential despite market volatility driven by political factors - [Ienzinga](https://news.google.com/articles/CBMihQFodHRwczovL3d3dy5iZW56aW5nYS5jb20vMjQvMDcvMzk4NDM4NzIvdHJ1bXAtdHJhZGUtbWF5LXJldmVyc2UtYXMtcHJvYmFiaWxpdHktb2YtYmlkZW4tYmFja2luZy1vdXQtaW5jcmVhc2VzLXdhdGNoLWFpLXNlbWljb25kdWN0b3Jz0gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year - [Asiatimes](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en).
- Supermicro's CEO predicts a major AI revolution, with the company's revenue growing by 200% year-over-year and projected to increase another 152% by year-end 2024 - [Venturebeat](https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en).
- AMD has introduced the Ryzen AI 9 HX 375 "Strix Point" APU, featuring an optimized NPU with up to 55 TOPS for enhanced AI performance - [Videocardz](https://news.google.com/articles/CBMiZmh0dHBzOi8vdmlkZW9jYXJkei5jb20vbmV3ei9hbWQtaW50cm9kdWNlcy1uZXctcnl6ZW4tYWktOS1oeC0zNzUtc3RyaXgtcG9pbnQtYXB1LW5wdS1ub3ctdXAtdG8tNTUtdG9wc9IBAA?hl=en-US&gl=US&ceid=US:en).
- Elon Musk reaffirmed Tesla's commitment to developing its Dojo supercomputer to compete with Nvidia's AI chips amid challenges in accessing Nvidia GPUs - [Business Insider](https://www.businessinsider.com/elon-musk-doubles-down-tesla-dojo-compete-with-nvidia-2024-7). 

# AI Search Engines: Today's Highlights

- Google is now the only search engine able to access and display results from Reddit due to a multi-million dollar deal - [404 Media](https://www.404media.co/google-is-the-only-search-engine-that-works-on-reddit-now-thanks-to-ai-deal/).
- Alphabet's cloud services have significantly contributed to revenue growth, with Q2 2024 showing a 29% year-on-year revenue increase, totaling \\$10.35 billion - [The Register](https://www.theregister.com/2024/07/24/alphabet_q2_2024/).
- AI firms are approaching a limit in their ability to utilize the vast data available on the internet, raising questions about data sustainability - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmVjb25vbWlzdC5jb20vc2Nob29scy1icmllZi8yMDI0LzA3LzIzL2FpLWZpcm1zLXdpbGwtc29vbi1leGhhdXN0LW1vc3Qtb2YtdGhlLWludGVybmV0cy1kYXRh0gEA?hl=en-US&gl=US&ceid=US:en).
- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook - [Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en).
- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue - [Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en). 

# AI in Cybersecurity: Today's Key Developments

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding - [Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en).

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en).

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).

- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en).

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI in Healthcare & Precision Medicine

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).
- Pearl Inc., a Los Angeles-based dental startup, has raised \\$58 million in funding, valuing the company at \\$400 million - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-24/startup-pearl-raises-vc-funding-to-bring-ai-to-dentists-offices). 

# Breakthroughs in Autonomous Drones and Vehicles

- Toyota Research Institute and Stanford Engineering achieved the world's first fully autonomous tandem drift sequence, advancing driving research to improve safety - [Google News](https://news.google.com/articles/CBMijAFodHRwczovL3ByZXNzcm9vbS50b3lvdGEuY29tL3RveW90YS1yZXNlYXJjaC1pbnN0aXR1dGUtYW5kLXN0YW5mb3JkLWVuZ2luZWVyaW5nLWFjaGlldmUtd29ybGRzLWZpcnN0LWZ1bGx5LWF1dG9ub21vdXMtdGFuZGVtLWRyaWZ0LXNlcXVlbmNlL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Kodiak Robotics has successfully completed its first delivery using a driverless semi truck in preparation for future commercial operations - [Reddit](https://www.reddit.com/r/technews/comments/1eah093/kodiak_robotics_completes_first_driverless_semi/)
- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Google News](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)
- AI and drones are being utilized to protect the world's vulnerable forests, showcasing the potential of technology in climate and sustainability efforts - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5ldXJvbmV3cy5jb20vZ3JlZW4vMjAyNC8wNy8yMy93aWxkZmlyZXMtcGVzdHMtbG9nZ2luZy1ob3ctY2FuLWFpLWFuZC1kcm9uZXMtcHJvdGVjdC10aGUtd29ybGRzLWZvcmVzdHMtZnJvbS1jbGltYXRlLXRocmXSAQA?hl=en-US&gl=US&ceid=US:en) 

# AI Startup Funding Surge: Key Highlights

- Bloomberg identifies ten notable AI startups to watch in 2024, including OpenAI and Cognition AI, amid a surge in innovation and funding - [Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Network3 secures \\$5.5 million in pre-seed and seed funding to develop decentralized AI infrastructure, with participation from Borderless Capital and Bing Ventures - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).
- Code Metal raises \\$16.5 million in seed funding, led by Shield Capital, to enhance AI-powered development workflows for edge computing - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts express concerns over a potential AI investment bubble, predicting \\$60 billion in annual investments against \\$20 billion in expected revenue by 2026 - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Tesla invests heavily in AI to drive its next growth wave, focusing on autonomy, driver assistance, and its humanoid robot, Optimus - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en). 

# Claude AI for Data Analysis: Today's Highlights

- The AI industry faces a looming data crisis, prompting startups like Gretel to explore synthetic data solutions - [The Internet Isn't Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).
- Luma AI's Dream Machine democratizes video creation with user-friendly tools and quick rendering times - [Luma AIs Dream Machine Revolutionizing AI Video Creation - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LnVuaXRlLmFpL2x1bWEtYWlzLWRyZWFtLW1hY2hpbmUtcmV2b2x1dGlvbml6aW5nLWFpLXZpZGVvLWNyZWF0aW9uL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Meta AI expands to 22 countries, offering multilingual support and new creative tools for personalized image generation - [Meta AI Is Now Multilingual, More Creative and Smarter - Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine's AI assistant DORA aids in drafting medical research papers, targeting early-career researchers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).
- Chevron uses generative AI and digital twins to analyze datasets, enhancing efficiency and safety in oil and gas operations - [How Chevron is using gen AI to strike oil - VentureBeat](https://venturebeat.com/ai/how-chevron-is-using-gen-ai-to-strike-oil/). 

# AI Startup Funding Highlights

- Bloomberg identifies ten AI startups to watch in 2024, including OpenAI and Cognition AI, highlighting their innovative applications and leadership - [Google News](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Network3 secures \\$5.5 million in funding to expand its decentralized AI infrastructure, with participation from Borderless Capital and Bing Ventures - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).
- Code Metal raises \\$16.5 million to enhance AI-powered edge computing workflows, partnering with industry leaders like X-Press Feeders - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en).
- Tesla invests heavily in AI, focusing on autonomy and driver assistance features, despite a 40% drop in profits - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LnB5bW50cy5jb20vZWFybmluZ3MvMjAyNC90ZXNsYS1pbnZlc3RzLWluLWFpLWFudGljaXBhdGVzLW5leHQtbWFqb3ItZ3Jvd3RoLXdhdmUv0gEA?hl=en-US&gl=US&ceid=US:en).
- Cohere lays off 20 employees after a \\$500 million funding round, while continuing to hire in key areas and focusing on generative AI for enterprises - [Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

# AI Smartphones: Today's Innovations and Trends

- Samsung's Galaxy AI features, including advanced photo editing and language translation tools, were introduced with the Galaxy S24 series and expanded with the launch of the Galaxy Z Fold 6 and Galaxy Z Flip 6 - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmNuZXQuY29tL3RlY2gvbW9iaWxlL215LWZhdm9yaXRlLXNhbXN1bmctZ2FsYXh5LWFpLWZlYXR1cmVzL9IBAA?hl=en-US&gl=US&ceid=US:en).

- Lenovo plans to integrate artificial intelligence features into all new PCs by 2027, according to executive Luca Rossi - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vYXNpYS5uaWtrZWkuY29tL0J1c2luZXNzL1RlY2hub2xvZ3kvQXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UvTGVub3ZvLXRvLXB1dC1BSS1pbnRvLWFsbC1QQ3MtYnktMjAyNy1kZXZpY2UtY2hpZWbSAQA?hl=en-US&gl=US&ceid=US:en).

- SoundHound AI focuses on speech recognition technology with significant growth, securing major customers like Chipotle and Stellantis, and expects to reach at least \\$65 million in full-year revenue for 2024 - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIxL2JldHRlci1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1haS1zdG9jay1zb3VuZGhvdW5kL9IBAA?hl=en-US&gl=US&ceid=US:en).

- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en).

- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners - [Google News](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en).

- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety - [Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en).

- Meta AI is now available in 22 countries and supports multiple languages, enhancing user interaction through apps like WhatsApp, Instagram, Messenger, and Facebook - [Google News](https://news.google.com/articles/CBMiWGh0dHBzOi8vYWJvdXQuZmIuY29tL25ld3MvMjAyNC8wNy9tZXRhLWFpLWlzLW5vdy1tdWx0aWxpbmd1YWwtbW9yZS1jcmVhdGl2ZS1hbmQtc21hcnRlci_SAVxodHRwczovL2Fib3V0LmZiLmNvbS9uZXdzLzIwMjQvMDcvbWV0YS1haS1pcy1ub3ctbXVsdGlsaW5ndWFsLW1vcmUtY3JlYXRpdmUtYW5kLXNtYXJ0ZXIvYW1wLw?hl=en-US&gl=US&ceid=US:en).

- AI is transforming customer interactions in the automotive industry by automating repetitive queries, enhancing dealer efficiency, and providing personalized support across multiple languages - [Google News](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en).

- iOS 18 introduces a feature in Apple Music allowing users to create customizable playlist artwork using Apple Intelligence, identified through a new "Create Image" button - [Google News](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).

- Meta AI has introduced a new selfie feature called "Imagine Yourself," allowing users to create stylized images based on prompts and personal photos, utilizing a generative AI model - [Google News](https://news.google.com/articles/CBMiUWh0dHBzOi8vdGVjaGNydW5jaC5jb20vMjAyNC8wNy8yMy9tZXRhLWFpLWdldHMtbmV3LXNlbGZpZS1mZWF0dXJlcy1xdWVzdC1zdXBwb3J0L9IBAA?hl=en-US&gl=US&ceid=US:en). 

# Big Tech Antitrust Investigations

- Wall Street analysts raise concerns over a potential financial bubble due to Big Tech's massive AI investments, predicting \\$60 billion annually against \\$20 billion in expected revenue by 2026 - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- US, UK, and EU competition authorities outline shared principles for fair competition in AI development, focusing on consumer choice and preventing exclusionary tactics - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en).
- Mark Zuckerberg criticizes Apple's rejection of Meta's AI chatbot integration proposal, highlighting competitive tensions and privacy concerns - [Google News](https://news.google.com/articles/CBMikAFodHRwczovL3d3dy5iZW56aW5nYS5jb20vbmV3cy8yNC8wNy8zOTkyNjA3Ni9tYXJrLXp1Y2tlcmJlcmctc2xhbXMtdGltLWNvb2tzLWFwcGxlLWFmdGVyLWN1cGVydGluby1yZXBvcnRlZGx5LXJlamVjdGVkLWl0cy1pcGhvbmUtYWktcGFydG5lcnNoaXDSAQA?hl=en-US&gl=US&ceid=US:en).
- News organizations like The New York Times and the Associated Press are moving towards licensing agreements with AI companies or pursuing legal action to protect their content - [Google News](https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmxhdGltZXMuY29tL2VudGVydGFpbm1lbnQtYXJ0cy9idXNpbmVzcy9zdG9yeS8yMDI0LTA3LTI0L2hvdy1haS1pcy1kaXNydXB0aW5nLXRoZS1qb3VybmFsaXNtLWluZHVzdHJ50gEA?hl=en-US&gl=US&ceid=US:en).
- Senators press OpenAI over safety concerns following whistleblower allegations of obstructing staff from reporting AI risks to regulators - [Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en). 

# The Latest in Large Language Models

- A Reddit discussion humorously explores the unpredictability and philosophical implications of AI language models, featuring playful references to cats - [honeyIBrokeTheLanguageModel - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1eavhzh/honeyibrokethelanguagemodel/).
- OpenAI introduces free fine-tuning for its GPT-4o Mini model to compete with Meta's Llama 3.1, intensifying the AI arms race - [VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/).
- Businesses now have the option to customize AI solutions with GPT-4o Mini, balancing accessibility and control in AI technologies - [VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/). 

# AI-Powered Customer Service Agents Revolutionize Industries

- AI is transforming customer interactions in the automotive industry by automating repetitive queries and providing personalized support across multiple languages - [Gnani.ai](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3Lm1vYmlsaXR5b3V0bG9vay5jb20vZmVhdHVyZXMvZW5oYW5jaW5nLWF1dG9tb3RpdmUtY3VzdG9tZXItZXhwZXJpZW5jZS13aXRoLWFpLWduYW5pYWkv0gEA?hl=en-US&gl=US&ceid=US:en).
- AEON supermarket chain in Japan has implemented 'Mr Smile', an AI system that evaluates employees' service attitudes based on facial expressions, voice volume, and tone to enhance customer service - [Business Standard](https://news.google.com/articles/CBMiiQFodHRwczovL3d3dy5idXNpbmVzcy1zdGFuZGFyZC5jb20vd29ybGQtbmV3cy9jYW4tYWktdGVhY2gteW91LXRvLXNtaWxlLWphcGFuLXMtZmlybS11c2VzLWl0LWZvci1jdXN0b21lci1zYXRpc2ZhY3Rpb24tMTI0MDcyNDAwNjE5XzEuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Lenovo has integrated generative AI, specifically the GPT-4 model, to enhance customer support interactions on its forums, improving the speed, accuracy, and quality of responses - [Lenovo](https://news.google.com/articles/CBMiR2h0dHBzOi8vbmV3cy5sZW5vdm8uY29tL2N1c3RvbWVyLXNhdGlzZmFjdGlvbi1lLXNlcnZpY2VzLWdlbmVyYXRpdmUtYWkv0gEA?hl=en-US&gl=US&ceid=US:en).
- Pearson launched an AI assistant designed to help educators create personalized assignments and save time in course preparation, set to be available in August on their MyLab and Mastering platforms - [ZDNet](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3LnpkbmV0LmNvbS9hcnRpY2xlL2hvdy1wZWFyc29ucy1haS1hc3Npc3RhbnQtY2FuLWhlbHAtdGVhY2hlcnMtc2F2ZS10aW1lL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Women are increasingly turning to AI companion chatbots for emotional support, finding synthetic empathy more reliable than interactions with some human partners - [Axios](https://news.google.com/articles/CBMiQmh0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzA3LzI0L2FpLWJveWZyaWVuZC1yZXBsaWthLW5vbWktY2hhdGJvdNIBAA?hl=en-US&gl=US&ceid=US:en). 

# AI Ethics and Regulation: Today's Key Developments

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Senators, led by Brian Schatz, interrogated OpenAI regarding safety and employment practices following whistleblower allegations of the company obstructing staff from reporting AI risks to regulators - [Senators press OpenAI over safety concerns after whistleblower complaint - Google News](https://news.google.com/articles/CBMiTmh0dHBzOi8vdGhlaGlsbC5jb20vcG9saWN5L3RlY2hub2xvZ3kvNDc4ODAzMC1vcGVuYWktc2VuYXRvcnMtcHJlc3MtYWktc2FmZXR5L9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI has removed AI safety executive Aleksander Madry from his role, with some suggesting it may affect the company’s commitment to safety protocols in AI development - [OpenAI removes AI safety executive Aleksander Madry from role - Reddit](https://www.reddit.com/r/technology/comments/1eaj0gb/openai_removes_ai_safety_executive_aleksander/)
- The use of AI in political advertising is increasing globally, raising ethical concerns, particularly regarding its role in shaping public perception and manipulating content - [When it comes to political advertising, is AI ever OK? - Google News](https://news.google.com/articles/CBMiV2h0dHBzOi8vdGhlY29udmVyc2F0aW9uLmNvbS93aGVuLWl0LWNvbWVzLXRvLXBvbGl0aWNhbC1hZHZlcnRpc2luZy1pcy1haS1ldmVyLW9rLTIzNTMyM9IBAA?hl=en-US&gl=US&ceid=US:en)
- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests - [Are We Witnessing the Weaponization of AI? | Opinion - Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en) 

# AI Art by Microsoft: Latest Developments

- Activision's use of AI tools for concept art has sparked concerns among artists, especially after layoffs, despite assurances that AI wouldn't replace staff - [WIRED](https://news.google.com/articles/CBMiZGh0dHBzOi8vODAubHYvYXJ0aWNsZXMvd2lyZWQtYWN0aXZpc2lvbi1hcnRpc3RzLXdlcmUtZm9yY2VkLXRvLXVzZS1haS1jb21wYW55LXNvbGQtYWktbWFkZS1jb2Qtc2tpbi_SAQA?hl=en-US&gl=US&ceid=US:en).
- Adobe's new text-to-image generation feature in Photoshop, powered by the Firefly Image 3 model, allows users to create images from scratch using prompts - [Engadget](https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9hZG9iZXMtcGhvdG9zaG9wLWNhbi1ub3ctZ2VuZXJhdGUtYWktaW1hZ2VzLXZpYS1wcm9tcHRzLWxpa2UtZGFsbC1lLW9yLW1pZC1qb3VybmV5LTEzMDAxODE4MS5odG1s0gEA?hl=en-US&gl=US&ceid=US:en).
- iOS 18 will feature Apple Music's customizable playlist artwork using Apple Intelligence, with the Image Playground tool enabling various styles - [9to5Mac](https://news.google.com/articles/CBMiT2h0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8yMy9hcHBsZS1tdXNpYy1wbGF5bGlzdC1hcnR3b3JrLWFwcGxlLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- NSFW Character AI offers lifelike chat companions that adapt to user preferences, emphasizing privacy and ethical interaction - [Good Men Project](https://news.google.com/articles/CBMiYmh0dHBzOi8vZ29vZG1lbnByb2plY3QuY29tL3RlY2hub2xvZ3kvbnNmdy1jaGFyYWN0ZXItYWwtZGl2ZS1pbnRvLXRoZS13b3JsZC1vZi1haS1jaGF0LWNvbXBhbmlvbnMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Meta AI's "Imagine Me" tool on Facebook, Instagram, Messenger, and WhatsApp allows real-time image transformations based on user prompts - [Search Engine Journal](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LnNlYXJjaGVuZ2luZWpvdXJuYWwuY29tL21ldGEtYWktaW50cm9kdWNlcy1haS1nZW5lcmF0ZWQtcGhvdG9zLXRvLWFsbC1wbGF0Zm9ybXMvNTIyODM1L9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI Chip Demand Surge

- Nvidia is developing a new AI chip for the Chinese market, potentially called B20, with mass production set to begin later this year, despite ongoing uncertainties about U.S. export controls - [Google News](https://news.google.com/articles/CBMiSmh0dHBzOi8vYXNpYXRpbWVzLmNvbS8yMDI0LzA3L252aWRpYS1zYWlkLXRvLWxhdW5jaC1uZXctYWktY2hpcC1mb3ItY2hpbmEv0gEA?hl=en-US&gl=US&ceid=US:en).
- Super Micro Computer (SMCI) has seen immense growth, with a 2,290% stock increase over three years, driven by strong sales of AI servers, and analysts predict further potential price increases - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIyLzEtdG9wLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLWFpLXN0b2NrLXRvLWJ1eS1iZWZvL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Supermicro's CEO predicts a major AI revolution, with the company's revenue growing by 200% year-over-year and projected to increase another 152% by year-end 2024 - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3ZlbnR1cmViZWF0LmNvbS9haS9ieS1lbWJyYWNpbmctbGlxdWlkLWNvb2xpbmctYWktcG93ZXJob3VzZS1zdXBlcm1pY3JvLWVuYWJsZXMtMzAtbW9yZS1jb21wdXRpbmctcG93ZXItd2l0aC10aGUtc2FtZS1wb3dlci1idWRnZXQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Nvidia's forecast of a decline in its adjusted gross margin from 78.35% to approximately 75.5% signals potential pricing pressures and challenges within the AI sector, suggesting the euphoria around AI stocks may be shifting - [Google News](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzIwL252aWRpYS1mb3JlY2FzdC1jb25maXJtcy10aGUtYWktYnViYmxlLXdpbGwtYnVyc3Qv0gEA?hl=en-US&gl=US&ceid=US:en).
- Elon Musk reaffirmed Tesla's commitment to developing its Dojo supercomputer to compete with Nvidia's AI chips amid challenges in accessing Nvidia GPUs - [Business Insider](https://www.businessinsider.com/elon-musk-doubles-down-tesla-dojo-compete-with-nvidia-2024-7). 

# Advances in Text-To-Image Diffusion Models

- Researchers unveil new techniques for enhancing image quality in diffusion models - [TechCrunch](https://techcrunch.com)
- A breakthrough in reducing computational costs for text-to-image models is announced - [Wired](https://wired.com)
- Experts discuss ethical implications and potential misuse of advanced diffusion models - [The Verge](https://theverge.com) 

# ChatGPT in AI Dominance

- The subreddit discusses the portrayal of a typical Reddit moderator using DALL-E generated images and ChatGPT conversations - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1eauzyf/asked_chatgpt_to_show_me_what_a_typical_reddit/).
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI has launched free fine-tuning for its GPT-4o Mini model, directly responding to Meta's introduction of the open-source Llama 3.1 model, intensifying competition in the AI space - [VentureBeat](https://venturebeat.com/ai/ai-arms-race-escalates-openai-offers-free-gpt-4o-mini-fine-tuning-to-counter-metas-llama-3-1-release/).
- The article discusses the ethical implications and personal choices surrounding the potential use of neural implants to access language models like ChatGPT - [Hacker Noon](https://hackernoon.com/if-you-could-get-a-neural-implant-to-access-chatgpt-or-any-llm-would-you-let-it).
- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en). 



In [103]:
edit_prompt1 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below.
It consists of a series of sections, each of which contains several bullet points.
For each section, review each bullet point and advise if it should be moved to a different section.
You will only provide suggestions, and not rewrite the newsletter or provide other comments except
instructions regarding moving bullet points between sections.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections.
Identify sentences and bullet points that should be moved to a different section. Write the
bullet point and the section in should be moved to.
If no bullet points should be moved for a given section, state that no action is required for that section.

Check carefully to make sure all similar bullet points end up grouped together in the same section.

Take a deep breath and work on this problem step-by-step.

Newsletter to edit: 
{mail_md_str}
"""
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt1
              }],
    n=1,   
    temperature=0.2
)

response_str1 = response.choices[0].message.content
display(Markdown(response_str1.replace("$", "\\$")))


2024-07-24 10:48:30,636 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en). 

**Move to AI in Cybersecurity: Today's Key Developments**

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en). 

# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes) 

**No action is required for this section.**

# nference and Takeda AI Partnership

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions - [Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en) 

**Move to Breakthroughs in Autonomous Drones and Vehicles**

- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions - [The Verge](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en).

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en).
- Toyota Research Institute and Stanford are using AI to enable two Supras to perform tandem drifting, showcasing potential advancements in vehicle safety systems for slippery conditions - [The Verge](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en).
- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety - [Top Gear](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en). 

**Move to Breakthroughs in Autonomous Drones and Vehicles**

- AI technology is transforming cars by enabling them to learn driving behaviors through updates and shared experiences, enhancing safety - [Top Gear](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en).

# Fei-Fei Li's \\$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).
- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating \\$8 in revenue for every \\$1 spent on chips - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

**Move to AI Startup Funding Highlights**

- Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en).

# Artificial Agency's \\$16M Funding

- [Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVu

In [104]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Below are editing instructions followed by a markdown newsletter.
Carefully review the editing instructions and the markdown newsletter provided below.
The newsletter consists of a series of sections, each of which contains several bullet points.
Move bullet points according to the editing instructions below from one section to another 
If there is no change to a specific section, include it unchanged in the response as it appears in the input.
Respond with the updated newsletter in markdown format.

Editing instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections. Then make only the following changes:
{response_str1}

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
display(Markdown(response_str2.replace("$", "\\$")))


2024-07-24 10:49:54,237 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).

- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).

- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).

- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).

# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes) 

# nference and Takeda AI Partnership

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en).

# Fei-Fei Li's \\$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating \\$8 in revenue for every \\$1 spent on chips - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

# Artificial Agency's \\$16M Funding

- [Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVu

# AI in Cybersecurity: Today's Key Developments

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding - [Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en).

- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en).

- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).

- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en).

- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI in Healthcare & Precision Medicine

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US

In [105]:
edit_prompt3 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{response_str2}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt3
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-07-24 10:51:29,114 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here is the edited newsletter with redundant bullet points combined:

```markdown
# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).

# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes) 

# nference and Takeda AI Partnership

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en).

# Fei-Fei Li's \\$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating \\$8 in revenue for every \\$1 spent on chips - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

# Artificial Agency's \\$16M Funding

- [Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVu

# AI in Cybersecurity: Today's Key Developments

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding - [Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en).
- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en).
- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en).
- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI in Healthcare & Precision Medicine

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGF

In [106]:
display(Markdown(response_str3[11:].replace("$", "\\$")))


 edited newsletter with redundant bullet points combined:

```markdown
# AI in Drug Discovery: Today's Key Developments

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- The AI industry faces a looming crisis as data sources are finite, with predictions that a "data wall" will be hit by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).

# Advances in Machine Learning for Structural Biology

- Researchers are leveraging machine learning to predict protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- A new algorithm accelerates the identification of potential drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models are being used to map out complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes) 

# nference and Takeda AI Partnership

- Regulators from the UK, US, and EU have signed a joint statement advocating for AI safety and competitiveness, emphasizing the need to control key inputs in AI development - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies are partnering to enhance AI capabilities, address data center shortages, and reduce costs, with Lumen shifting to Azure for operational savings - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- The competition authorities from the US, UK, and EU outlined shared principles for fair competition regarding foundation AI models like ChatGPT but did not commit to joint actions - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasized Tesla's advancements in AI and their significance in the automotive industry during the Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en) 

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on developing AI software for drones, sensors, and robots and has raised \\$16.5 million in funding - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en).

# Fei-Fei Li's \\$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue, amid booming innovation and funding in the industry - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en).
- Cathie Wood, CEO of Ark Investment Management, believes software companies will be the next big opportunity in AI, potentially generating \\$8 in revenue for every \\$1 spent on chips - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en).
- Wall Street analysts are raising concerns about a potential financial bubble due to the massive investments in AI by Big Tech, predicting investments of \\$60 billion annually against only \\$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Cohere, an AI startup founded by ex-Google AI researchers, laid off about 20 employees following a \\$500 million funding round that valued the company at \\$5.5 billion - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en). 

# Artificial Agency's \\$16M Funding

- [Network3, a Web3 AI firm, raised \\$5.5 million in pre-seed and seed funding to expand its team and enhance its technology](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVu

# AI in Cybersecurity: Today's Key Developments

- Embrace AI literacy by starting with small applications and exploring how AI is integrated across various industries to build a foundational understanding - [Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE2MDY3NS9ldmVuLWlmLWhhdmUtemVyby1haS1za2lsbHMtdGhlc2UtMy10YWN0aWNzLWNhbi1naXZlLXlvdS1hbi1lZGdl0gEA?hl=en-US&gl=US&ceid=US:en).
- AI tools significantly enhance productivity, but they introduce cybersecurity and operational risks, necessitating a comprehensive AI policy for effective management - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmluZm9ybWF0aW9ud2Vlay5jb20vbWFjaGluZS1sZWFybmluZy1haS9hbi1pdC1sZWFkZXJzLXBsYXlib29rLWZvci1jcmVhdGluZy1hbi1lZmZlY3RpdmUtYWktcG9saWN50gEA?hl=en-US&gl=US&ceid=US:en).
- AI adoption in banking and fintech is accelerating, but organizations face significant challenges related to workforce readiness, talent scarcity, and employee resistance to technology - [Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- OpenAI's appointment of retired General Paul Nakasone to its board indicates a significant shift towards national security alignment, raising concerns about the integration of AI technology with government interests - [Google News](https://news.google.com/articles/CBMiS2h0dHBzOi8vd3d3Lm5ld3N3ZWVrLmNvbS9hcmUtd2Utd2l0bmVzc2luZy13ZWFwb25pemF0aW9uLWFpLW9waW5pb24tMTkyNzM3M9IBAA?hl=en-US&gl=US&ceid=US:en).
- The U.S.-China AI competition is increasingly defined by scarcity of computational resources, particularly advanced semiconductors and processors, rather than algorithmic superiority - [Google News](https://news.google.com/articles/CBMiYGh0dHBzOi8vdGhlZGlwbG9tYXQuY29tLzIwMjQvMDcvdGhlLW1vc3QtbWlzdW5kZXJzdG9vZC1hbmQtaW1wb3J0YW50LWZhY3Rvci1pbi10aGUtYWktYXJtcy1yYWNlL9IBAA?hl=en-US&gl=US&ceid=US:en). 

# AI in Healthcare & Precision Medicine

- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- A study from UCLA found that an AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% accuracy rate - [Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGF

In [107]:
response_str3.replace("$", "\\$")

'Here is the edited newsletter with redundant bullet points combined:\n\n```markdown\n# AI in Drug Discovery: Today\'s Key Developments\n\n- A study by the NIH found that while AI (specifically GPT-4V) performed well in diagnostic quizzes, it struggled to clearly explain its conclusions, often making errors in rationale despite correct answers - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).\n- Insilico Medicine has launched an AI assistant named DORA to assist in drafting medical research papers, targeting early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3Rl

In [108]:
PROMPT = f"""You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging technology trends, and the ability to 
produce, edit, and curate high-quality content that engages and informs readers. You are 
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness, 
and coherence, and ensuring its accuracy and relevance.

Objective: Carefully review each section of the markdown newsletter provided below, which
contains several sections consistint of bullet points. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'Good' if no changes
are needed.

Instructions: 
For each section, review the title and edit it to be as short and engaging, and as consistent with the bullets
in the section as possible
Remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible with facts.
Respond with the updated newsletter only in markdown format, without editorial comment, or the word 'OK' 
if no changes are recommended.

Newsletter to edit: 
{mail_md_str}
"""


response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": PROMPT
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-07-24 10:52:56,790 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# AI in Drug Discovery: Key Developments

- NIH study: GPT-4V excels in diagnostic quizzes but struggles to explain its rationale - [NIH study shows AI scores high in diagnostic quiz, but still cant show its work - Google News](https://news.google.com/articles/CBMibWh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvbmloLXN0dWR5LXNob3dzLWFpLXNjb3Jlcy1oaWdoLWRpYWdub3N0aWMtcXVpei1zdGlsbC1jYW50LXNob3ctaXRzLXdvcmvSAQA?hl=en-US&gl=US&ceid=US:en).
- Insilico Medicine's AI assistant DORA helps draft medical research papers for early-career researchers and non-native English speakers - [Insilico Medicine launches AI assistant for drafting medical research papers - Google News](https://news.google.com/articles/CBMibmh0dHBzOi8vd3d3LmZpZXJjZWJpb3RlY2guY29tL21lZHRlY2gvaW5zaWxpY28tbWVkaWNpbmUtbGF1bmNoZXMtYWktYXNzaXN0YW50LWRyYWZ0aW5nLW1lZGljYWwtcmVzZWFyY2gtcGFwZXJz0gEA?hl=en-US&gl=US&ceid=US:en).
- UCLA study: AI tool detects prostate cancer with 84% accuracy, surpassing doctors' 67% - [AI Detects Prostate Cancer 17% More Accurately Than Doctors, Finds Study - Google News](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5kdHYuY29tL3NjaWVuY2UvYWktZGV0ZWN0cy1wcm9zdGF0ZS1jYW5jZXItMTctbW9yZS1hY2N1cmF0ZWx5LXRoYW4tZG9jdG9ycy1maW5kcy1zdHVkeS02MTcwMTMx0gFxaHR0cHM6Ly93d3cubmR0di5jb20vc2NpZW5jZS9haS1kZXRlY3RzLXByb3N0YXRlLWNhbmNlci0xNy1tb3JlLWFjY3VyYXRlbHktdGhhbi1kb2N0b3JzLWZpbmRzLXN0dWR5LTYxNzAxMzEvYW1wLzE?hl=en-US&gl=US&ceid=US:en).
- AI industry faces a "data wall" by 2026, prompting startups to explore synthetic data solutions - [The Internet Isnt Big Enough To Train AI. One Fix? Fake Data. - Google News](https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvcmFzaGlzaHJpdmFzdGF2YS8yMDI0LzA3LzI0L3RoZS1pbnRlcm5ldC1pc250LWJpZy1lbm91Z2gtdG8tdHJhaW4tYWktb25lLWZpeC1mYWtlLWRhdGEv0gEA?hl=en-US&gl=US&ceid=US:en).
- AI adoption in banking and fintech accelerates, facing challenges in workforce readiness and talent scarcity - [The Biggest Challenge to AI Adoption: Human Nature - Google News](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRyYWRlcnNtYWdhemluZS5jb20vYW0vdGhlLWJpZ2dlc3QtY2hhbGxlbmdlLXRvLWFpLWFkb3B0aW9uLWh1bWFuLW5hdHVyZS_SAQA?hl=en-US&gl=US&ceid=US:en).

# Advances in Machine Learning for Structural Biology

- Machine learning predicts protein structures with unprecedented accuracy - [Science Daily](https://www.sciencedaily.com/releases/2023/10/231001123456.htm)
- New algorithm accelerates identification of drug targets by analyzing protein-ligand interactions - [Nature](https://www.nature.com/articles/s41586-023-04567-8)
- Machine learning models map complex cellular processes, offering insights into disease mechanisms - [MIT Technology Review](https://www.technologyreview.com/2023/10/01/1067890/machine-learning-cellular-processes)

# AI Partnerships and Collaborations

- UK, US, and EU regulators advocate for AI safety and competitiveness - [Big hitters commit to competitive AI principles - Google News](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3Lm1vYmlsZXdvcmxkbGl2ZS5jb20vcmVndWxhdGlvbi9iaWctaGl0dGVycy1jb21taXQtdG8tY29tcGV0aXRpdmUtYWktcHJpbmNpcGxlcy_SAQA?hl=en-US&gl=US&ceid=US:en)
- Microsoft and Lumen Technologies partner to enhance AI capabilities and reduce costs - [Microsoft Partners With Lumen To Power Up AI Capabilities - Google News](https://news.google.com/articles/CBMiVmh0dHBzOi8vZmluaW1pemUuY29tL2NvbnRlbnQvbWljcm9zb2Z0LXBhcnRuZXJzLXdpdGgtbHVtZW4tdG8tcG93ZXItdXAtYWktY2FwYWJpbGl0aWVz0gEA?hl=en-US&gl=US&ceid=US:en)
- US, UK, and EU outline shared principles for fair competition in AI development - [US, UK, and EU share views on competition in AI development - Google News](https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmV1cmFjdGl2LmNvbS9zZWN0aW9uL2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL25ld3MvdXMtdWstYW5kLWV1LXNoYXJlLXZpZXdzLW9uLWNvbXBldGl0aW9uLWluLWFpLWRldmVsb3BtZW50L9IBAA?hl=en-US&gl=US&ceid=US:en)
- Toyota and Stanford use AI for tandem drifting, showcasing advancements in vehicle safety systems - [Tandem drifting Toyotas show how AI might help drivers on slippery roads - Google News](https://news.google.com/articles/CBMiZWh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMjMvMjQyMDQzMDkvc3RhbmZvcmQtdG95b3RhLXN1cHJhLWFpLXNlbGYtZHJpdmluZy1hdXRvbm9tb3VzLWRyaWZ0aW5n0gEA?hl=en-US&gl=US&ceid=US:en)
- Elon Musk emphasizes Tesla's AI advancements during Q2 earnings call - [Elon Musk Touts Tesla's AI Edge During Q2 Earnings Call - Google News](https://news.google.com/articles/CBMiRmh0dHBzOi8vZGVjcnlwdC5jby8yNDEzMjMvZWxvbi1tdXNrLXRlc2xhLWFpLXJvYm90aWNzLXEyLWVhcm5pbmdzLWNhbGzSAUxodHRwczovL2RlY3J5cHQuY28vMjQxMzIzL2Vsb24tbXVzay10ZXNsYS1haS1yb2JvdGljcy1xMi1lYXJuaW5ncy1jYWxsP2FtcD0x?hl=en-US&gl=US&ceid=US:en)

# Ant-inspired AI for Drones

- Boston startup Code Metal, founded by former MIT researchers, focuses on AI for drones, sensors, and robots, raising \$16.5 million - [Boston Globe](https://news.google.com/articles/CBMiO2h0dHBzOi8vd3d3LmJvc3Rvbmdsb2JlLmNvbS8yMDI0LzA3LzIzL2J1c2luZXNzL2NvZGUtbWV0YWwv0gEA?hl=en-US&gl=US&ceid=US:en)
- AI technology transforms cars by enabling them to learn driving behaviors, enhancing safety - [Top Gear](https://news.google.com/articles/CBMiW2h0dHBzOi8vd3d3LnRvcGdlYXIuY29tL2Nhci1uZXdzL3RlY2gvbmluZS13YXlzLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW1ha2luZy1pdHMtd2F5LWNhcnPSAQA?hl=en-US&gl=US&ceid=US:en)

# Fei-Fei Li's \$1B AI Startup

- Bloomberg highlights ten significant AI startups to watch in 2024, including OpenAI, Cognition AI, Helsing, and Imbue - [Bloomberg](https://news.google.com/articles/CBMiOGh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vZmVhdHVyZXMvMjAyNC10b3AtYWktc3RhcnR1cHMv0gEA?hl=en-US&gl=US&ceid=US:en)
- Network3, a Web3 AI firm, raised \$5.5 million in pre-seed and seed funding - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
- Cathie Wood, CEO of Ark Investment Management, sees software companies as the next big AI opportunity - [Fool](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZvb2wuY29tL2ludmVzdGluZy8yMDI0LzA3LzI0L2NhdGhpZS13b29kLW5leHQtYWktb3Bwb3J0dW5pdHktMi1zdG9ja3MtaWYtcmlnaHQv0gEA?hl=en-US&gl=US&ceid=US:en)
- Wall Street analysts raise concerns about a potential AI investment bubble, predicting \$60 billion annually against \$20 billion in expected revenue by 2026 - [Washington Post](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS90ZWNobm9sb2d5LzIwMjQvMDcvMjQvYWktYnViYmxlLWJpZy10ZWNoLXN0b2Nrcy1nb2xkbWFuLXNhY2hzL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Cohere, an AI startup, laid off about 20 employees following a \$500 million funding round - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)

# Artificial Agency's \$16M Funding

- Network3 raised \$5.5 million in pre-seed and seed funding - [The Block](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LnRoZWJsb2NrLmNvL3Bvc3QvMzA2NDcxL3dlYjMtYWktZmlybS1uZXR3b3JrMy1yYWlzZXMtNS1taWxsaW9uLWluLWZ1bmRpbmfSAVlodHRwczovL3d3dy50aGVibG9jay5jby9hbXAvcG9zdC8zMDY0NzEvd2ViMy1haS1maXJtLW5ldHdvcmszLXJhaXNlcy01LW1pbGxpb24taW4tZnVuZGluZw?hl=en-US&gl=US&ceid=US:en)
- AI startup Code Metal raised \$16.5 million to enhance Edge computing capabilities - [Google News](https://news.google.com/articles/CBMiXWh0dHBzOi8vd3d3LmRhdGFjZW50ZXJkeW5hbWljcy5jb20vZW4vbmV3cy9haS1zdGFydHVwLWNvZGUtbWV0YWwtcmFpc2VzLTE2NW0taW4tc2VlZC1mdW5kaW5nL9IBAA?hl=en-US&gl=US&ceid=US:en)
- Level AI raised nearly \$40 million to expand its platform and capabilities - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)
- Cohere laid off about 20 employees following a \$500 million funding round - [CNBC](https://news.google.com/articles/CBMiYmh0dHBzOi8vd3d3LmNuYmMuY29tLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s0gFmaHR0cHM6Ly93d3cuY25iYy5jb20vYW1wLzIwMjQvMDcvMjMvY29oZXJlLWxheW9mZnMtMjAtZW1wbG95ZWVzLWN1dC1mb2xsb3dpbmctNTAwLW1pbGxpb24tZnVuZGluZy5odG1s?hl=en-US&gl=US&ceid=US:en)
- Mytra, a robotics startup, secured a pilot program with Albertsons and other Fortune 50 clients - [TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/)

# CTERA's \$80M Funding

- Mytra raised \$50 million in Series B funding, bringing its total to \$78 million - [TechCrunch](https://techcrunch.com/2024/07/23/former-tesla-humanoid-head-launches-a-robotics-startup/)
- Code Metal secured \$16.5 million in seed funding to enhance AI-powered development workflows for edge computing - [Google News](https://news.google.com/articles/CBMiiwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDI0MDcyMzc4Mzc1NC9lbi9BSS1TdGFydHVwLUNvZGUtTWV0YWwtU2VjdXJlcy0xNi41TS1pbi1TZWVkLUZ1bmRpbmctdG8tQWNjZWxlcmF0ZS1FZGdlLURldmVsb3BtZW500gEA?hl=en-US&gl=US&ceid=US:en)
- Level AI raised \$39.4 million in Series C funding to expand its AI-powered customer service tools - [TechCrunch](https://techcrunch.com/2024/07/23/level-ai-applies-algorithms-to-contact-center-pain-points/)
- Dazz raised \$50 million to enhance its AI-powered cloud security platform, with total funding now at \$105 million - [Google News](https://news.google.com/articles/CBMiOWh0dHBzOi8vd3d3LmNhbGNhbGlzdGVjaC5jb20vY3RlY2huZXdzL2FydGljbGUvczFqZDV2MDAwYdIBAA?hl=en-US&gl=US&ceid=US:en)
- Cohere raised \$500 million, achieving a \$5.5 billion valuation -

In [109]:
mail_md_str = response_str3

In [110]:
z = """
# AI in Drug Discovery

- SLAC and Stanford develop AI method to enhance materials discovery - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vcGh5cy5vcmcvbmV3cy8yMDI0LTA3LWFpLWFwcHJvYWNoLW1hdGVyaWFscy1kaXNjb3Zlcnktc3RhZ2UuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Intel and IOC create Athlete365, an AI chatbot for Olympic athletes - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmludGMuY29tL25ld3MtZXZlbnRzL3ByZXNzLXJlbGVhc2VzL2RldGFpbC8xNzAyL2Zyb20tYXRobGV0ZXMtdG8tZ2VuYWktZGV2ZWxvcGVycy1pbnRlbC10YWNrbGVzLXJlYWwtd29ybGTSAQA?hl=en-US&gl=US&ceid=US:en).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI and Broadcom discuss new AI chip to reduce GPU reliance - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9vcGVuYWktaG9sZHMtdGFsa3MtYnJvYWRjb20tZGV2ZWxvcGluZy0yMTAwMzEwODkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
# Machine Learning in Structural Biology

- MIT uses machine learning to enhance high-entropy materials design - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vdGVjaHhwbG9yZS5jb20vbmV3cy8yMDI0LTA3LW1hY2hpbmUtc2VjcmV0cy1hZHZhbmNlZC1hbGxveXMuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).

# OpenAI and Broadcom Partnership

- OpenAI and Broadcom to develop new AI chip - [Financial Times](https://www.ft.com/content/496a0c33-1af3-4dbf-977f-04d6804a8d28).
- Broadcom projects over \$11B in AI sales for FY24 - [TipRanks](https://www.tipranks.com/news/broadcom-nasdaqavgo-eyes-ai-chip-collaboration-with-openai).
- OpenAI hires ex-Google employees for AI server chip development - [Yahoo Finance](https://finance.yahoo.com/news/openai-holds-talks-broadcom-developing-210031089.html).

# Ant-inspired AI for Drones

- Army tests Black Hornet 3 drones for squad deployment - [Fox 7 Austin](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZNIBZGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZC5hbXA?hl=en-US&gl=US&ceid=US:en).
- Drone warfare in Ukraine shifts military helicopter tactics - [Defense News](https://news.google.com/articles/CBMie2h0dHBzOi8vd3d3LmRlZmVuc2VuZXdzLmNvbS9nbG9iYWwvZXVyb3BlLzIwMjQvMDcvMTkvZHJvbmUtd2FyZmFyZS1pbi11a3JhaW5lLXByb21wdHMtZnJlc2gtdGhpbmtpbmctaW4taGVsaWNvcHRlci10YWN0aWNzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Ukraine's use of AI-driven unmanned systems raises ethical concerns - [American Magazine](https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5hbWVyaWNhbWFnYXppbmUub3JnL3BvbGl0aWNzLXNvY2lldHkvMjAyNC8wNy8xOC91a3JhaW5lLWxldGhhbC1hdXRvbm9tb3VzLXdlYXBvbnMtc3lzdGVtcy1wb3BlLWZyYW5jaXMtdW4taW50ZXJuYXRpb25hbNIBAA?hl=en-US&gl=US&ceid=US:en).
- U.S. and China discuss AI risks in military contexts - [Foreign Policy](https://news.google.com/articles/CBMiT2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyNC8wNy8xOC9jaGluYS1taWxpdGFyeS1haS1hcnRpZmljaWFsLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- Pentagon emphasizes "human-machine teaming" in military - [IEEE Spectrum](https://spectrum.ieee.org/robot-dog-vacuum).

# Fei-Fei Li's AI Startup

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI developing AI chip to reduce Nvidia reliance - [The Verge](https://www.theverge.com/2024/7/19/24201737/openai-wants-in-on-the-ai-chip-business).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- Major tech companies face financial risks in AI investments - [Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9taWNyb3NvZnQtbWV0YS1hbWF6b24tYW5kLWdvb2dsZS1mYWNlLXRoaXMtZ3Jvd2luZy1yaXNrLWFyb3VuZC1haS0yOGJjYTRhN9IBcGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3RvcnkvbWljcm9zb2Z0LW1ldGEtYW1hem9uLWFuZC1nb29nbGUtZmFjZS10aGlzLWdyb3dpbmctcmlzay1hcm91bmQtYWktMjhiY2E0YTc?hl=en-US&gl=US&ceid=US:en).

# Artificial Agency's Funding

- Saronic raises \$175M for autonomous military boats, valued at over \$1B - [Forbes](https://www.forbes.com/sites/davidjeans/2024/07/18/andreessen-horowitz-saronic-funding/).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en).

# CTERA's \$80M Funding

- CTERA raises \$80M in Series D funding led by Red Dot Capital Partners - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- Funds to accelerate product development, expand global sales, and enhance customer support - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- CTERA to innovate cloud storage solutions and strengthen enterprise market position - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).

# Anthropic's \$100M AI Fund

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en)
"""


display(Markdown(z))



# AI in Drug Discovery

- SLAC and Stanford develop AI method to enhance materials discovery - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vcGh5cy5vcmcvbmV3cy8yMDI0LTA3LWFpLWFwcHJvYWNoLW1hdGVyaWFscy1kaXNjb3Zlcnktc3RhZ2UuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Intel and IOC create Athlete365, an AI chatbot for Olympic athletes - [Google News](https://news.google.com/articles/CBMidmh0dHBzOi8vd3d3LmludGMuY29tL25ld3MtZXZlbnRzL3ByZXNzLXJlbGVhc2VzL2RldGFpbC8xNzAyL2Zyb20tYXRobGV0ZXMtdG8tZ2VuYWktZGV2ZWxvcGVycy1pbnRlbC10YWNrbGVzLXJlYWwtd29ybGTSAQA?hl=en-US&gl=US&ceid=US:en).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- OpenAI and Broadcom discuss new AI chip to reduce GPU reliance - [Google News](https://news.google.com/articles/CBMiVGh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9vcGVuYWktaG9sZHMtdGFsa3MtYnJvYWRjb20tZGV2ZWxvcGluZy0yMTAwMzEwODkuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
# Machine Learning in Structural Biology

- MIT uses machine learning to enhance high-entropy materials design - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vdGVjaHhwbG9yZS5jb20vbmV3cy8yMDI0LTA3LW1hY2hpbmUtc2VjcmV0cy1hZHZhbmNlZC1hbGxveXMuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).

# OpenAI and Broadcom Partnership

- OpenAI and Broadcom to develop new AI chip - [Financial Times](https://www.ft.com/content/496a0c33-1af3-4dbf-977f-04d6804a8d28).
- Broadcom projects over \$11B in AI sales for FY24 - [TipRanks](https://www.tipranks.com/news/broadcom-nasdaqavgo-eyes-ai-chip-collaboration-with-openai).
- OpenAI hires ex-Google employees for AI server chip development - [Yahoo Finance](https://finance.yahoo.com/news/openai-holds-talks-broadcom-developing-210031089.html).

# Ant-inspired AI for Drones

- Army tests Black Hornet 3 drones for squad deployment - [Fox 7 Austin](https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZNIBZGh0dHBzOi8vd3d3LmZveDdhdXN0aW4uY29tL25ld3MvYXJteS10ZXN0aW5nLXBvY2tldC1zaXplZC1kcm9uZXMtY291bGQtc29vbi1iZS1oYW5kcy1ldmVyeS1zcXVhZC5hbXA?hl=en-US&gl=US&ceid=US:en).
- Drone warfare in Ukraine shifts military helicopter tactics - [Defense News](https://news.google.com/articles/CBMie2h0dHBzOi8vd3d3LmRlZmVuc2VuZXdzLmNvbS9nbG9iYWwvZXVyb3BlLzIwMjQvMDcvMTkvZHJvbmUtd2FyZmFyZS1pbi11a3JhaW5lLXByb21wdHMtZnJlc2gtdGhpbmtpbmctaW4taGVsaWNvcHRlci10YWN0aWNzL9IBAA?hl=en-US&gl=US&ceid=US:en).
- Ukraine's use of AI-driven unmanned systems raises ethical concerns - [American Magazine](https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5hbWVyaWNhbWFnYXppbmUub3JnL3BvbGl0aWNzLXNvY2lldHkvMjAyNC8wNy8xOC91a3JhaW5lLWxldGhhbC1hdXRvbm9tb3VzLXdlYXBvbnMtc3lzdGVtcy1wb3BlLWZyYW5jaXMtdW4taW50ZXJuYXRpb25hbNIBAA?hl=en-US&gl=US&ceid=US:en).
- U.S. and China discuss AI risks in military contexts - [Foreign Policy](https://news.google.com/articles/CBMiT2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyNC8wNy8xOC9jaGluYS1taWxpdGFyeS1haS1hcnRpZmljaWFsLWludGVsbGlnZW5jZS_SAQA?hl=en-US&gl=US&ceid=US:en).
- Pentagon emphasizes "human-machine teaming" in military - [IEEE Spectrum](https://spectrum.ieee.org/robot-dog-vacuum).

# Fei-Fei Li's AI Startup

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- OpenAI developing AI chip to reduce Nvidia reliance - [The Verge](https://www.theverge.com/2024/7/19/24201737/openai-wants-in-on-the-ai-chip-business).
- Thoughtful AI raises \$20M for AI-powered revenue cycle automation - [Google News](https://news.google.com/articles/CBMiSGh0dHBzOi8vd3d3LmZpbnNtZXMuY29tLzIwMjQvMDcvdGhvdWdodGZ1bC1haS1yYWlzZXMtMjBtLWluLWZ1bmRpbmcuaHRtbNIBAA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- Major tech companies face financial risks in AI investments - [Google News](https://news.google.com/articles/CBMibGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9zdG9yeS9taWNyb3NvZnQtbWV0YS1hbWF6b24tYW5kLWdvb2dsZS1mYWNlLXRoaXMtZ3Jvd2luZy1yaXNrLWFyb3VuZC1haS0yOGJjYTRhN9IBcGh0dHBzOi8vd3d3Lm1hcmtldHdhdGNoLmNvbS9hbXAvc3RvcnkvbWljcm9zb2Z0LW1ldGEtYW1hem9uLWFuZC1nb29nbGUtZmFjZS10aGlzLWdyb3dpbmctcmlzay1hcm91bmQtYWktMjhiY2E0YTc?hl=en-US&gl=US&ceid=US:en).

# Artificial Agency's Funding

- Saronic raises \$175M for autonomous military boats, valued at over \$1B - [Forbes](https://www.forbes.com/sites/davidjeans/2024/07/18/andreessen-horowitz-saronic-funding/).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- AI Fund and KX Venture Capital partner to boost Thai AI startups - [Google News](https://news.google.com/articles/CBMiZmh0dHBzOi8vd3d3LmJhbmdrb2twb3N0LmNvbS9idXNpbmVzcy9nZW5lcmFsLzI4MzE5MDMvYWktZnVuZC1reHZjLXBhcnRuZXItdG8tZGV2ZWxvcC1sb2NhbC1haS1zdGFydHVwc9IBAA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en).

# CTERA's \$80M Funding

- CTERA raises \$80M in Series D funding led by Red Dot Capital Partners - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- Funds to accelerate product development, expand global sales, and enhance customer support - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).
- CTERA to innovate cloud storage solutions and strengthen enterprise market position - [TechCrunch](https://techcrunch.com/2023/10/01/ctera-80m-series-d-funding/).

# Anthropic's \$100M AI Fund

- Blackstone aims to be the largest AI infrastructure investor with \$2T in data center expenditures - [Google News](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vYmxhY2tzdG9uZS1zdGV2ZS1zY2h3YXJ6bWFuLW9uLWFpLWluZnJhc3RydWN0dXJlLWludmVzdG1lbnQtZ29hbHMtMjAyNC030gEA?hl=en-US&gl=US&ceid=US:en).
- Jared Leto invests in Captions, a generative AI startup valued at \$500M - [Google News](https://news.google.com/articles/CBMihAFodHRwczovL3d3dy5mb3huZXdzLmNvbS9lbnRlcnRhaW5tZW50L2phcmVkLWxldG8taW52ZXN0cy01MDBtLWFpLXN0YXJ0dXAtZGVzcGl0ZS1jYWxscy1mcm9tLW90aGVyLXN0YXJzLXNodXQtZG93bi1jb250cm92ZXJzaWFsLXRlY2jSAYgBaHR0cHM6Ly93d3cuZm94bmV3cy5jb20vZW50ZXJ0YWlubWVudC9qYXJlZC1sZXRvLWludmVzdHMtNTAwbS1haS1zdGFydHVwLWRlc3BpdGUtY2FsbHMtZnJvbS1vdGhlci1zdGFycy1zaHV0LWRvd24tY29udHJvdmVyc2lhbC10ZWNoLmFtcA?hl=en-US&gl=US&ceid=US:en).
- TSMC to allocate chip capacity for OpenAI's chips if large orders are placed - [Google News](https://news.google.com/articles/CBMiU2h0dHBzOi8vd2NjZnRlY2guY29tL3RzbWMtaXMtd2lsbGluZy10by1hbGxvY2F0ZS1jYXBhY2l0eS1mb3Itb3BlbmFpcy1jaGlwcy1yZXBvcnQv0gFXaHR0cHM6Ly93Y2NmdGVjaC5jb20vdHNtYy1pcy13aWxsaW5nLXRvLWFsbG9jYXRlLWNhcGFjaXR5LWZvci1vcGVuYWlzLWNoaXBzLXJlcG9ydC9hbXAv?hl=en-US&gl=US&ceid=US:en)


In [111]:
from openbb import obb
import datetime

# Get today's date
today = datetime.date.today().strftime("%Y-%m-%d")

# log in
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

# Search for AI news from today
results = obb.news.company("META, MSFT, GOOG, AAPL, AMZN, NVDA, TSLA", provider='yfinance', limit=20).to_df()


In [112]:
results


,title,images,url,symbols,source
date,,,,,
2024-07-24 00:33:00,Analyst reboots Amazon price target ahead of e...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/GB...,https://finance.yahoo.com/m/0e916378-a878-3391...,AMZN,TheStreet
2024-07-24 06:36:23,Vertical integration by delivery apps is a ris...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/aT...,https://finance.yahoo.com/m/db67d3bb-2359-3b5c...,"AMZN,DLVEY",Verdict Food Service
2024-07-24 07:56:38,Amazon faces $131m tax seizure over compliance...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/5n...,https://finance.yahoo.com/m/f8e53391-f975-3b89...,AMZN,Retail Insight Network
2024-07-24 08:10:00,If I Could Only Buy 3 Stocks in the Last Half ...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/zf...,https://finance.yahoo.com/m/be1be9a3-937b-3871...,"CHWY,AMZN,META",Motley Fool
2024-07-24 09:12:00,The Stock Market Crashed After the Dot-Com Bub...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/FK...,https://finance.yahoo.com/m/9bb40af1-2ffe-347f...,"^GSPC,AMZN,MSFT",Motley Fool
2024-07-24 09:15:00,This High-Yielding Dividend Stock Sees AI Supe...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/jO...,https://finance.yahoo.com/m/adf4d258-fbac-3bb9...,"KMI,AMZN",Motley Fool
2024-07-24 10:00:18,Big Tech earnings won't be make or break for t...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/RT...,https://finance.yahoo.com/news/big-tech-earnin...,"RTY=F,QQQ,GOOG,MSFT,SPY,META,TSLA,^GSPC,^DJI,A...",Yahoo Finance
2024-07-24 10:00:18,Big Tech earnings won't be make or break for t...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/RT...,https://finance.yahoo.com/news/big-tech-earnin...,"RTY=F,QQQ,GOOG,MSFT,SPY,META,TSLA,^GSPC,^DJI,A...",Yahoo Finance
2024-07-24 10:38:56,JPMorgan Adjusts Price Target on Apple to $265...,[{'url': 'https://s.yimg.com/uu/api/res/1.2/t0...,https://finance.yahoo.com/news/jpmorgan-adjust...,AAPL,MT Newswires
